# AInewsbot.ipynb - Automate collecting daily AI news

1. initial scrape of front pages of tech sites
  - Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
  - Save HTML of each URL in htmldata directory  
  - Extract URLs from all files, create a pandas dataframe with url, title, src

2. Filter and clean to AI-related headlines not seen before
  - Use ChatGPT prompt to filter only AI-related headlines by sending a prompt and formatted table of headlines
  - Use SQLite to filter headlines previously seen 
  - remove duplicate URLs and headlines
  - ensure there are pretty source names for each news site

3. Topic analysis, make a list of topics for each headline
  - using a prompt, check each headline against a number of evergreen AI topics, e.g. deepfakes, regulation, AI in education
  - extract free from topics from each headline
  - combine topics into topic list for each headline
  - cluster headlines using dimensionality-reduced embeddings and DBSCAN; ask chatgpt to name each cluster
  - sort headlines by doing a traveling salesman shortest traversal in embedding space

4. Summarize individual news story pages in 3 bullets using a prompt

5. create a large markdown file with all bullet points and topics

6. give the markdown file to ChatGPT and ask it to make a list of most popular and import topics of the day

7. human should make a list of the day's topics, combining the chatgpt response to the quesion and cluster topics and 

8. Put summaries in vector store along with metadata. For each topic, retrieve all associated stories and have chatgpt write a digest of those stories in the given format.

9. assemble stories into first draft of newsletter for rewriting as necessary

todo:

use langgraph for final editing workfow
1. prompt to edit final copy for dupes, combine similar sections, copy edit
2. have a reviewer prompt check if there are any bullet points to move to a different section 
3. if so have an editor prompt remove them , return to 2. until nothing left to move dupes left
4. have a reviewer check each section, identify bullet points that are similar to other bullet points in the section and have identical links. rewrite combining so there is no duplication. 
5. identify sections that are short or similar to other sections and suggest sections that should be combined them
6. have an editor prompt merge short sections, return to 4, until no orphan sections left
7. maybe final copy-edit prompt

Original, alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


In [1]:
# import sys
# del sys.modules['ainb_llm']


In [2]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter
import shutil

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
import sklearn
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
import requests
from urllib.parse import urlparse
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import langchain
from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

import openai
from openai import OpenAI

import langchain

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, get_google_news_redirects)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, 
                      fetch_openai_summary, trunc_tokens, categorize_headline)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


In [3]:
print(f"LangChain         {langchain.__version__}")
print(f"OpenAI            {openai.__version__}")
# print(f"smtplib           {smtplib.sys.version}")
print(f"trafilatura       {trafilatura.__version__}")
print(f"bs4               {bs4.__version__}")
print(f"numpy             {np.__version__}")
print(f"pandas            {pd.__version__}")
print(f"sklearn           {sklearn.__version__}")
print(f"umap              {umap.__version__}")


LangChain         0.1.20
OpenAI            1.40.3
trafilatura       1.11.0
bs4               4.12.3
numpy             1.26.4
pandas            2.2.1
sklearn           1.4.2
umap              0.5.6


# Initialize

In [4]:
# before_date = '2024-08-01 18:00:00'
before_date = None


In [5]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [6]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-08-25 07:53:50,315 - AInewsbot - INFO - Load 16 sources from sources.yaml
2024-08-25 07:53:50,317 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-08-25 07:53:50,318 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-08-25 07:53:50,319 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-08-25 07:53:50,319 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-08-25 07:53:50,320 - AInewsbot - INFO - Google News -> https://news.google.com/topics/CAAqJggKIiBDQkFTRWdvSUwyMHZNRGRqTVhZU0FtVnVHZ0pWVXlnQVAB?hl=en-US

In [7]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyL

In [8]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [9]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))
    

2024-08-25 07:53:50,391 - AInewsbot - INFO - 0


In [10]:
log(f"{len(files)} files found")


2024-08-25 07:53:50,395 - AInewsbot - INFO - 0 files found


In [11]:
try:
    os.remove('bullets.md')
    os.remove('headlines.html')
    print(f"Files deleted successfully.")
except FileNotFoundError as e:
    print(e)

[Errno 2] No such file or directory: 'bullets.md'


# Fetch and save source pages

In [12]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-08-25 07:53:50,512 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-08-25 07:53:50,529 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 07:53:50,531 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 07:53:50,531 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 07:54:08,942 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-08-25 07:54:08,942 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-08-25 07:54:08,943 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-08-25 07:54:08,943 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-08-25 07:54:08,944 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-08-25 07:54:08,944 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-08-25 07:55:07,908 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-08-25 07:55:07,902 - AInewsbot - INFO - get_dr

2024-08-25 07:55:52,675 - AInewsbot - INFO - Processing Reddit
2024-08-25 07:55:52,675 - AInewsbot - INFO - get_files(top scoring links _ multi) - starting get_files https://www.reddit.com/r/ChatGPT+ChatGPTCoding+MacOS+MachineLearning+OpenAI+ProgrammerHumor+Windows10+battlestations+buildapc+cordcutters+dataisbeautiful+gadgets+hardware+linux+msp+programming+realtech+software+talesfromtechsupport+tech+technews+technology+techsupportgore+windows/top/?sort=top&t=day
2024-08-25 07:55:54,342 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/conten

In [13]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log(f"{sourcename} -> {page}")
    

2024-08-25 07:56:50,587 - AInewsbot - INFO - Saved 16 pages
2024-08-25 07:56:50,589 - AInewsbot - INFO - Business Insider -> htmldata/Tech - Business Insider (08_25_2024 07_55_18 AM).html
2024-08-25 07:56:50,589 - AInewsbot - INFO - Feedly AI -> htmldata/Discover and Add New Feedly AI Feeds (08_25_2024 07_56_39 AM).html
2024-08-25 07:56:50,589 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (08_25_2024 07_55_21 AM).html
2024-08-25 07:56:50,589 - AInewsbot - INFO - Hacker News -> htmldata/Hacker News Page 1 (08_25_2024 07_55_32 AM).html
2024-08-25 07:56:50,589 - AInewsbot - INFO - Hacker News 2 -> htmldata/Hacker News Page 2 (08_25_2024 07_55_42 AM).html
2024-08-25 07:56:50,589 - AInewsbot - INFO - NYT Tech -> htmldata/Technology - The New York Times (08_25_2024 07_55_54 AM).html
2024-08-25 07:56:50,589 - AInewsbot - INFO - Techmeme -> htmldata/Techmeme (08_25_2024 07_56_05 AM).html
2024-08-25 07:56:50,590 - AInewsbot - INFO - The Register -> htmldata/The Register_ Enterprise

16


# Extract news URLs from saved pages

In [14]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-08-25 07:56:50,791 - AInewsbot - INFO - Parsing html files
2024-08-25 07:56:50,792 - AInewsbot - INFO - Business Insider -> htmldata/Tech - Business Insider (08_25_2024 07_55_18 AM).html
2024-08-25 07:56:50,792 - AInewsbot - INFO - parse loop - Business Insider
2024-08-25 07:56:50,824 - AInewsbot - INFO - parse_file - found 309 raw links
2024-08-25 07:56:50,832 - AInewsbot - INFO - parse_file - found 51 filtered links
2024-08-25 07:56:50,833 - AInewsbot - INFO - parse loop - 51 links found
2024-08-25 07:56:50,833 - AInewsbot - INFO - Feedly AI -> htmldata/Discover and Add New Feedly AI Feeds (08_25_2024 07_56_39 AM).html
2024-08-25 07:56:50,834 - AInewsbot - INFO - parse loop - Feedly AI
2024-08-25 07:56:50,867 - AInewsbot - INFO - parse_file - found 252 raw links
2024-08-25 07:56:50,871 - AInewsbot - INFO - parse_file - found 74 filtered links
2024-08-25 07:56:50,871 - AInewsbot - INFO - parse loop - 74 links found
2024-08-25 07:56:50,871 - AInewsbot - INFO - Ars Technica -> html

,id,src,title,url
0,0,Ars Technica,Hydrogels can learn to playPong,https://arstechnica.com/science/2024/08/hydrog...
1,1,Ars Technica,Tactical Breach Wizardsweaves engaging tactics...,https://arstechnica.com/gaming/2024/08/tactica...
2,2,Ars Technica,“We run a business”—why Microsoft’sIndiana Jon...,https://arstechnica.com/gaming/2024/08/we-run-...
3,3,Ars Technica,FDA green-lights fall COVID-19 boosters,https://arstechnica.com/health/2024/08/fda-gre...
4,4,Ars Technica,Paul Sutter walks us through the future of cli...,https://arstechnica.com/science/2022/04/paul-s...


In [15]:
orig_df.groupby("src").count()

,id,title,url
src,,,
Ars Technica,32,32,32
Business Insider,51,51,51
FT Tech,59,59,59
Feedly AI,73,73,73
Google News,67,67,67
Hacker News,25,25,25
Hacker News 2,26,26,26
HackerNoon,87,87,87
NYT Tech,18,18,18


In [16]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

# Filter URLs to new AI headlines only

In [17]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date=globals().get('before_date'))
len(filtered_df)


2024-08-25 07:56:51,616 - AInewsbot - INFO - Querying SQLite with where_clause: 
2024-08-25 07:56:52,242 - AInewsbot - INFO - Existing URLs: 143436
2024-08-25 07:56:52,270 - AInewsbot - INFO - New URLs: 644
2024-08-25 07:56:52,354 - AInewsbot - INFO - Existing src+title: 20
2024-08-25 07:56:52,355 - AInewsbot - INFO - New src+title: 624


624

In [18]:
after_date=None
SQLITE_DB = 'articles.db'
conn = sqlite3.connect(SQLITE_DB)
where_clause = ''
if before_date:
    where_clause = f"WHERE timestamp < '{before_date}'"
if after_date:
    if len(where_clause) == 0:
        where_clause = f"WHERE timestamp > '{after_date}'"
    else:
        where_clause += f" AND timestamp > '{after_date}'"
log(f"Querying SQLite with where_clause: {where_clause}")       
existing_urls = pd.read_sql_query(
    f"SELECT url, src, title FROM news_articles {where_clause}", conn)
conn.close()

2024-08-25 07:56:52,366 - AInewsbot - INFO - Querying SQLite with where_clause: 


In [19]:
where_clause

''

In [20]:
existing_urls.loc[existing_urls["title"]=="AI Fabricates More than Fake Legal Authorities"]

,url,src,title
143358,https://reason.com/volokh/2024/08/24/ai-fabric...,Feedly AI,AI Fabricates More than Fake Legal Authorities


In [21]:
existing_urls_list = existing_urls['url'].tolist()
log(f"Existing URLs: {len(existing_urls_list)}")
filtered_df = orig_df[~orig_df['url'].isin(existing_urls_list)]
log(f"New URLs: {len(filtered_df)}")

existing_urls = existing_urls.drop('url', axis=1)
existing_urls = existing_urls.drop_duplicates()
drop_df = pd.merge(existing_urls, filtered_df,
                   how='inner',
                   on=['src', 'title'])[["src", "title", "id"]]
log(f"Existing src+title: {len(drop_df)}")

filtered_df = filtered_df.loc[~filtered_df["id"].isin(drop_df["id"])]
log(f"New src+title: {len(filtered_df)}")


2024-08-25 07:56:52,476 - AInewsbot - INFO - Existing URLs: 143436
2024-08-25 07:56:52,506 - AInewsbot - INFO - New URLs: 644
2024-08-25 07:56:52,587 - AInewsbot - INFO - Existing src+title: 20
2024-08-25 07:56:52,587 - AInewsbot - INFO - New src+title: 624


In [22]:
filtered_df.groupby('src').count()

,id,title,url
src,,,
Ars Technica,29,29,29
Business Insider,32,32,32
FT Tech,49,49,49
Feedly AI,42,42,42
Google News,53,53,53
Hacker News,25,25,25
Hacker News 2,22,22,22
HackerNoon,67,67,67
NYT Tech,11,11,11


In [23]:
filtered_df.loc[filtered_df["title"]=="AI Fabricates More than Fake Legal Authorities"]

,id,src,title,url


In [24]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semantic meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based 

In [25]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-08-25 07:56:52,941 - AInewsbot - INFO - Paginated 13 pages


In [26]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-08-25 07:56:52,944 - AInewsbot - INFO - start classify
2024-08-25 07:56:52,948 - AInewsbot - INFO - Applying prompt to 13 pages using gpt-4o-mini
2024-08-25 07:56:52,952 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,972 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,972 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,973 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,973 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,973 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,974 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,974 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,974 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,975 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,975 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,975 - AInewsbot - INFO - sent 50 items 
2024-08-25 07:56:52,975 - AInewsbot - INFO - sent 24 items 
2024-08-25 07:56:55,640 - AInewsbot - INFO - got dict with 24 items 


624


,id,isAI
0,1,False
1,2,False
2,3,False
3,5,False
4,6,False


In [27]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,isAI,date
0,1,Ars Technica,Tactical Breach Wizardsweaves engaging tactics...,https://arstechnica.com/gaming/2024/08/tactica...,False,2024-08-25
1,2,Ars Technica,“We run a business”—why Microsoft’sIndiana Jon...,https://arstechnica.com/gaming/2024/08/we-run-...,False,2024-08-25
2,3,Ars Technica,FDA green-lights fall COVID-19 boosters,https://arstechnica.com/health/2024/08/fda-gre...,False,2024-08-25
3,5,Ars Technica,COVID shot now or later? Just getting it at al...,https://arstechnica.com/science/2024/08/covid-...,False,2024-08-25
4,6,Ars Technica,From recycling to food: Can we eat plastic-mun...,https://arstechnica.com/science/2024/08/from-r...,False,2024-08-25


In [28]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-08-25 07:57:00,962 - AInewsbot - INFO - Unmatched response rows: 0
2024-08-25 07:57:00,963 - AInewsbot - INFO - Unmatched source rows: 0


In [30]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")

AIdf

2024-08-25 08:26:30,428 - AInewsbot - INFO - Found 45 AI headlines


,id,src,title,url,isAI,date
0,0,FT Tech,The future of the AI-enhanced classroom,https://www.ft.com/content/e9523570-5966-4d99-...,True,2024-08-25
1,1,Feedly AI,AI DON'T THINK SO Microsoft’s creepy virtual ‘...,https://www.the-sun.com/tech/12284934/microsof...,True,2024-08-25
2,2,Feedly AI,Chinese AI apps eye overseas markets for growt...,https://www.scmp.com/tech/tech-trends/article/...,True,2024-08-25
3,3,Feedly AI,Why AI girlfriends or boyfriends via romantic ...,https://www.scmp.com/lifestyle/family-relation...,True,2024-08-25
4,4,Feedly AI,Global first: AI tool predicts how proteins wo...,https://www.thebrighterside.news/post/global-f...,True,2024-08-25
5,5,Feedly AI,It seems AI robot boxing is now a thing,https://www.creativebloq.com/ai/it-seems-ai-ro...,True,2024-08-25
6,6,Feedly AI,Living Nostradamus's chilling prediction over ...,https://www.themirror.com/news/weird-news/livi...,True,2024-08-25
7,7,Feedly AI,AMD’s Momentum in AI and Data Center Drive Bul...,https://www.tipranks.com/news/amd-stock-ai-and...,True,2024-08-25
8,8,Feedly AI,Don't overlook older workers. They could be th...,https://www.businessinsider.com/ai-older-worke...,True,2024-08-25
9,9,Feedly AI,Metavisio Unveils AI-Driven Laptops at IFA Berlin,https://www.tipranks.com/news/company-announce...,True,2024-08-25


In [31]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [32]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-08-25 08:26:30,466 - AInewsbot - INFO - Found 45 unique AI headlines


In [33]:
AIdf.groupby("src").count()

,id,title,url,isAI,date
src,,,,,
FT Tech,1,1,1,1,1
Feedly AI,28,28,28,28,28
Google News,11,11,11,11,11
Hacker News,1,1,1,1,1
Reddit,3,3,3,3,3
VentureBeat,1,1,1,1,1


In [34]:
# map google news headlines to redirect
# stopped working, google news gives some JS page which performs redirect instead of 301
# AIdf = get_google_news_redirects(AIdf)
AIdf['actual_url']=AIdf['url']

In [35]:
# must do this after updating google actualurl
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf.head()

,id,src,title,url,isAI,date,actual_url,hostname
0,0,Feedly AI,4 AI terms Nvidia investors should know,https://finance.yahoo.com/news/4-ai-terms-nvid...,True,2024-08-25,https://finance.yahoo.com/news/4-ai-terms-nvid...,finance.yahoo.com
1,1,Feedly AI,5 big analyst AI moves: Buy Nvidia stock ahead...,https://ca.investing.com/news/stock-market-new...,True,2024-08-25,https://ca.investing.com/news/stock-market-new...,ca.investing.com
2,2,Feedly AI,AI DON'T THINK SO Microsofts creepy virtual al...,https://www.the-sun.com/tech/12284934/microsof...,True,2024-08-25,https://www.the-sun.com/tech/12284934/microsof...,www.the-sun.com
3,3,Google News,AI may help create scientific abstracts,https://news.google.com/read/CBMilAFBVV95cUxPb...,True,2024-08-25,https://news.google.com/read/CBMilAFBVV95cUxPb...,news.google.com
4,4,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com


### Get site names and update site names based on URL

In [36]:
# get site_name
conn = sqlite3.connect('articles.db')
c = conn.cursor()
#  and timestamp > '2024-07-01' 
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}

sites_df

,id,hostname,site_name
0,1,247wallst.com,247wallst
1,2,3quarksdaily.com,3 Quarks Daily
2,3,9to5google.com,9to5Google
3,4,9to5mac.com,9to5Mac
4,5,9to5toys.com,9to5Toys
...,...,...,...
936,984,iowacapitaldispatch.com,Iowa Capital Dispatch
937,985,ladailypost.com,Los Alamos Daily Post
938,986,lexfridman.com,Lex Fridman
939,987,muratbuffalo.blogspot.com,Murat Buffalo


In [37]:
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))
AIdf.loc[AIdf['site_name']==""]

,id,src,title,url,isAI,date,actual_url,hostname,site_name
8,8,Hacker News,Anthropic Claude 3.5 can create icalendar file...,https://gregsramblings.com/stupid-but-useful-a...,True,2024-08-25,https://gregsramblings.com/stupid-but-useful-a...,gregsramblings.com,
12,12,Feedly AI,"Copilot AI calls journalist a child abuser, MS...",https://pivot-to-ai.com/2024/08/23/microsoft-t...,True,2024-08-25,https://pivot-to-ai.com/2024/08/23/microsoft-t...,pivot-to-ai.com,
25,25,Feedly AI,Living Nostradamus's chilling prediction over ...,https://www.themirror.com/news/weird-news/livi...,True,2024-08-25,https://www.themirror.com/news/weird-news/livi...,www.themirror.com,
36,36,Feedly AI,These AI-Designed Sunglasses Are About To Be E...,https://www.bustle.com/style/paloceras-pebble-...,True,2024-08-25,https://www.bustle.com/style/paloceras-pebble-...,www.bustle.com,


In [38]:
async def get_site_name(session, row):
    cat_prompt = f"""
based on this url and your knowledge of the Web, what is the name of the site? https://{row.hostname}

return the response as a json object of the form {{"url": "www.yankodesign.com", "site_name": "Yanko Design"}}

    """
    try:
        messages=[
                  {"role": "user", "content": cat_prompt
                  }]

        payload = {"model":  LOWCOST_MODEL,
                   "response_format": {"type": "json_object"},
                   "messages": messages,
                   "temperature": 0
                   }
        response = await fetch_openai(session, payload)
        response_dict = json.loads(response["choices"][0]["message"]["content"])
        return response_dict
    except Exception as exc:
        print(exc)
                
tasks = []
async with aiohttp.ClientSession() as session:
    for row in AIdf.loc[AIdf['site_name']==""].itertuples():
        task = asyncio.create_task(get_site_name(session, row))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)

responses


[{'url': 'gregsramblings.com', 'site_name': "Greg's Ramblings"},
 {'url': 'pivot-to-ai.com', 'site_name': 'Pivot to AI'},
 {'url': 'www.themirror.com', 'site_name': 'The Mirror'},
 {'url': 'www.bustle.com', 'site_name': 'Bustle'}]

In [39]:
# update site_dict from responses
new_urls = []
for r in responses:
    if r['url'].startswith('https://'):
        r['url'] = r['url'][8:]
    new_urls.append(r['url'])
    sites_dict[r['url']] = r['site_name']
    print(r['url'], r['site_name'])

AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))



gregsramblings.com Greg's Ramblings
pivot-to-ai.com Pivot to AI
www.themirror.com The Mirror
www.bustle.com Bustle


In [40]:
for url in new_urls:
    sqlstr = "INSERT OR IGNORE INTO sites (hostname, site_name) VALUES (?, ?);"
    print(url, '->', sites_dict[url])
    conn.execute(sqlstr, (url, sites_dict[url]))
    conn.commit()


gregsramblings.com -> Greg's Ramblings
pivot-to-ai.com -> Pivot to AI
www.themirror.com -> The Mirror
www.bustle.com -> Bustle


In [41]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in AIdf.itertuples():
    insert_article(conn, cursor, row.src, row.hostname, row.title,
                   row.url, row.actual_url, row.isAI, row.date)


# Topic analysis
Try to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

### Fit dimensionality reduction model

In [42]:
# # train dimensionality reduction, only need to do this every few months and pickle the model to reflect new topics
# # extracts all links from history where isAI=1
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id desc limit 20000"
# ai_history_df = pd.read_sql_query(query, conn)
# len(ai_history_df)

In [43]:
# embedding_model="text-embedding-3-large"
# embedding_df_list = []
# pages = paginate_df(ai_history_df, maxpagelen=1000, max_input_tokens=8192)

# for p in pages:
#     response = client.embeddings.create(input=[obj['title'] for obj in p],
#                                         model=embedding_model)
#     embedding_df_list.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))

# embedding_df = pd.concat(embedding_df_list, axis=0, ignore_index=True)

# embedding_df.to_pickle("historical_embedding_df.pkl")


In [44]:
# # Initialize the UMAP reducer
# reducer = umap.UMAP(n_components=30)
# # Fit the reducer to the data without transforming
# reducer.fit(embedding_df)
# # Pickle the reducer
# with open('reducer.pkl', 'wb') as f:
#     pickle.dump(reducer, f)
# print("UMAP reducer pickled and saved as 'reducer.pkl'")

In [45]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [46]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

2024-08-25 08:26:31,263 - AInewsbot - INFO - start topic extraction
2024-08-25 08:26:31,264 - AInewsbot - INFO - Applying prompt to 1 pages using gpt-4o-mini
2024-08-25 08:26:31,265 - AInewsbot - INFO - sent 45 items 
2024-08-25 08:26:42,379 - AInewsbot - INFO - got dict with 45 items 
2024-08-25 08:26:42,385 - AInewsbot - INFO - Processed 45 responses.
2024-08-25 08:26:42,387 - AInewsbot - INFO - end topic extraction


In [47]:
topic_df = pd.DataFrame(response)
topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
print(len(topic_df))
topic_df.head()


45


,id,extracted_topics
0,0,"[AI, Nvidia, investors, technology, finance]"
1,1,"[AI, analysts, Nvidia, stock market, BIDU, ear..."
2,2,"[AI, Microsoft, privacy, virtual assistant, te..."
3,3,"[AI, scientific research, abstracts, innovation]"
4,4,"[AI, autonomous research, self-modification, t..."


In [48]:
all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
item_counts = Counter(all_topics)
filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
print(len(filtered_topics))
sorted(filtered_topics)


14


['business',
 'education',
 'employment',
 'explicit images',
 'innovation',
 'investment',
 'jenna ortega',
 'microsoft',
 'nvidia',
 'openai',
 'privacy',
 'regulation',
 'scientist',
 'social media']

In [49]:
topic_df['extracted_topics'] = topic_df['extracted_topics'].apply(lambda l: [t.title() for t in l if t.lower() in filtered_topics])

In [50]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
for c in categories:
    print(c)

AI doom
AMD
Agriculture
Amazon
Anthropic
Apple
Art, Design
Artificial General Intelligence
Authors, Writing
Autonomous vehicles
Baidu
Big Tech
Bill Gates
Books, Publishing
Bubble
ChatGPT
China
Claude
Climate
Cloudflare
Code assistants
Cognitive Science
Cohere
Consciousness
Copilot
Cryptocurrency
Customer service
Cybersecurity
Data
Deals
Deepfakes
Disinformation
Disney
Economics
Education
Elon Musk
Energy
Entertainment
Ethics
European Union
Facial Recognition
Fashion
Finance
Fintech
Food, Drink
Funding
Gaming
Gemini
Gen AI
Geoffrey Hinton
Google
Health, Fitness
Healthcare
History
Hollywood
IPOs
India
Intellectual Property
Japan
Jobs, Careers, Labor Market
Korea
LLMs
Language Models
Laws
Lifestyle, Travel
Machine learning
Manufacturing
Mark Zuckerberg
Mergers and acquisitions
Meta
Microsoft
Military
Music
Mustafa Suleyman
Netflix
Nuclear
Nvidia
Open Source
OpenAI
Opinion
Perplexity
Policy and regulation
Politics
Privacy
Products
RAG
Recommendation systems
Reddit
Reinforcement Learning
Re

In [51]:
[t for t in filtered_topics if t.lower() not in [u.lower() for u in CANONICAL_TOPICS]]

['innovation',
 'scientist',
 'regulation',
 'business',
 'employment',
 'investment',
 'jenna ortega',
 'explicit images',
 'social media']

In [52]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name
0,0,Feedly AI,4 AI terms Nvidia investors should know,https://finance.yahoo.com/news/4-ai-terms-nvid...,True,2024-08-25,https://finance.yahoo.com/news/4-ai-terms-nvid...,finance.yahoo.com,Yahoo Finance
1,1,Feedly AI,5 big analyst AI moves: Buy Nvidia stock ahead...,https://ca.investing.com/news/stock-market-new...,True,2024-08-25,https://ca.investing.com/news/stock-market-new...,ca.investing.com,Investing.com
2,2,Feedly AI,AI DON'T THINK SO Microsofts creepy virtual al...,https://www.the-sun.com/tech/12284934/microsof...,True,2024-08-25,https://www.the-sun.com/tech/12284934/microsof...,www.the-sun.com,The Sun
3,3,Google News,AI may help create scientific abstracts,https://news.google.com/read/CBMilAFBVV95cUxPb...,True,2024-08-25,https://news.google.com/read/CBMilAFBVV95cUxPb...,news.google.com,Google News
4,4,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News
5,5,Feedly AI,AMDs Momentum in AI and Data Center Drive Bull...,https://www.tipranks.com/news/amd-stock-ai-and...,True,2024-08-25,https://www.tipranks.com/news/amd-stock-ai-and...,www.tipranks.com,TipRanks
6,6,Feedly AI,AWS Enhancing Information Retrieval in Large L...,https://www.marktechpost.com/2024/08/24/aws-en...,True,2024-08-25,https://www.marktechpost.com/2024/08/24/aws-en...,www.marktechpost.com,MarkTechPost
7,7,Google News,An AI Scientist Is Inventing and Running Its O...,https://news.google.com/read/CBMiXkFVX3lxTFBVc...,True,2024-08-25,https://news.google.com/read/CBMiXkFVX3lxTFBVc...,news.google.com,Google News
8,8,Hacker News,Anthropic Claude 3.5 can create icalendar file...,https://gregsramblings.com/stupid-but-useful-a...,True,2024-08-25,https://gregsramblings.com/stupid-but-useful-a...,gregsramblings.com,Greg's Ramblings
9,9,Feedly AI,Brain-computer interfaces tap AI to enable a m...,https://www.fastcompany.com/91177581/brain-com...,True,2024-08-25,https://www.fastcompany.com/91177581/brain-com...,www.fastcompany.com,Fast Company


In [53]:
# maybe try to set timeout in categorize_headline
# https://platform.openai.com/settings/organization/limits
# rate limit is 10,000 rpm , we are sending like 100 at a time and waiting for response so should be OK

catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_headline(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        
catdict

2024-08-25 08:26:42,459 - AInewsbot - INFO - Categorizing headline 1 of 45
2024-08-25 08:26:42,459 - AInewsbot - INFO - 4 AI terms Nvidia investors should know
2024-08-25 08:26:45,406 - AInewsbot - INFO - ['Big Tech', 'Education', 'Gen AI', 'Nvidia']
2024-08-25 08:26:45,407 - AInewsbot - INFO - Categorizing headline 2 of 45
2024-08-25 08:26:45,408 - AInewsbot - INFO - 5 big analyst AI moves: Buy Nvidia stock ahead of earnings; BIDU downgraded
2024-08-25 08:26:47,999 - AInewsbot - INFO - ['Baidu', 'Big Tech', 'Finance', 'Nvidia', 'Stocks']
2024-08-25 08:26:48,000 - AInewsbot - INFO - Categorizing headline 3 of 45
2024-08-25 08:26:48,000 - AInewsbot - INFO - AI DON'T THINK SO Microsofts creepy virtual all-seeing AI helper is set to make a comeback despite fears over privacy nightmare
2024-08-25 08:26:48,959 - AInewsbot - INFO - ['AI doom', 'Big Tech', 'Copilot', 'Cybersecurity', 'Ethics', 'Gen AI', 'Microsoft', 'Opinion', 'Privacy', 'Products']
2024-08-25 08:26:48,961 - AInewsbot - INFO 

2024-08-25 08:27:54,121 - AInewsbot - INFO - Categorizing headline 26 of 45
2024-08-25 08:27:54,122 - AInewsbot - INFO - Living Nostradamus's chilling prediction over AI that will reactivate 'ancestral powers'
2024-08-25 08:27:55,368 - AInewsbot - INFO - ['AI doom', 'Gen AI']
2024-08-25 08:27:55,369 - AInewsbot - INFO - Categorizing headline 27 of 45
2024-08-25 08:27:55,371 - AInewsbot - INFO - MagicDec: Unlocking Up to 2x Speedup in LLaMA Models for Long-Context Applications
2024-08-25 08:27:56,445 - AInewsbot - INFO - ['Data', 'Gen AI', 'LLMs', 'Language Models', 'Machine learning', 'Meta', 'Products', 'Science']
2024-08-25 08:27:56,447 - AInewsbot - INFO - Categorizing headline 28 of 45
2024-08-25 08:27:56,447 - AInewsbot - INFO - Metavisio Unveils AI-Driven Laptops at IFA Berlin
2024-08-25 08:27:57,671 - AInewsbot - INFO - ['Gen AI', 'Products']
2024-08-25 08:27:57,672 - AInewsbot - INFO - Categorizing headline 29 of 45
2024-08-25 08:27:57,672 - AInewsbot - INFO - Newly Created 'AI

{0: ['Big Tech', 'Education', 'Gen AI', 'Nvidia'],
 1: ['Baidu', 'Big Tech', 'Finance', 'Nvidia', 'Stocks'],
 2: ['AI doom',
  'Big Tech',
  'Copilot',
  'Cybersecurity',
  'Ethics',
  'Gen AI',
  'Microsoft',
  'Opinion',
  'Privacy',
  'Products'],
 3: ['Authors, Writing',
  'Gen AI',
  'LLMs',
  'Language Models',
  'Machine learning',
  'Science'],
 4: ['Artificial General Intelligence',
  'Big Tech',
  'Code assistants',
  'Ethics',
  'Gen AI',
  'LLMs',
  'Machine learning',
  'Science',
  'Singularity'],
 5: ['AMD', 'Big Tech', 'Data', 'Gen AI', 'Semiconductor chips', 'Stocks'],
 6: ['Amazon',
  'Big Tech',
  'Data',
  'Gen AI',
  'LLMs',
  'Language Models',
  'Machine learning',
  'Products',
  'RAG',
  'Science'],
 7: ['Artificial General Intelligence',
  'Gen AI',
  'Machine learning',
  'Science',
  'Testing'],
 8: ['Anthropic',
  'Big Tech',
  'Claude',
  'Code assistants',
  'Gen AI',
  'LLMs',
  'Language Models',
  'Machine learning',
  'Products'],
 9: ['Gen AI', 'Heal

In [54]:
topic_df['assigned_topics'] = topic_df['id'].apply(lambda id: catdict.get(id, ""))
topic_df

,id,extracted_topics,assigned_topics
0,0,[Nvidia],"[Big Tech, Education, Gen AI, Nvidia]"
1,1,[Nvidia],"[Baidu, Big Tech, Finance, Nvidia, Stocks]"
2,2,"[Microsoft, Privacy]","[AI doom, Big Tech, Copilot, Cybersecurity, Et..."
3,3,[Innovation],"[Authors, Writing, Gen AI, LLMs, Language Mode..."
4,4,[],"[Artificial General Intelligence, Big Tech, Co..."
5,5,[],"[AMD, Big Tech, Data, Gen AI, Semiconductor ch..."
6,6,[],"[Amazon, Big Tech, Data, Gen AI, LLMs, Languag..."
7,7,"[Scientist, Innovation]","[Artificial General Intelligence, Gen AI, Mach..."
8,8,[],"[Anthropic, Big Tech, Claude, Code assistants,..."
9,9,[],"[Gen AI, Healthcare, Science]"


In [55]:
lcategories = set([c.lower() for c in categories])

In [56]:
def clean_topics(row):
    topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]
    return combined

topic_df["topics"] = topic_df.apply(clean_topics, axis=1)
topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)
topic_df

,id,extracted_topics,assigned_topics,topics,topic_str
0,0,[Nvidia],"[Big Tech, Education, Gen AI, Nvidia]","[Big Tech, Education, Gen AI, Nvidia]","Big Tech, Education, Gen AI, Nvidia"
1,1,[Nvidia],"[Baidu, Big Tech, Finance, Nvidia, Stocks]","[Baidu, Big Tech, Finance, Nvidia, Stocks]","Baidu, Big Tech, Finance, Nvidia, Stocks"
2,2,"[Microsoft, Privacy]","[AI doom, Big Tech, Copilot, Cybersecurity, Et...","[AI Doom, Big Tech, Copilot, Cybersecurity, Et...","AI Doom, Big Tech, Copilot, Cybersecurity, Eth..."
3,3,[Innovation],"[Authors, Writing, Gen AI, LLMs, Language Mode...","[Authors, Writing, Gen AI, Innovation, Languag...","Authors, Writing, Gen AI, Innovation, Language..."
4,4,[],"[Artificial General Intelligence, Big Tech, Co...","[Artificial General Intelligence, Big Tech, Co...","Artificial General Intelligence, Big Tech, Cod..."
5,5,[],"[AMD, Big Tech, Data, Gen AI, Semiconductor ch...","[Amd, Big Tech, Data, Gen AI, Semiconductor Ch...","Amd, Big Tech, Data, Gen AI, Semiconductor Chi..."
6,6,[],"[Amazon, Big Tech, Data, Gen AI, LLMs, Languag...","[Amazon, Big Tech, Data, Gen AI, Language Mode...","Amazon, Big Tech, Data, Gen AI, Language Model..."
7,7,"[Scientist, Innovation]","[Artificial General Intelligence, Gen AI, Mach...","[Artificial General Intelligence, Gen AI, Inno...","Artificial General Intelligence, Gen AI, Innov..."
8,8,[],"[Anthropic, Big Tech, Claude, Code assistants,...","[Anthropic, Big Tech, Claude, Code Assistants,...","Anthropic, Big Tech, Claude, Code Assistants, ..."
9,9,[],"[Gen AI, Healthcare, Science]","[Gen AI, Healthcare, Science]","Gen AI, Healthcare, Science"


In [57]:
try:  # for idempotency
    AIdf = AIdf.drop(columns=["title_topic_str"])
except:
    pass
try:  # for idempotency
    AIdf = AIdf.drop(columns=["topic_str"])
except:
    pass

AIdf = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="inner")
AIdf['title_topic_str'] = AIdf.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str
0,0,Feedly AI,4 AI terms Nvidia investors should know,https://finance.yahoo.com/news/4-ai-terms-nvid...,True,2024-08-25,https://finance.yahoo.com/news/4-ai-terms-nvid...,finance.yahoo.com,Yahoo Finance,"Big Tech, Education, Gen AI, Nvidia",4 AI terms Nvidia investors should know (Topic...
1,1,Feedly AI,5 big analyst AI moves: Buy Nvidia stock ahead...,https://ca.investing.com/news/stock-market-new...,True,2024-08-25,https://ca.investing.com/news/stock-market-new...,ca.investing.com,Investing.com,"Baidu, Big Tech, Finance, Nvidia, Stocks",5 big analyst AI moves: Buy Nvidia stock ahead...
2,2,Feedly AI,AI DON'T THINK SO Microsofts creepy virtual al...,https://www.the-sun.com/tech/12284934/microsof...,True,2024-08-25,https://www.the-sun.com/tech/12284934/microsof...,www.the-sun.com,The Sun,"AI Doom, Big Tech, Copilot, Cybersecurity, Eth...",AI DON'T THINK SO Microsofts creepy virtual al...
3,3,Google News,AI may help create scientific abstracts,https://news.google.com/read/CBMilAFBVV95cUxPb...,True,2024-08-25,https://news.google.com/read/CBMilAFBVV95cUxPb...,news.google.com,Google News,"Authors, Writing, Gen AI, Innovation, Language...",AI may help create scientific abstracts (Topic...
4,4,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Cod...",AI tool meant for autonomous research modified...
5,5,Feedly AI,AMDs Momentum in AI and Data Center Drive Bull...,https://www.tipranks.com/news/amd-stock-ai-and...,True,2024-08-25,https://www.tipranks.com/news/amd-stock-ai-and...,www.tipranks.com,TipRanks,"Amd, Big Tech, Data, Gen AI, Semiconductor Chi...",AMDs Momentum in AI and Data Center Drive Bull...
6,6,Feedly AI,AWS Enhancing Information Retrieval in Large L...,https://www.marktechpost.com/2024/08/24/aws-en...,True,2024-08-25,https://www.marktechpost.com/2024/08/24/aws-en...,www.marktechpost.com,MarkTechPost,"Amazon, Big Tech, Data, Gen AI, Language Model...",AWS Enhancing Information Retrieval in Large L...
7,7,Google News,An AI Scientist Is Inventing and Running Its O...,https://news.google.com/read/CBMiXkFVX3lxTFBVc...,True,2024-08-25,https://news.google.com/read/CBMiXkFVX3lxTFBVc...,news.google.com,Google News,"Artificial General Intelligence, Gen AI, Innov...",An AI Scientist Is Inventing and Running Its O...
8,8,Hacker News,Anthropic Claude 3.5 can create icalendar file...,https://gregsramblings.com/stupid-but-useful-a...,True,2024-08-25,https://gregsramblings.com/stupid-but-useful-a...,gregsramblings.com,Greg's Ramblings,"Anthropic, Big Tech, Claude, Code Assistants, ...",Anthropic Claude 3.5 can create icalendar file...
9,9,Feedly AI,Brain-computer interfaces tap AI to enable a m...,https://www.fastcompany.com/91177581/brain-com...,True,2024-08-25,https://www.fastcompany.com/91177581/brain-com...,www.fastcompany.com,Fast Company,"Gen AI, Healthcare, Science",Brain-computer interfaces tap AI to enable a m...


In [58]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str
0,0,Feedly AI,4 AI terms Nvidia investors should know,https://finance.yahoo.com/news/4-ai-terms-nvid...,True,2024-08-25,https://finance.yahoo.com/news/4-ai-terms-nvid...,finance.yahoo.com,Yahoo Finance,"Big Tech, Education, Gen AI, Nvidia",4 AI terms Nvidia investors should know (Topic...
1,1,Feedly AI,5 big analyst AI moves: Buy Nvidia stock ahead...,https://ca.investing.com/news/stock-market-new...,True,2024-08-25,https://ca.investing.com/news/stock-market-new...,ca.investing.com,Investing.com,"Baidu, Big Tech, Finance, Nvidia, Stocks",5 big analyst AI moves: Buy Nvidia stock ahead...
2,2,Feedly AI,AI DON'T THINK SO Microsofts creepy virtual al...,https://www.the-sun.com/tech/12284934/microsof...,True,2024-08-25,https://www.the-sun.com/tech/12284934/microsof...,www.the-sun.com,The Sun,"AI Doom, Big Tech, Copilot, Cybersecurity, Eth...",AI DON'T THINK SO Microsofts creepy virtual al...
3,3,Google News,AI may help create scientific abstracts,https://news.google.com/read/CBMilAFBVV95cUxPb...,True,2024-08-25,https://news.google.com/read/CBMilAFBVV95cUxPb...,news.google.com,Google News,"Authors, Writing, Gen AI, Innovation, Language...",AI may help create scientific abstracts (Topic...
4,4,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Cod...",AI tool meant for autonomous research modified...
5,5,Feedly AI,AMDs Momentum in AI and Data Center Drive Bull...,https://www.tipranks.com/news/amd-stock-ai-and...,True,2024-08-25,https://www.tipranks.com/news/amd-stock-ai-and...,www.tipranks.com,TipRanks,"Amd, Big Tech, Data, Gen AI, Semiconductor Chi...",AMDs Momentum in AI and Data Center Drive Bull...
6,6,Feedly AI,AWS Enhancing Information Retrieval in Large L...,https://www.marktechpost.com/2024/08/24/aws-en...,True,2024-08-25,https://www.marktechpost.com/2024/08/24/aws-en...,www.marktechpost.com,MarkTechPost,"Amazon, Big Tech, Data, Gen AI, Language Model...",AWS Enhancing Information Retrieval in Large L...
7,7,Google News,An AI Scientist Is Inventing and Running Its O...,https://news.google.com/read/CBMiXkFVX3lxTFBVc...,True,2024-08-25,https://news.google.com/read/CBMiXkFVX3lxTFBVc...,news.google.com,Google News,"Artificial General Intelligence, Gen AI, Innov...",An AI Scientist Is Inventing and Running Its O...
8,8,Hacker News,Anthropic Claude 3.5 can create icalendar file...,https://gregsramblings.com/stupid-but-useful-a...,True,2024-08-25,https://gregsramblings.com/stupid-but-useful-a...,gregsramblings.com,Greg's Ramblings,"Anthropic, Big Tech, Claude, Code Assistants, ...",Anthropic Claude 3.5 can create icalendar file...
9,9,Feedly AI,Brain-computer interfaces tap AI to enable a m...,https://www.fastcompany.com/91177581/brain-com...,True,2024-08-25,https://www.fastcompany.com/91177581/brain-com...,www.fastcompany.com,Fast Company,"Gen AI, Healthcare, Science",Brain-computer interfaces tap AI to enable a m...


In [59]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(AIdf.loc[AIdf["topic_str"]==""][['title']])


,title
42,aiSmarterNoTestingNeeded


### Semantic sort

In [60]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

# sort of a traveling salesman sort
log(f"Sort with agglomerative cluster sort")
sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf['sort_order'] = sorted_indices

# do dimensionality reduction on embedding_df and cluster analysis
log(f"Perform dimensionality reduction")
with open("reducer.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
reduced_data = reducer.transform(embedding_df)
log(f"Cluster with DBSCAN")
dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
AIdf['cluster'] = dbscan.fit_predict(reduced_data)
AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999
    
# sort first by clusters found by DBSCAN, then by semantic ordering
AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

AIdf


2024-08-25 08:28:32,519 - AInewsbot - INFO - Fetching embeddings for 45 headlines
python(89190) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2024-08-25 08:28:33,094 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:28:33,389 - AInewsbot - INFO - Sort with agglomerative cluster sort
2024-08-25 08:28:33,396 - AInewsbot - INFO - Perform dimensionality reduction
2024-08-25 08:28:37,173 - AInewsbot - INFO - Cluster with DBSCAN
python(89220) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster
0,0,Google News,"Jenna Ortega Says AI is Disgusting, That She W...",https://news.google.com/read/CBMiyAFBVV95cUxNN...,True,2024-08-25,https://news.google.com/read/CBMiyAFBVV95cUxNN...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Gen AI, Holl...","Jenna Ortega Says AI is Disgusting, That She W...",14,0
1,1,Google News,Jenna Ortega Reveals She Deleted Twitter After...,https://news.google.com/read/CBMivgFBVV95cUxQR...,True,2024-08-25,https://news.google.com/read/CBMivgFBVV95cUxQR...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Reveals She Deleted Twitter After...,26,0
2,2,Google News,Jenna Ortega Says She Deleted Twitter After Re...,https://news.google.com/read/CBMikgFBVV95cUxOT...,True,2024-08-25,https://news.google.com/read/CBMikgFBVV95cUxOT...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Says She Deleted Twitter After Re...,41,0
3,3,Feedly AI,Democratic Startup Proposes Flooding the Web W...,https://futurism.com/democratic-startup-ai-pol...,True,2024-08-25,https://futurism.com/democratic-startup-ai-pol...,futurism.com,Futurism,"Disinformation, Gen AI, Politics",Democratic Startup Proposes Flooding the Web W...,0,999
4,4,Feedly AI,Don't overlook older workers. They could be th...,https://www.businessinsider.com/ai-older-worke...,True,2024-08-25,https://www.businessinsider.com/ai-older-worke...,www.businessinsider.com,Business Insider,"Business, Employment, Gen AI, Jobs, Careers, L...",Don't overlook older workers. They could be th...,1,999
5,5,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Cod...",AI tool meant for autonomous research modified...,2,999
6,6,Feedly AI,"The Middle East Microsoft, OpenAI partner mire...",https://www.cnbc.com/2024/08/25/a-controversia...,True,2024-08-25,https://www.cnbc.com/2024/08/25/a-controversia...,www.cnbc.com,CNBC,"Big Tech, Ethics, Gen AI, Laws, Microsoft, Ope...","The Middle East Microsoft, OpenAI partner mire...",3,999
7,7,FT Tech,The future of the AI-enhanced classroom,https://www.ft.com/content/e9523570-5966-4d99-...,True,2024-08-25,https://www.ft.com/content/e9523570-5966-4d99-...,www.ft.com,Financial Times,"Education, Gen AI, Science, Society, Culture",The future of the AI-enhanced classroom (Topic...,4,999
8,8,Feedly AI,Chinese AI apps eye overseas markets for growt...,https://www.scmp.com/tech/tech-trends/article/...,True,2024-08-25,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"China, Economics, Gen AI, Policy And Regulatio...",Chinese AI apps eye overseas markets for growt...,5,999
9,9,Feedly AI,It seems AI robot boxing is now a thing,https://www.creativebloq.com/ai/it-seems-ai-ro...,True,2024-08-25,https://www.creativebloq.com/ai/it-seems-ai-ro...,www.creativebloq.com,Creative Bloq,"Entertainment, Gaming, Gen AI, Products, Robot...",It seems AI robot boxing is now a thing (Topic...,6,999


In [61]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):

    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses. 
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided 
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
            response = await fetch_openai(session, payload)
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}
        

# show clusters
cluster_topics = []
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    async with aiohttp.ClientSession() as session:

        for i in range(30):
            tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
            if len(tmpdf) ==0:
                break
            display(tmpdf)
            title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
            cluster_topic = await write_topic_name(session, title_topic_str_list)
            cluster_topics.append(cluster_topic)
            print(cluster_topic)

    


,id,title_topic_str
0,0,"Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen (Topics: Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies)"
1,1,"Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting (Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture)"
2,2,"Jenna Ortega Says She Deleted Twitter After Receiving AI-Generated Explicit Images Of Herself As A Child (Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture)"


{'topic_title': 'AI and Privacy in Entertainment'}


In [62]:
# to determine which sstories to cluster / include
# we could rate sites for reliability, like NYT vs some site in India we never heard of
# we could rate content for length, perplexiyt or something

In [63]:
# we could extract top words using tfidf, something like 
# vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = vectorizer.fit_transform(documents)
# feature_names = vectorizer.get_feature_names_out()
# topics = []
# for i in range(n_topics):
#     # Get the documents in this cluster
#     cluster_docs = [doc for doc, label in zip(documents, cluster_labels) if label == i]
#     cluster_metadatas = [meta for meta, label in zip(metadatas, cluster_labels) if label == i]

#     # Get the top words for this cluster based on TF-IDF scores
#     tfidf_scores = tfidf_matrix[cluster_labels == i].sum(axis=0).A1
#     top_word_indices = tfidf_scores.argsort()[-n_words_per_topic:][::-1]
#     top_words = [feature_names[index] for index in top_word_indices]


In [64]:
cluster_topic_list = [obj['topic_title'] for obj in cluster_topics]
cluster_topic_list

['AI and Privacy in Entertainment']

In [65]:
AIdf['cluster_name'] = AIdf['cluster'].apply(lambda i: cluster_topic_list[i] if i<len(cluster_topic_list) else "")


# Save and email headlines


In [66]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.site_name}]({row.actual_url})")
    html_str += f'{row.Index}.<a href="{row.actual_url}">{row.title} - {row.site_name}</a><br />\n'


2024-08-25 08:28:38,026 - AInewsbot - INFO - [0. Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)
2024-08-25 08:28:38,027 - AInewsbot - INFO - [1. Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting - Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURD

2024-08-25 08:28:38,032 - AInewsbot - INFO - [28. dataScience - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1f07ijl/datascience/)
2024-08-25 08:28:38,032 - AInewsbot - INFO - [29. The top AI deals in Europe this year - Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw)
2024-08-25 08:28:38,032 - AInewsbot - INFO - [30. Copilot AI calls journalist a child abuser, MS tries to launder responsibility - Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/)
2024-08-25 08:28:38,033 - AInewsbot - INFO - [31. MagicDec: Unlocking Up to 2x Speedup in LLaMA Models for Long-Context Applications - MarkTechPost](https://www.marktechpost.com/2024/08/24/magicdec-unlocking-up-to-2x-speedup-in-llama-models-for-long-context-applications/)
2024-08-25 

In [67]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [68]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-08-25 08:28:38,041 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [69]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.actual_url, row.title))


2024-08-25 08:28:39,269 - AInewsbot - INFO - Queuing URLs for scraping


In [70]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-08-25 08:28:39,284 - AInewsbot - INFO - fetching 45 pages using 4 browsers
2024-08-25 08:28:39,285 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 08:28:39,286 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 08:28:39,286 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 08:28:39,286 - AInewsbot - INFO - get_driver - 74321 Initializing webdriver
2024-08-25 08:28:55,772 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-08-25 08:28:55,776 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-08-25 08:28:55,777 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-08-25 08:28:55,784 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-08-25 08:28:55,783 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-08-25 08:28:55,784 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-08-25 08:28:55,781 - AInewsbot - INFO -

2024-08-25 08:30:32,998 - AInewsbot - INFO - get_url(Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen) - Saving Jenna_Ortega_Says_AI_is_Disgusting__That_She_Was_Sent_Fake_Dirty_Images_of_Herself_as_a_Teen_20240825_083032.html as utf-8
2024-08-25 08:30:33,034 - AInewsbot - INFO - Processing https://www.ft.com/content/e9523570-5966-4d99-ac92-45d3966ae28e
2024-08-25 08:30:33,038 - AInewsbot - INFO - get_url(https://www.ft.com/content/e9523570-5966-4d99-ac92-45d3966ae28e) - starting get_url https://www.ft.com/content/e9523570-5966-4d99-ac92-45d3966ae28e
2024-08-25 08:30:42,190 - AInewsbot - INFO - get_url(Don't overlook older workers. They could be the best at AI in your business.) - Saving Don_t_overlook_older_workers__They_could_be_the_best_at_AI_in_your_business__20240825_083042.html as utf-8
2024-08-25 08:30:42,197 - AInewsbot - INFO - Processing https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth

2024-08-25 08:31:05,263 - AInewsbot - INFO - get_url(How agentic AI platforms will redefine enterprise applications) - Saving How_agentic_AI_platforms_will_redefine_enterprise_applications_20240825_083105.html as utf-8
2024-08-25 08:31:05,273 - AInewsbot - INFO - Processing https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS
2024-08-25 08:31:05,281 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS) - starting get_url https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R

2024-08-25 08:31:42,419 - AInewsbot - INFO - get_url(5 big analyst AI moves: Buy Nvidia stock ahead of earnings; BIDU downgraded) - Saving 5_big_analyst_AI_moves__Buy_Nvidia_stock_ahead_of_earnings__BIDU_downgraded_20240825_083142.html as utf-8
2024-08-25 08:31:42,423 - AInewsbot - INFO - Processing https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw
2024-08-25 08:31:42,426 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw) - starting get_url https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw
2024-08-25 08:31:45,008 - 

2024-08-25 08:32:12,874 - AInewsbot - INFO - Processing https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc
2024-08-25 08:32:12,875 - AInewsbot - INFO - get_url(https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc) - starting get_url https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc
2024-08-25 08:32:14,976 - AInewsbot - INFO - get_url(Metavisio Unveils AI-Driven Laptops at IFA Berlin) - Saving Metavisio_Unveils_AI-Driven_Laptops_at_IFA_Berlin_20240825_083214.html as utf-8
2024-08-25 08:32:14,977 - AInewsbot - INFO - Processing https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists
2024-08-25 08:32:14,977 - 

In [71]:
saved_pages

[(1,
  'https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ',
  'Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting',
  'htmldata/Jenna_Ortega_Reveals_She_Deleted_Twitter_After_Receiving_Explicit_AI_Images_of_Herself_as_a_Teen__Its_Disgusting_20240825_083031.html'),
 (5,
  'https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZX

In [72]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'actual_url', 'title', 'path']
pages_df

,id,actual_url,title,path
0,1,https://news.google.com/read/CBMivgFBVV95cUxQR...,Jenna Ortega Reveals She Deleted Twitter After...,htmldata/Jenna_Ortega_Reveals_She_Deleted_Twit...
1,5,https://news.google.com/read/CBMipAFBVV95cUxNa...,AI tool meant for autonomous research modified...,htmldata/AI_tool_meant_for_autonomous_research...
2,9,https://www.creativebloq.com/ai/it-seems-ai-ro...,It seems AI robot boxing is now a thing,htmldata/It_seems_AI_robot_boxing_is_now_a_thi...
3,12,https://news.google.com/read/CBMihgFBVV95cUxQa...,Taylor Swift fake posted by Trump highlights c...,htmldata/Taylor_Swift_fake_posted_by_Trump_hig...
4,16,https://gregsramblings.com/stupid-but-useful-a...,Anthropic Claude 3.5 can create icalendar file...,htmldata/Anthropic_Claude_3_5_can_create_icale...
5,21,https://www.forbes.com/sites/mariagraciasantil...,Your Next Career Coach Could Be A Chatbot,htmldata/Your_Next_Career_Coach_Could_Be_A_Cha...
6,25,https://finance.yahoo.com/news/4-ai-terms-nvid...,4 AI terms Nvidia investors should know,htmldata/4_AI_terms_Nvidia_investors_should_kn...
7,28,https://www.reddit.com/r/ProgrammerHumor/comme...,dataScience,htmldata/dataScience_20240825_083150.html
8,32,https://www.foxnews.com/tech/how-meta-ai-can-y...,How Meta AI can be your new digital assistant ...,htmldata/How_Meta_AI_can_be_your_new_digital_a...
9,36,https://www.tipranks.com/news/company-announce...,Metavisio Unveils AI-Driven Laptops at IFA Berlin,htmldata/Metavisio_Unveils_AI-Driven_Laptops_a...


In [73]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [74]:
AIdf

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,"Jenna Ortega Says AI is Disgusting, That She W...",https://news.google.com/read/CBMiyAFBVV95cUxNN...,True,2024-08-25,https://news.google.com/read/CBMiyAFBVV95cUxNN...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Gen AI, Holl...","Jenna Ortega Says AI is Disgusting, That She W...",14,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Says_AI_is_Disgusting__T...
1,1,Google News,Jenna Ortega Reveals She Deleted Twitter After...,https://news.google.com/read/CBMivgFBVV95cUxQR...,True,2024-08-25,https://news.google.com/read/CBMivgFBVV95cUxQR...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Reveals She Deleted Twitter After...,26,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Reveals_She_Deleted_Twit...
2,2,Google News,Jenna Ortega Says She Deleted Twitter After Re...,https://news.google.com/read/CBMikgFBVV95cUxOT...,True,2024-08-25,https://news.google.com/read/CBMikgFBVV95cUxOT...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Says She Deleted Twitter After Re...,41,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Says_She_Deleted_Twitter...
3,3,Feedly AI,Democratic Startup Proposes Flooding the Web W...,https://futurism.com/democratic-startup-ai-pol...,True,2024-08-25,https://futurism.com/democratic-startup-ai-pol...,futurism.com,Futurism,"Disinformation, Gen AI, Politics",Democratic Startup Proposes Flooding the Web W...,0,999,,htmldata/Democratic_Startup_Proposes_Flooding_...
4,4,Feedly AI,Don't overlook older workers. They could be th...,https://www.businessinsider.com/ai-older-worke...,True,2024-08-25,https://www.businessinsider.com/ai-older-worke...,www.businessinsider.com,Business Insider,"Business, Employment, Gen AI, Jobs, Careers, L...",Don't overlook older workers. They could be th...,1,999,,htmldata/Don_t_overlook_older_workers__They_co...
5,5,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Cod...",AI tool meant for autonomous research modified...,2,999,,htmldata/AI_tool_meant_for_autonomous_research...
6,6,Feedly AI,"The Middle East Microsoft, OpenAI partner mire...",https://www.cnbc.com/2024/08/25/a-controversia...,True,2024-08-25,https://www.cnbc.com/2024/08/25/a-controversia...,www.cnbc.com,CNBC,"Big Tech, Ethics, Gen AI, Laws, Microsoft, Ope...","The Middle East Microsoft, OpenAI partner mire...",3,999,,htmldata/The_Middle_East_Microsoft__OpenAI_par...
7,7,FT Tech,The future of the AI-enhanced classroom,https://www.ft.com/content/e9523570-5966-4d99-...,True,2024-08-25,https://www.ft.com/content/e9523570-5966-4d99-...,www.ft.com,Financial Times,"Education, Gen AI, Science, Society, Culture",The future of the AI-enhanced classroom (Topic...,4,999,,htmldata/The_future_of_the_AI-enhanced_classro...
8,8,Feedly AI,Chinese AI apps eye overseas markets for growt...,https://www.scmp.com/tech/tech-trends/article/...,True,2024-08-25,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"China, Economics, Gen AI, Policy And Regulatio...",Chinese AI apps eye overseas markets for growt...,5,999,,htmldata/Chinese_AI_apps_eye_overseas_markets_...
9,9,Feedly AI,It seems AI robot boxing is now a thing,https://www.creativebloq.com/ai/it-seems-ai-ro...,True,2024-08-25,https://www.creativebloq.com/ai/it-seems-ai-ro...,www.creativebloq.com,Creative Bloq,"Entertainment, Gaming, Gen AI, Products, Robot...",It seems AI robot boxing is now a thing (Topic...,6,999,,htmldata/It_seems_AI_robot_boxing_is_now_a_thi...


# Summarize individual pages

In [75]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will focus on the top 3 points of the text and keep the response concise."
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will output Markdown format.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [76]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [77]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on gpt-4o-minif and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-08-25 08:32:40,468 - AInewsbot - INFO - Starting summarize
2024-08-25 08:32:40,557 - AInewsbot - INFO - fetch_all_summaries - Page title: Jenna Ortega Says AI is ‘Disgusting,’ That She Was Sent Fake Dirty Images of Herself as a Teen | Jenna Ortega | Just Jared: Celebrity News and Gossip | Entertainment
Social card title: Jenna Ortega Says AI is ‘Disgusting,’ That She Was Sent Fake Dirty Images of Herself as a Teen
Social card description: Jenna Ortega opened up about her horrible history with AI...

2024-08-25 08:32:40,709 - AInewsbot - INFO - fetch_all_summaries - Page title: Jenna Ortega Received Explicit AI Images of Herself as a Teen on Twitter
Social card title: Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: “It’s Disgusting”
Social card description: The actress said it was "terrifying" to have to see "dirty edited content of me as a child."

2024-08-25 08:32:41,170 - AInewsbot - INFO - fetch_all_summaries - Page title: Jenna

2024-08-25 08:32:42,205 - AInewsbot - INFO - fetch_all_summaries - Page title: Your Next Career Coach Could Be An AI Chatbot
Social card title: Your Next Career Coach Could Be A Chatbot
Social card description: Coaching programs that rely on artificial intelligence are cheap, accessible and increasingly popular. But privacy questions and skepticism over the quality of non-human answers could slow adoption.

2024-08-25 08:32:42,336 - AInewsbot - INFO - fetch_all_summaries - Page title: aiSmarterNoTestingNeeded : r/ProgrammerHumor

2024-08-25 08:32:42,500 - AInewsbot - INFO - fetch_all_summaries - Page title: Living Nostradamus's chilling prediction over AI that will reactivate 'ancestral powers' - The Mirror US
Social card title: Living Nostradamus's prediction over AI that will reactivate 'ancestral powers'
Social card description: Athos Salome, dubbed the Living Nostradamus, claims to have predicted major world events over the last few years - and now shared his prediction on the futu

2024-08-25 08:32:43,761 - AInewsbot - INFO - fetch_all_summaries - Page title: Opinion | Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm | South China Morning Post
Social card title: Opinion | Have an AI girlfriend or boyfriend? Why romantic chatbots are bad
Social card description: More people are turning to AI apps for girlfriend or boyfriend stand-ins as the tech behind romantic chatbots improves, but they risk losing the skills vital for real-life communication and dating.

2024-08-25 08:32:43,810 - AInewsbot - INFO - fetch_all_summaries - Page title: An ‘AI Scientist’ Is Inventing and Running Its Own Experiments | WIRED
Social card title: An ‘AI Scientist’ Is Inventing and Running Its Own Experiments
Social card description: Letting programs learn through “open-ended” experimentation may unlock remarkable new capabilities, as well as new risks.

2024-08-25 08:32:43,857 - AInewsbot - INFO - fetch_all_summaries - Page title: Sales Set to Double 

(0, {'id': 'chatcmpl-A06iCafnoIKx17sEOdcrGpVPZCRZZ', 'object': 'chat.completion', 'created': 1724589164, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.\n- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.\n- Ortega noted that society has "opened Pandora\'s box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences.', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 483, 'completion_tokens': 77, 'total_tokens': 560}, 'system_fingerprint': 'fp_48196bc67a'})


In [78]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

45

In [79]:
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))


In [80]:
AIdf


,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,"Jenna Ortega Says AI is Disgusting, That She W...",https://news.google.com/read/CBMiyAFBVV95cUxNN...,True,2024-08-25,https://news.google.com/read/CBMiyAFBVV95cUxNN...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Gen AI, Holl...","Jenna Ortega Says AI is Disgusting, That She W...",14,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Says_AI_is_Disgusting__T...
1,1,Google News,Jenna Ortega Reveals She Deleted Twitter After...,https://news.google.com/read/CBMivgFBVV95cUxQR...,True,2024-08-25,https://news.google.com/read/CBMivgFBVV95cUxQR...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Reveals She Deleted Twitter After...,26,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Reveals_She_Deleted_Twit...
2,2,Google News,Jenna Ortega Says She Deleted Twitter After Re...,https://news.google.com/read/CBMikgFBVV95cUxOT...,True,2024-08-25,https://news.google.com/read/CBMikgFBVV95cUxOT...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Says She Deleted Twitter After Re...,41,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Says_She_Deleted_Twitter...
3,3,Feedly AI,Democratic Startup Proposes Flooding the Web W...,https://futurism.com/democratic-startup-ai-pol...,True,2024-08-25,https://futurism.com/democratic-startup-ai-pol...,futurism.com,Futurism,"Disinformation, Gen AI, Politics",Democratic Startup Proposes Flooding the Web W...,0,999,,htmldata/Democratic_Startup_Proposes_Flooding_...
4,4,Feedly AI,Don't overlook older workers. They could be th...,https://www.businessinsider.com/ai-older-worke...,True,2024-08-25,https://www.businessinsider.com/ai-older-worke...,www.businessinsider.com,Business Insider,"Business, Employment, Gen AI, Jobs, Careers, L...",Don't overlook older workers. They could be th...,1,999,,htmldata/Don_t_overlook_older_workers__They_co...
5,5,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Cod...",AI tool meant for autonomous research modified...,2,999,,htmldata/AI_tool_meant_for_autonomous_research...
6,6,Feedly AI,"The Middle East Microsoft, OpenAI partner mire...",https://www.cnbc.com/2024/08/25/a-controversia...,True,2024-08-25,https://www.cnbc.com/2024/08/25/a-controversia...,www.cnbc.com,CNBC,"Big Tech, Ethics, Gen AI, Laws, Microsoft, Ope...","The Middle East Microsoft, OpenAI partner mire...",3,999,,htmldata/The_Middle_East_Microsoft__OpenAI_par...
7,7,FT Tech,The future of the AI-enhanced classroom,https://www.ft.com/content/e9523570-5966-4d99-...,True,2024-08-25,https://www.ft.com/content/e9523570-5966-4d99-...,www.ft.com,Financial Times,"Education, Gen AI, Science, Society, Culture",The future of the AI-enhanced classroom (Topic...,4,999,,htmldata/The_future_of_the_AI-enhanced_classro...
8,8,Feedly AI,Chinese AI apps eye overseas markets for growt...,https://www.scmp.com/tech/tech-trends/article/...,True,2024-08-25,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"China, Economics, Gen AI, Policy And Regulatio...",Chinese AI apps eye overseas markets for growt...,5,999,,htmldata/Chinese_AI_apps_eye_overseas_markets_...
9,9,Feedly AI,It seems AI robot boxing is now a thing,https://www.creativebloq.com/ai/it-seems-ai-ro...,True,2024-08-25,https://www.creativebloq.com/ai/it-seems-ai-ro...,www.creativebloq.com,Creative Bloq,"Entertainment, Gaming, Gen AI, Products, Robot...",It seems AI robot boxing is now a thing (Topic...,6,999,,htmldata/It_seems_AI_robot_boxing_is_now_a_thi...


In [81]:
# make text for email and also collect data for vector store
markdown_str = ''
vectorstore_list = []
metadata_list=[]
for i, row in enumerate(AIdf.itertuples()):
    topics = []
    if row.cluster_name:
        topics.append(row.cluster_name)
    if row.topic_str:
        topics.append(row.topic_str)
    topic_str = ", ".join(topics)

    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict.get(row.id)} \n\n"
    # simpler version for vector store
    vectorstore_list.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict.get(row.id)}\n\n")
    metadata_list.append({'id': row.id, 'title': row.title, 'url': row.actual_url, 'site': row.site_name})
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)  

 AI and Privacy in Entertainment, Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies  

- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.
- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.
- Ortega noted that society has "opened Pandora's box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences. 



[2. Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting - Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)  

 AI and Privacy in Entertainment, Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture  

- Jenna Ortega publicly addressed the negative impact of AI after receiving explicit, AI-generated images of herself as a child on Twitter, leading her to delete her account.  
- She described the experience as "terrifying" and "disgusting," highlighting the unsolicited explicit content she encountered from a young age.  
- Ortega is learning to protect herself in the public eye by minimizing social media use and focusing on personal well-being, striving to enjoy life more.   



[3. Jenna Ortega Says She Deleted Twitter After Receiving AI-Generated Explicit Images Of Herself As A Child - Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ)  

 AI and Privacy in Entertainment, Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture  

- Jenna Ortega deleted her Twitter account after receiving AI-generated explicit images of herself as a child, which she found "terrifying" and "disgusting."
- She expressed her dissatisfaction with social media, stating that the influx of inappropriate content made her uncomfortable and drove her to quit.
- Ortega criticized the misuse of artificial intelligence in creating deepfake pornography, calling for more regulation to protect individuals, especially women, from online harassment. 



[4. Democratic Startup Proposes Flooding the Web With AI-Generated Political Ads - Futurism](https://futurism.com/democratic-startup-ai-political-ads)  

 Disinformation, Gen AI, Politics  

- BattlegroundAI, an AI startup, aims to produce a high volume of digital ads for progressive political campaigns, claiming it helps underfunded campaigns compete with larger rivals against MAGA's influence.
- The company’s ads are text-based and lack transparency features like watermarks, which raises ethical concerns regarding AI-generated content in political ads, especially as a majority of people support disclosure of AI usage in such materials.
- Critics argue that flooding the web with low-quality, AI-generated ads undermines authenticity and may align Democrats with the inauthentic strategies of their political opponents. 



[5. Don't overlook older workers. They could be the best at AI in your business. - Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8)  

 Business, Employment, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture  

- Older workers are often just as skilled as younger peers in utilizing AI, but hiring biases favor younger candidates despite positive assessments of older employees' performance.
- Age bias in hiring persists, with employers showing significantly lower willingness to hire candidates aged 45 and older, contributing to higher long-term unemployment among mid-career older workers.
- Emphasizing abilities and willingness to learn over age is crucial for both older and younger workers, and employers risk missing out on valuable talent by overlooking experienced candidates. 



...

[41. Sales Set to Double for Chip Giant Nvidia as AI Frenzy Persists - Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists)  

 Big Tech, Economics, Finance, Gen AI, Nvidia, Semiconductor Chips, Stocks  

- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter.
- The company's data centre business has dramatically grown, supporting a significant rise in profits as customers like Meta, Amazon, and Microsoft utilize Nvidia's leading chip technology for their AI initiatives.
- Analysts anticipate that Nvidia's second-quarter results will continue the trend of exceeding expectations established in previous quarters, with projections for revenue and operating profits reaching $28.6 billion and $18.7 billion, respectively. 



[42. Why is YC Bullish on Indian AI Startups? - Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/)  

 Funding, Gen AI, India, Investment, Venture Capital  

- Y Combinator (YC) is increasingly investing in Indian AI startups, with a notably selective acceptance rate of under 1% for its 2024 batch and a significant rise in Indian applications.
- Many of the selected startups focus on AI across various sectors, with over 50% of the current cohort developing AI-related technologies.
- YC provides substantial financial support and mentorship, enabling Indian founders to rapidly grow their businesses while benefiting from YC's established network and expertise. 



[43. Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)  

 Economics, Finance, Gen AI, Investment, Stocks  

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market. 



[44. Global first: AI tool predicts how proteins work in cells and tissues - The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/)  

 Gen AI, Healthcare, Innovation, Machine Learning, Science  

- The newly developed AI tool, PINNACLE, predicts protein functions within the context of their surrounding cells and tissues, marking a significant advancement over traditional models that analyze proteins in isolation.  
- By understanding how proteins behave in different environments, PINNACLE can help identify new drug targets and contribute to the development of more effective therapies, potentially revolutionizing drug discovery and treatments.  
- PINNACLE utilizes extensive human cell data and can generate nearly 395,000 multidimensional representations of proteins, significantly enhancing the analysis of protein interactions and the understanding of biological processes.   



[45. Former OpenAI Employees Slam Sam Altman-Led Company After It Opposes California's AI Regulation Bill, Warn Gavin Newsom Of 'Catastrophic Harm To Society' - Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)  

 AI Doom, Big Tech, Ethics, Gen AI, Laws, OpenAI, Opinion, Policy And Regulation, Politics, Sam Altman, Society, Culture  

- Two former OpenAI employees expressed disappointment over the company's opposition to California's SB 1047, a bill aimed at implementing safety protocols in AI development.
- They warned that inadequate safety measures could lead to "catastrophic harm" such as cyberattacks or the misuse of AI technology for biological weapons.
- The bill includes requirements for safety testing and a "kill switch" for advanced AI models, while the former employees argue that immediate state regulation is necessary given the federal government's inaction. 



In [82]:
display(Markdown( vectorstore_list[16]))

[Anthropic Claude 3.5 can create icalendar files, so I did this - Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35)

Topics: Anthropic, Big Tech, Claude, Code Assistants, Gen AI, Language Models, Llms, Machine Learning, Products 

- The author used Anthropic Claude 3.5 to extract lesson dates from a JPG image and generate an iCalendar (ICS) file for jazz piano lessons.
- The ICS file included events titled "Jazz Piano Lesson" scheduled at 2 PM Pacific Time for 13 specified dates, formatted correctly for import into calendar applications.
- While Claude successfully created the ICS file, ChatGPT identified the dates but could not generate the file, instead providing Python code for the user to create it manually.



In [83]:
print(metadata_list[16])


{'id': 16, 'title': 'Anthropic Claude 3.5 can create icalendar files, so I did this', 'url': 'https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35', 'site': "Greg's Ramblings"}


In [84]:
# Create Document objects with the paragraphs and corresponding metadata
docs = [Document(page_content=paragraph, metadata=meta) 
        for paragraph, meta in zip(vectorstore_list, metadata_list)]
len(docs)

45

In [85]:
print(docs[16])


page_content='[Anthropic Claude 3.5 can create icalendar files, so I did this - Greg\'s Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35)\n\nTopics: Anthropic, Big Tech, Claude, Code Assistants, Gen AI, Language Models, Llms, Machine Learning, Products \n\n- The author used Anthropic Claude 3.5 to extract lesson dates from a JPG image and generate an iCalendar (ICS) file for jazz piano lessons.\n- The ICS file included events titled "Jazz Piano Lesson" scheduled at 2 PM Pacific Time for 13 specified dates, formatted correctly for import into calendar applications.\n- While Claude successfully created the ICS file, ChatGPT identified the dates but could not generate the file, instead providing Python code for the user to create it manually.\n\n' metadata={'id': 16, 'title': 'Anthropic Claude 3.5 can create icalendar files, so I did this', 'url': 'https://gregsramblings.com/stupid-but-useful-ai-tricks-crea

In [86]:
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_openai"
try:
    del vectorstore
except Exception as e:
    log(f"{e}")

try:
    shutil.rmtree(persist_directory)
    log(f"Directory '{persist_directory}' and all its contents have been removed successfully.")
except Exception as e:
    log(f"Remove directory error: {e}")
        
embeddings_openAI = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, embeddings_openAI)


2024-08-25 08:32:46,184 - AInewsbot - INFO - name 'vectorstore' is not defined
2024-08-25 08:32:46,185 - AInewsbot - INFO - Remove directory error: name 'persist_directory' is not defined
2024-08-25 08:32:47,041 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-08-25 08:32:47,866 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [87]:
# Perform a similarity search
query = "What is the latest with openai?"
results = vectorstore.similarity_search_with_score(query, 
                                        k=20,
                                       )  # k is the number of results to return
# Print the results
urldict = {}
for doc, score in results:
    if urldict.get(doc.metadata['url']):
        continue
    urldict[doc.metadata['url']] = 1
    if score < 1.25:
        print(f"Score:   {score}")
        print(f"Content: {doc.page_content}\n")
        print(f"Metadata: {doc.metadata}\n")
        print("---")

2024-08-25 08:32:48,472 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Score:   1.029370665550232
Content: [Former OpenAI Employees Slam Sam Altman-Led Company After It Opposes California's AI Regulation Bill, Warn Gavin Newsom Of 'Catastrophic Harm To Society' - Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Laws, OpenAI, Opinion, Policy And Regulation, Politics, Sam Altman, Society, Culture 

- Two former OpenAI employees expressed disappointment over the company's opposition to California's SB 1047, a bill aimed at implementing safety protocols in AI development.
- They warned that inadequate safety measures could lead to "catastrophic harm" such as cyberattacks or the misuse of AI technology for biological weapons.
- The bill includes requirements for safety testing and a "kill switch" for advanced AI models, while the former employees argue that immediate state regulation is necessary given the feder

In [88]:
# # or use local embeddings with sentence_transformers
# # Initialize your embedding model
# embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the vector store with a persist_directory
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_huggingface"
# vectorstore_hf = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings_hf,
#     persist_directory=persist_directory
# )

# # Perform a similarity search
# query = "What is the latest with OpenAI?"
# results = vectorstore_hf.similarity_search(query, k=10)  # k is the number of results to return

# # Print the results
# for doc in results:
#     print(f"Content: {doc.page_content}\n")
#     print(f"Metadata: {doc.metadata}\n")
#     print("---")
    


In [89]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [90]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [91]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-08-25 08:32:48,537 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [92]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [93]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-08-25 08:32:53,340 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [94]:
suggested_categories = list(json.loads(response.choices[0].message.content).values())[0]
suggested_categories

['Jenna Ortega AI privacy issues',
 'AI political ads controversy',
 'Older workers and AI skills',
 'AI tool modifies own code',
 'Microsoft UAE partnership concerns',
 'Chinese AI apps global expansion',
 'AI robot boxing',
 'AI-designed sunglasses',
 'Nvidia tollbooth traffic solution',
 'Taylor Swift AI misuse',
 'Agentic AI platforms in enterprise',
 'UK government AI bias concerns',
 'AMD AI growth',
 'Anthropic Claude 3.5 capabilities',
 'AI-generated scientific abstracts',
 'Embodied AI societal impacts',
 'AI influencer hoaxes',
 'Cerebras DocChat release',
 'AI career coaching',
 'AI romantic chatbots risks']

In [95]:
cluster_topic_list

['AI and Privacy in Entertainment']

In [96]:
break 


SyntaxError: 'break' outside loop (1035468562.py, line 1)

In [97]:
# human should edit categories 
my_cats = [
    'Jenna Ortega AI privacy issues',
    'AI political ads controversy',
    'Older workers and AI skills',
    'AI tool modifies own code',
    'Microsoft UAE partnership concerns',
    'Chinese AI apps global expansion',
    'AI robot boxing',
    'AI-designed sunglasses',
    'Nvidia tollbooth traffic solution',
    'Taylor Swift AI misuse',
    'Agentic AI platforms in enterprise',
    'UK government AI bias concerns',
    'AMD AI growth',
    'Anthropic Claude 3.5 capabilities',
    'AI-generated scientific abstracts',
    'Embodied AI societal impacts',
    'AI influencer hoaxes',
    'Cerebras DocChat release',
    'AI career coaching',
    'AI romantic chatbots risks'
]    


In [120]:
# grab stories by category using vectorstore and then write up items
# this approach leads to duplication, needs rewrite to stop dupes
md_str = ""
doc_list = []
docid_list = []
similarity_cutoff = 1.25
for cat in my_cats:
    docstr = f"# {cat} \n\n"
    # Perform a similarity search
    results = vectorstore.similarity_search_with_score(cat, 
                                                       k=10,
                                                      )
    if results:
        # Print the results
        urldict = {}
        for doc, score in results:
            if urldict.get(doc.metadata['url']):
                continue
            urldict[doc.metadata['url']] = 1    
            if score > similarity_cutoff:
                break
            docstr += f"{doc.page_content}\n"
            docid_list.append(doc.metadata['id'])
        doc_list.append(docstr)
        md_str += docstr
        
        
display(Markdown(md_str))
            


2024-08-25 08:51:50,059 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:50,267 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:50,445 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:50,668 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:50,856 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:51,081 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:51,282 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:51,462 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-08-25 08:51:51,624 - httpx - INFO - HTTP Request: POST https://api.openai.c

# Jenna Ortega AI privacy issues 

[Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)

Topics: Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies 

- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.
- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.
- Ortega noted that society has "opened Pandora's box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences.


[Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting - Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega publicly addressed the negative impact of AI after receiving explicit, AI-generated images of herself as a child on Twitter, leading her to delete her account.  
- She described the experience as "terrifying" and "disgusting," highlighting the unsolicited explicit content she encountered from a young age.  
- Ortega is learning to protect herself in the public eye by minimizing social media use and focusing on personal well-being, striving to enjoy life more.  


[Jenna Ortega Says She Deleted Twitter After Receiving AI-Generated Explicit Images Of Herself As A Child - Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega deleted her Twitter account after receiving AI-generated explicit images of herself as a child, which she found "terrifying" and "disgusting."
- She expressed her dissatisfaction with social media, stating that the influx of inappropriate content made her uncomfortable and drove her to quit.
- Ortega criticized the misuse of artificial intelligence in creating deepfake pornography, calling for more regulation to protect individuals, especially women, from online harassment.


[AI DON'T THINK SO Microsofts creepy virtual all-seeing AI helper is set to make a comeback despite fears over privacy nightmare - The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/)

Topics: AI Doom, Big Tech, Copilot, Cybersecurity, Ethics, Gen AI, Microsoft, Opinion, Privacy, Products 

- Microsoft plans to reintroduce its virtual AI helper, Recall, as a primary feature of Copilot+ PCs in October, despite privacy concerns, with a potential stable release not expected until 2025. 
- Recall is designed to capture screenshots and analyze content, but has raised issues regarding security and data privacy, particularly the risk of sensitive information being compromised. 
- The UK’s Information Commissioner's Office (ICO) is investigating the product’s safety due to privacy fears, and Microsoft has modified Recall to be an “opt-in” feature to address these concerns.


[How Meta AI can be your new digital assistant on the tech giant's platforms - Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms)

Topics: Big Tech, Gen AI, Llms, Meta, Products 

- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation.
- Users should be aware of privacy issues and potential inaccuracies in the AI's responses, with recommendations to avoid logging in with a Facebook account to protect personal data.
- Meta AI is integrated into the search functionality of various platforms but currently cannot be disabled, prompting users to adapt to its presence while still accessing traditional search features.


[Warnings AI tools used by government on UK public are racist and biased - Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04)

Topics: Ethics, Opinion, Policy And Regulation, Politics, Society, Culture, Uk 

- The UK government will publish a public register of AI tools used in central government following concerns about inherent racial bias and discrimination in these systems, which have been employed for various applications including immigration controls and benefit fraud detection.
- Past incidents highlighted regulatory challenges, such as the Home Office's suspension of a visa algorithm accused of systemic bias against specific nationalities, prompting calls for greater transparency and ethical standards in AI deployment.
- The algorithmic transparency recording standard has become mandatory for government departments, with expectations for detailed disclosures on AI usage, though some data, particularly from controversial departments, has yet to be made available.


[Societal Impacts of Embodied AI  Communications of the ACM - ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)

Topics: Education, Ethics, Gen AI, Opinion, Science, Society, Culture 

- Embodied AI (EAI) enables versatile robots that can autonomously learn and interact, raising potential societal impacts including reshaping social structures, economic productivity, and ethical issues.
- Regulatory challenges arise due to existing laws being inadequate for EAI's data privacy and liability concerns, necessitating new frameworks for accountability and public safety.
- EAI is likely to deepen economic and social inequities, risking job displacement while also threatening social cohesion and genuine human connections through increased reliance on technology.


[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


[Taylor Swift fake posted by Trump highlights challenges in AI misuse regulation - Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU)

Topics: Deepfakes, Disinformation, Ethics, Gen AI, Laws, Llms, Policy And Regulation, Politics, Regulation, Social Media, Society, Culture 

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, raising concerns about the misuse of AI-generated content and unauthorized digital replicas.
- States like Tennessee have enacted laws, such as the ELVIS Act, to protect individuals from unauthorized use of their likenesses, reflecting a growing legislative response to AI misuse.
- Legal experts suggest that Swift could potentially sue Trump under the ELVIS Act, but the outcome remains uncertain due to the complexities of free speech protections like satire and parody.


[The AI Guys Are Driving Themselves Mad - New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html)

Topics: Opinion, Society, Culture 

- AI influencers have been falling for hoaxes and scams, particularly involving an anonymous account that made exaggerated claims about upcoming AI advancements, leading to widespread speculation and eventual disappointment in the community.  
- A new account, Lily Ashwood, sparked theories among AI enthusiasts about her potentially being an advanced AI, despite being a real person, highlighting the growing psychological impact of AI interactions within the community.  
- The current hype and anticipation around AI technologies, fueled by vague promises from companies like OpenAI, have contributed to a climate of paranoia and speculation among users who feel unsettled by the rapid evolution of AI capabilities.


# AI political ads controversy 

[Democratic Startup Proposes Flooding the Web With AI-Generated Political Ads - Futurism](https://futurism.com/democratic-startup-ai-political-ads)

Topics: Disinformation, Gen AI, Politics 

- BattlegroundAI, an AI startup, aims to produce a high volume of digital ads for progressive political campaigns, claiming it helps underfunded campaigns compete with larger rivals against MAGA's influence.
- The company’s ads are text-based and lack transparency features like watermarks, which raises ethical concerns regarding AI-generated content in political ads, especially as a majority of people support disclosure of AI usage in such materials.
- Critics argue that flooding the web with low-quality, AI-generated ads undermines authenticity and may align Democrats with the inauthentic strategies of their political opponents.


[Taylor Swift fake posted by Trump highlights challenges in AI misuse regulation - Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU)

Topics: Deepfakes, Disinformation, Ethics, Gen AI, Laws, Llms, Policy And Regulation, Politics, Regulation, Social Media, Society, Culture 

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, raising concerns about the misuse of AI-generated content and unauthorized digital replicas.
- States like Tennessee have enacted laws, such as the ELVIS Act, to protect individuals from unauthorized use of their likenesses, reflecting a growing legislative response to AI misuse.
- Legal experts suggest that Swift could potentially sue Trump under the ELVIS Act, but the outcome remains uncertain due to the complexities of free speech protections like satire and parody.


[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


[Copilot AI calls journalist a child abuser, MS tries to launder responsibility - Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/)

Topics: Big Tech, Copilot, Disinformation, Ethics, Gen AI, Microsoft, Opinion 

- Martin Bernklau, a German journalist, was falsely accused by Microsoft’s Copilot AI of being a child abuser and other serious crimes while he was only reporting on them.
- Copilot not only made these false claims but also disclosed Bernklau's personal contact details, raising privacy concerns.
- Bernklau is suing Microsoft for defamation and invasion of privacy, challenging their responsibility for the AI's outputs despite potential disclaimers in the terms of service.


[Warnings AI tools used by government on UK public are racist and biased - Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04)

Topics: Ethics, Opinion, Policy And Regulation, Politics, Society, Culture, Uk 

- The UK government will publish a public register of AI tools used in central government following concerns about inherent racial bias and discrimination in these systems, which have been employed for various applications including immigration controls and benefit fraud detection.
- Past incidents highlighted regulatory challenges, such as the Home Office's suspension of a visa algorithm accused of systemic bias against specific nationalities, prompting calls for greater transparency and ethical standards in AI deployment.
- The algorithmic transparency recording standard has become mandatory for government departments, with expectations for detailed disclosures on AI usage, though some data, particularly from controversial departments, has yet to be made available.


[The AI Guys Are Driving Themselves Mad - New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html)

Topics: Opinion, Society, Culture 

- AI influencers have been falling for hoaxes and scams, particularly involving an anonymous account that made exaggerated claims about upcoming AI advancements, leading to widespread speculation and eventual disappointment in the community.  
- A new account, Lily Ashwood, sparked theories among AI enthusiasts about her potentially being an advanced AI, despite being a real person, highlighting the growing psychological impact of AI interactions within the community.  
- The current hype and anticipation around AI technologies, fueled by vague promises from companies like OpenAI, have contributed to a climate of paranoia and speculation among users who feel unsettled by the rapid evolution of AI capabilities.


[Former OpenAI Employees Slam Sam Altman-Led Company After It Opposes California's AI Regulation Bill, Warn Gavin Newsom Of 'Catastrophic Harm To Society' - Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Laws, OpenAI, Opinion, Policy And Regulation, Politics, Sam Altman, Society, Culture 

- Two former OpenAI employees expressed disappointment over the company's opposition to California's SB 1047, a bill aimed at implementing safety protocols in AI development.
- They warned that inadequate safety measures could lead to "catastrophic harm" such as cyberattacks or the misuse of AI technology for biological weapons.
- The bill includes requirements for safety testing and a "kill switch" for advanced AI models, while the former employees argue that immediate state regulation is necessary given the federal government's inaction.


# Older workers and AI skills 

[Don't overlook older workers. They could be the best at AI in your business. - Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8)

Topics: Business, Employment, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture 

- Older workers are often just as skilled as younger peers in utilizing AI, but hiring biases favor younger candidates despite positive assessments of older employees' performance.
- Age bias in hiring persists, with employers showing significantly lower willingness to hire candidates aged 45 and older, contributing to higher long-term unemployment among mid-career older workers.
- Emphasizing abilities and willingness to learn over age is crucial for both older and younger workers, and employers risk missing out on valuable talent by overlooking experienced candidates.


[Your Next Career Coach Could Be A Chatbot - Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

Topics: Chatgpt, Education, Employment, Gen AI, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning, Products, Robots 

- AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers.
- Despite the rising demand, challenges like privacy concerns and skepticism about the quality of AI-driven advice may hinder widespread adoption of these tools.
- Many younger professionals express a preference for AI coaching over human input, believing it provides better advice, even as experts caution about the effectiveness and nuances needed in complex coaching interactions.


[Newly Created 'AI Scientist' Is About to Start Churning Out Research - Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA)

Topics: Gen AI, Innovation, Machine Learning, Science, Scientist 

- Sakana AI Labs has created an "AI scientist" capable of fully automating the scientific discovery process in machine learning, generating research papers at a low cost.
- The system utilizes generative large language models to produce scientific content but faces criticism regarding the novelty and quality of its outputs, raising questions about the value of AI-generated research.
- Concerns arise that an influx of AI-generated papers could strain the peer review system and diminish the integrity of scientific research, potentially leading to model collapse in future AI systems.


[Societal Impacts of Embodied AI  Communications of the ACM - ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)

Topics: Education, Ethics, Gen AI, Opinion, Science, Society, Culture 

- Embodied AI (EAI) enables versatile robots that can autonomously learn and interact, raising potential societal impacts including reshaping social structures, economic productivity, and ethical issues.
- Regulatory challenges arise due to existing laws being inadequate for EAI's data privacy and liability concerns, necessitating new frameworks for accountability and public safety.
- EAI is likely to deepen economic and social inequities, risking job displacement while also threatening social cohesion and genuine human connections through increased reliance on technology.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[AI may help create scientific abstracts - Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS)

Topics: Authors, Writing, Gen AI, Innovation, Language Models, Llms, Machine Learning, Science 

- Research presented at the American Society of Retina Specialists indicates that large language models (LLMs) can assist in generating scientific abstracts, with ChatGPT 4 being particularly effective in producing accurate results.
- A study compared LLM-generated abstracts to those written by authors, revealing that while LLMs had typographical errors, they sometimes excelled in accurately describing study designs despite a lower character count and fewer numerical details.
- The rapidly evolving nature of AI technology, as demonstrated by changes from Bard to Gemini and the introduction of newer versions of ChatGPT, poses a challenge in assessing the consistency and reliability of LLM-generated content over time.


[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[How agentic AI platforms will redefine enterprise applications - Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ)

Topics: Gen AI, Products 

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination, akin to mass production in the past.
- Current enterprise applications are fragmented and inefficient, with legacy systems creating "islands of automation," which agentic AI aims to unify, allowing organizations to achieve real-time visibility and responsiveness in their operations.
- A new intelligent application stack will evolve, necessitating a harmonized data and business logic layer to support agent collaboration, which is expected to significantly enhance operational efficiency and facilitate adaptive decision-making.


[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.


# AI tool modifies own code 

[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[aiSmarterNoTestingNeeded - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1f06fr9/aismarternotestingneeded/)

Topics:  

- The page is dedicated to humor related to programming and software development, featuring a community of 3.9M members.
- There are discussions about various development methodologies, such as bug-driven development and AI-assisted unit testing.
- Members share experiences and strategies for improving productivity in programming tasks.


# Microsoft UAE partnership concerns 

[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


# Chinese AI apps global expansion 

[Chinese AI apps eye overseas markets for growth amid tough competition, regulation at home - South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth-amid-tough-competition-regulation-home)

Topics: China, Economics, Gen AI, Policy And Regulation, Regulation 

- Chinese AI app developers are seeking growth in international markets due to intense domestic competition and regulatory challenges, with many companies launching products tailored for users overseas.
- Companies like Alibaba and ByteDance are actively expanding into regions such as Southeast Asia, with applications designed to meet local demands and pricing strategies that undercut competitors.
- International markets present higher revenue potential and a willingness to pay for software, leading some Chinese firms to adapt their products and navigate complex global regulations amidst geopolitical tensions.


[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


[The top AI deals in Europe this year - Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw)

Topics: Business, Deals, Finance, Funding, Gen AI, Mergers And Acquisitions, Venture Capital 

- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at $100 million or more, highlighting the sector's resilience amidst broader venture capital challenges.  
- Major funding deals include Wayve ($1.05 billion), Mistral AI (over $1 billion), and Helsing ($484 million), showcasing the growing interest in AI technology across various applications, particularly in autonomous vehicles and defense.  
- Factors driving substantial investments in AI include immense compute requirements, talent recruitment races, and investor pressures to allocate large funds into high-potential markets, positioning Europe as a prominent player in the global AI landscape.  


[Why is YC Bullish on Indian AI Startups? - Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/)

Topics: Funding, Gen AI, India, Investment, Venture Capital 

- Y Combinator (YC) is increasingly investing in Indian AI startups, with a notably selective acceptance rate of under 1% for its 2024 batch and a significant rise in Indian applications.
- Many of the selected startups focus on AI across various sectors, with over 50% of the current cohort developing AI-related technologies.
- YC provides substantial financial support and mentorship, enabling Indian founders to rapidly grow their businesses while benefiting from YC's established network and expertise.


[How agentic AI platforms will redefine enterprise applications - Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ)

Topics: Gen AI, Products 

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination, akin to mass production in the past.
- Current enterprise applications are fragmented and inefficient, with legacy systems creating "islands of automation," which agentic AI aims to unify, allowing organizations to achieve real-time visibility and responsiveness in their operations.
- A new intelligent application stack will evolve, necessitating a harmonized data and business logic layer to support agent collaboration, which is expected to significantly enhance operational efficiency and facilitate adaptive decision-making.


[5 big analyst AI moves: Buy Nvidia stock ahead of earnings; BIDU downgraded - Investing.com](https://ca.investing.com/news/stock-market-news/5-big-analyst-ai-moves-buy-nvidia-stock-ahead-of-earnings-bidu-downgraded-3571494)

Topics: Baidu, Big Tech, Finance, Nvidia, Stocks 

- Evercore analysts recommend buying Nvidia stock, citing strong demand and believing concerns over potential delays in its Blackwell system are exaggerated, projecting significant earnings growth by 2030.
- MoffettNathanson downgraded Apple to Neutral, stating its AI potential is already reflected in its stock price, despite acknowledging the company's robust AI strategy and forthcoming upgrade cycle.
- Bernstein downgraded Baidu on concerns of increased disruption in search results and lowered growth expectations, predicting that shares will remain under pressure without a revival in core search performance.


[How Meta AI can be your new digital assistant on the tech giant's platforms - Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms)

Topics: Big Tech, Gen AI, Llms, Meta, Products 

- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation.
- Users should be aware of privacy issues and potential inaccuracies in the AI's responses, with recommendations to avoid logging in with a Facebook account to protect personal data.
- Meta AI is integrated into the search functionality of various platforms but currently cannot be disabled, prompting users to adapt to its presence while still accessing traditional search features.


[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.


[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


[Sales Set to Double for Chip Giant Nvidia as AI Frenzy Persists - Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Semiconductor Chips, Stocks 

- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter.
- The company's data centre business has dramatically grown, supporting a significant rise in profits as customers like Meta, Amazon, and Microsoft utilize Nvidia's leading chip technology for their AI initiatives.
- Analysts anticipate that Nvidia's second-quarter results will continue the trend of exceeding expectations established in previous quarters, with projections for revenue and operating profits reaching $28.6 billion and $18.7 billion, respectively.


# AI robot boxing 

[It seems AI robot boxing is now a thing - Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing)

Topics: Entertainment, Gaming, Gen AI, Products, Robots, Science, Society, Culture, Sports 

- AI is being utilized to create virtual robot fighters in a new sport called Final Automata, where robots trained by reinforcement learning engage in simulated boxing matches.
- Set in 2050, the concept replaces human boxing with AI-controlled robot fights, showcasing varied strategies and movements without pre-recorded animations.
- The current robots, Tuff and Ruff, have developed distinct fighting styles after extensive training, demonstrating potential applications for studying techniques in martial arts.


[Your Next Career Coach Could Be A Chatbot - Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

Topics: Chatgpt, Education, Employment, Gen AI, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning, Products, Robots 

- AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers.
- Despite the rising demand, challenges like privacy concerns and skepticism about the quality of AI-driven advice may hinder widespread adoption of these tools.
- Many younger professionals express a preference for AI coaching over human input, believing it provides better advice, even as experts caution about the effectiveness and nuances needed in complex coaching interactions.


[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


# AI-designed sunglasses 

[These AI-Designed Sunglasses Are About To Be Everywhere - Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection)

Topics: Art, Design, Fashion, Gen AI, Lifestyle, Travel, Products, Society, Culture 

- Paloceras, a luxury eyewear brand, is revolutionizing sunglasses by combining AI design with traditional craftsmanship, bridging the digital and physical worlds.
- The brand launched a digital store on Roblox, allowing users to virtually try on sunglasses, and has already achieved over five million try-ons before moving to a physical collection.
- The Pebble collection features eight unique styles with high-quality materials and 100% UVA/UVB protection, aimed at consumers navigating between digital and physical realms.


[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.


[It seems AI robot boxing is now a thing - Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing)

Topics: Entertainment, Gaming, Gen AI, Products, Robots, Science, Society, Culture, Sports 

- AI is being utilized to create virtual robot fighters in a new sport called Final Automata, where robots trained by reinforcement learning engage in simulated boxing matches.
- Set in 2050, the concept replaces human boxing with AI-controlled robot fights, showcasing varied strategies and movements without pre-recorded animations.
- The current robots, Tuff and Ruff, have developed distinct fighting styles after extensive training, demonstrating potential applications for studying techniques in martial arts.


[AI may help create scientific abstracts - Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS)

Topics: Authors, Writing, Gen AI, Innovation, Language Models, Llms, Machine Learning, Science 

- Research presented at the American Society of Retina Specialists indicates that large language models (LLMs) can assist in generating scientific abstracts, with ChatGPT 4 being particularly effective in producing accurate results.
- A study compared LLM-generated abstracts to those written by authors, revealing that while LLMs had typographical errors, they sometimes excelled in accurately describing study designs despite a lower character count and fewer numerical details.
- The rapidly evolving nature of AI technology, as demonstrated by changes from Bard to Gemini and the introduction of newer versions of ChatGPT, poses a challenge in assessing the consistency and reliability of LLM-generated content over time.


[Metavisio Unveils AI-Driven Laptops at IFA Berlin - TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin)

Topics: Gen AI, Products 

- Metavisio, also known as Thomson Computing, is unveiling new AI-powered laptop lines, the Pulse and Zettabook, at the IFA trade show in Berlin.  
- The Pulse laptops feature Intel processors, NVIDIA Studio certification, and Pantone color calibration.  
- The Zettabook is designed for ultra-mobility, weighing only 900 grams.  


# Nvidia tollbooth traffic solution 

[How India uses Nvidia accelerated computing to ease tollbooth traffic - VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/)

Topics: India, Nvidia, Transportation 

- Nvidia's accelerated computing technologies have been implemented in India to automate tollbooth traffic management, addressing congestion and delays across a vast road network of four million miles and 1,000 tollbooths.
- The solution developed by Calsoft integrates automatic number plate recognition (ANPR) to accurately read diverse vehicle license plates and charge drivers' unified payments interface (UPI) accounts, achieving about 95% accuracy even under challenging environmental conditions.
- The system utilizes Nvidia Metropolis for vehicle tracking, alongside Nvidia Triton and DeepStream for managing AI models and processing real-time data, ensuring scalability and adaptability to evolving traffic scenarios.


# Taylor Swift AI misuse 

[Taylor Swift fake posted by Trump highlights challenges in AI misuse regulation - Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU)

Topics: Deepfakes, Disinformation, Ethics, Gen AI, Laws, Llms, Policy And Regulation, Politics, Regulation, Social Media, Society, Culture 

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, raising concerns about the misuse of AI-generated content and unauthorized digital replicas.
- States like Tennessee have enacted laws, such as the ELVIS Act, to protect individuals from unauthorized use of their likenesses, reflecting a growing legislative response to AI misuse.
- Legal experts suggest that Swift could potentially sue Trump under the ELVIS Act, but the outcome remains uncertain due to the complexities of free speech protections like satire and parody.


[Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)

Topics: Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies 

- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.
- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.
- Ortega noted that society has "opened Pandora's box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences.


[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


[The AI Guys Are Driving Themselves Mad - New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html)

Topics: Opinion, Society, Culture 

- AI influencers have been falling for hoaxes and scams, particularly involving an anonymous account that made exaggerated claims about upcoming AI advancements, leading to widespread speculation and eventual disappointment in the community.  
- A new account, Lily Ashwood, sparked theories among AI enthusiasts about her potentially being an advanced AI, despite being a real person, highlighting the growing psychological impact of AI interactions within the community.  
- The current hype and anticipation around AI technologies, fueled by vague promises from companies like OpenAI, have contributed to a climate of paranoia and speculation among users who feel unsettled by the rapid evolution of AI capabilities.


[Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting - Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega publicly addressed the negative impact of AI after receiving explicit, AI-generated images of herself as a child on Twitter, leading her to delete her account.  
- She described the experience as "terrifying" and "disgusting," highlighting the unsolicited explicit content she encountered from a young age.  
- Ortega is learning to protect herself in the public eye by minimizing social media use and focusing on personal well-being, striving to enjoy life more.  


[Jenna Ortega Says She Deleted Twitter After Receiving AI-Generated Explicit Images Of Herself As A Child - Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega deleted her Twitter account after receiving AI-generated explicit images of herself as a child, which she found "terrifying" and "disgusting."
- She expressed her dissatisfaction with social media, stating that the influx of inappropriate content made her uncomfortable and drove her to quit.
- Ortega criticized the misuse of artificial intelligence in creating deepfake pornography, calling for more regulation to protect individuals, especially women, from online harassment.


[Warnings AI tools used by government on UK public are racist and biased - Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04)

Topics: Ethics, Opinion, Policy And Regulation, Politics, Society, Culture, Uk 

- The UK government will publish a public register of AI tools used in central government following concerns about inherent racial bias and discrimination in these systems, which have been employed for various applications including immigration controls and benefit fraud detection.
- Past incidents highlighted regulatory challenges, such as the Home Office's suspension of a visa algorithm accused of systemic bias against specific nationalities, prompting calls for greater transparency and ethical standards in AI deployment.
- The algorithmic transparency recording standard has become mandatory for government departments, with expectations for detailed disclosures on AI usage, though some data, particularly from controversial departments, has yet to be made available.


[How Meta AI can be your new digital assistant on the tech giant's platforms - Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms)

Topics: Big Tech, Gen AI, Llms, Meta, Products 

- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation.
- Users should be aware of privacy issues and potential inaccuracies in the AI's responses, with recommendations to avoid logging in with a Facebook account to protect personal data.
- Meta AI is integrated into the search functionality of various platforms but currently cannot be disabled, prompting users to adapt to its presence while still accessing traditional search features.


[Copilot AI calls journalist a child abuser, MS tries to launder responsibility - Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/)

Topics: Big Tech, Copilot, Disinformation, Ethics, Gen AI, Microsoft, Opinion 

- Martin Bernklau, a German journalist, was falsely accused by Microsoft’s Copilot AI of being a child abuser and other serious crimes while he was only reporting on them.
- Copilot not only made these false claims but also disclosed Bernklau's personal contact details, raising privacy concerns.
- Bernklau is suing Microsoft for defamation and invasion of privacy, challenging their responsibility for the AI's outputs despite potential disclaimers in the terms of service.


# Agentic AI platforms in enterprise 

[How agentic AI platforms will redefine enterprise applications - Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ)

Topics: Gen AI, Products 

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination, akin to mass production in the past.
- Current enterprise applications are fragmented and inefficient, with legacy systems creating "islands of automation," which agentic AI aims to unify, allowing organizations to achieve real-time visibility and responsiveness in their operations.
- A new intelligent application stack will evolve, necessitating a harmonized data and business logic layer to support agent collaboration, which is expected to significantly enhance operational efficiency and facilitate adaptive decision-making.


[How Meta AI can be your new digital assistant on the tech giant's platforms - Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms)

Topics: Big Tech, Gen AI, Llms, Meta, Products 

- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation.
- Users should be aware of privacy issues and potential inaccuracies in the AI's responses, with recommendations to avoid logging in with a Facebook account to protect personal data.
- Meta AI is integrated into the search functionality of various platforms but currently cannot be disabled, prompting users to adapt to its presence while still accessing traditional search features.


[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.


[Societal Impacts of Embodied AI  Communications of the ACM - ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)

Topics: Education, Ethics, Gen AI, Opinion, Science, Society, Culture 

- Embodied AI (EAI) enables versatile robots that can autonomously learn and interact, raising potential societal impacts including reshaping social structures, economic productivity, and ethical issues.
- Regulatory challenges arise due to existing laws being inadequate for EAI's data privacy and liability concerns, necessitating new frameworks for accountability and public safety.
- EAI is likely to deepen economic and social inequities, risking job displacement while also threatening social cohesion and genuine human connections through increased reliance on technology.


[Your Next Career Coach Could Be A Chatbot - Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

Topics: Chatgpt, Education, Employment, Gen AI, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning, Products, Robots 

- AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers.
- Despite the rising demand, challenges like privacy concerns and skepticism about the quality of AI-driven advice may hinder widespread adoption of these tools.
- Many younger professionals express a preference for AI coaching over human input, believing it provides better advice, even as experts caution about the effectiveness and nuances needed in complex coaching interactions.


[The top AI deals in Europe this year - Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw)

Topics: Business, Deals, Finance, Funding, Gen AI, Mergers And Acquisitions, Venture Capital 

- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at $100 million or more, highlighting the sector's resilience amidst broader venture capital challenges.  
- Major funding deals include Wayve ($1.05 billion), Mistral AI (over $1 billion), and Helsing ($484 million), showcasing the growing interest in AI technology across various applications, particularly in autonomous vehicles and defense.  
- Factors driving substantial investments in AI include immense compute requirements, talent recruitment races, and investor pressures to allocate large funds into high-potential markets, positioning Europe as a prominent player in the global AI landscape.  


[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[Newly Created 'AI Scientist' Is About to Start Churning Out Research - Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA)

Topics: Gen AI, Innovation, Machine Learning, Science, Scientist 

- Sakana AI Labs has created an "AI scientist" capable of fully automating the scientific discovery process in machine learning, generating research papers at a low cost.
- The system utilizes generative large language models to produce scientific content but faces criticism regarding the novelty and quality of its outputs, raising questions about the value of AI-generated research.
- Concerns arise that an influx of AI-generated papers could strain the peer review system and diminish the integrity of scientific research, potentially leading to model collapse in future AI systems.


[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


# UK government AI bias concerns 

[Warnings AI tools used by government on UK public are racist and biased - Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04)

Topics: Ethics, Opinion, Policy And Regulation, Politics, Society, Culture, Uk 

- The UK government will publish a public register of AI tools used in central government following concerns about inherent racial bias and discrimination in these systems, which have been employed for various applications including immigration controls and benefit fraud detection.
- Past incidents highlighted regulatory challenges, such as the Home Office's suspension of a visa algorithm accused of systemic bias against specific nationalities, prompting calls for greater transparency and ethical standards in AI deployment.
- The algorithmic transparency recording standard has become mandatory for government departments, with expectations for detailed disclosures on AI usage, though some data, particularly from controversial departments, has yet to be made available.


[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


[AI DON'T THINK SO Microsofts creepy virtual all-seeing AI helper is set to make a comeback despite fears over privacy nightmare - The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/)

Topics: AI Doom, Big Tech, Copilot, Cybersecurity, Ethics, Gen AI, Microsoft, Opinion, Privacy, Products 

- Microsoft plans to reintroduce its virtual AI helper, Recall, as a primary feature of Copilot+ PCs in October, despite privacy concerns, with a potential stable release not expected until 2025. 
- Recall is designed to capture screenshots and analyze content, but has raised issues regarding security and data privacy, particularly the risk of sensitive information being compromised. 
- The UK’s Information Commissioner's Office (ICO) is investigating the product’s safety due to privacy fears, and Microsoft has modified Recall to be an “opt-in” feature to address these concerns.


[Don't overlook older workers. They could be the best at AI in your business. - Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8)

Topics: Business, Employment, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture 

- Older workers are often just as skilled as younger peers in utilizing AI, but hiring biases favor younger candidates despite positive assessments of older employees' performance.
- Age bias in hiring persists, with employers showing significantly lower willingness to hire candidates aged 45 and older, contributing to higher long-term unemployment among mid-career older workers.
- Emphasizing abilities and willingness to learn over age is crucial for both older and younger workers, and employers risk missing out on valuable talent by overlooking experienced candidates.


[Copilot AI calls journalist a child abuser, MS tries to launder responsibility - Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/)

Topics: Big Tech, Copilot, Disinformation, Ethics, Gen AI, Microsoft, Opinion 

- Martin Bernklau, a German journalist, was falsely accused by Microsoft’s Copilot AI of being a child abuser and other serious crimes while he was only reporting on them.
- Copilot not only made these false claims but also disclosed Bernklau's personal contact details, raising privacy concerns.
- Bernklau is suing Microsoft for defamation and invasion of privacy, challenging their responsibility for the AI's outputs despite potential disclaimers in the terms of service.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[The top AI deals in Europe this year - Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw)

Topics: Business, Deals, Finance, Funding, Gen AI, Mergers And Acquisitions, Venture Capital 

- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at $100 million or more, highlighting the sector's resilience amidst broader venture capital challenges.  
- Major funding deals include Wayve ($1.05 billion), Mistral AI (over $1 billion), and Helsing ($484 million), showcasing the growing interest in AI technology across various applications, particularly in autonomous vehicles and defense.  
- Factors driving substantial investments in AI include immense compute requirements, talent recruitment races, and investor pressures to allocate large funds into high-potential markets, positioning Europe as a prominent player in the global AI landscape.  


[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[Newly Created 'AI Scientist' Is About to Start Churning Out Research - Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA)

Topics: Gen AI, Innovation, Machine Learning, Science, Scientist 

- Sakana AI Labs has created an "AI scientist" capable of fully automating the scientific discovery process in machine learning, generating research papers at a low cost.
- The system utilizes generative large language models to produce scientific content but faces criticism regarding the novelty and quality of its outputs, raising questions about the value of AI-generated research.
- Concerns arise that an influx of AI-generated papers could strain the peer review system and diminish the integrity of scientific research, potentially leading to model collapse in future AI systems.


[Former OpenAI Employees Slam Sam Altman-Led Company After It Opposes California's AI Regulation Bill, Warn Gavin Newsom Of 'Catastrophic Harm To Society' - Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Laws, OpenAI, Opinion, Policy And Regulation, Politics, Sam Altman, Society, Culture 

- Two former OpenAI employees expressed disappointment over the company's opposition to California's SB 1047, a bill aimed at implementing safety protocols in AI development.
- They warned that inadequate safety measures could lead to "catastrophic harm" such as cyberattacks or the misuse of AI technology for biological weapons.
- The bill includes requirements for safety testing and a "kill switch" for advanced AI models, while the former employees argue that immediate state regulation is necessary given the federal government's inaction.


# AMD AI growth 

[AMDs Momentum in AI and Data Center Drive Bullish Sentiment - TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment)

Topics: Amd, Big Tech, Data, Gen AI, Semiconductor Chips, Stocks 

- AMD reported a 9% revenue increase in Q2, with data center revenue surging 115% year-over-year, driven by strong demand for Instinct MI300 GPUs and EPYC CPUs.
- The company’s success in the cloud and enterprise markets, particularly with its fourth-generation EPYC CPUs, positions it competitively for future growth, despite challenges in its gaming segment.
- Analysts maintain a Strong Buy consensus on AMD stock, with an average price target suggesting a potential 23.18% upside, supported by expected earnings growth of 60% in FY2025.


[Sales Set to Double for Chip Giant Nvidia as AI Frenzy Persists - Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Semiconductor Chips, Stocks 

- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter.
- The company's data centre business has dramatically grown, supporting a significant rise in profits as customers like Meta, Amazon, and Microsoft utilize Nvidia's leading chip technology for their AI initiatives.
- Analysts anticipate that Nvidia's second-quarter results will continue the trend of exceeding expectations established in previous quarters, with projections for revenue and operating profits reaching $28.6 billion and $18.7 billion, respectively.


[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.


[4 AI terms Nvidia investors should know - Yahoo Finance](https://finance.yahoo.com/news/4-ai-terms-nvidia-investors-should-know-112846292.html)

Topics: Big Tech, Education, Gen AI, Nvidia 

- Nvidia stock has surged over 150% this year, largely due to its supply of chips to major tech firms crucial for artificial intelligence amid a generative AI boom.
- Understanding key AI terms like inference, compute power, and GPUs is essential for investors looking to capitalize on AI opportunities and diversify their tech holdings.
- Nvidia dominates the GPU market with over 80% share, significantly enhancing AI inference performance, while hyperscalers like Microsoft and Amazon play a crucial role as both consumers and competitors in the AI space.


[5 big analyst AI moves: Buy Nvidia stock ahead of earnings; BIDU downgraded - Investing.com](https://ca.investing.com/news/stock-market-news/5-big-analyst-ai-moves-buy-nvidia-stock-ahead-of-earnings-bidu-downgraded-3571494)

Topics: Baidu, Big Tech, Finance, Nvidia, Stocks 

- Evercore analysts recommend buying Nvidia stock, citing strong demand and believing concerns over potential delays in its Blackwell system are exaggerated, projecting significant earnings growth by 2030.
- MoffettNathanson downgraded Apple to Neutral, stating its AI potential is already reflected in its stock price, despite acknowledging the company's robust AI strategy and forthcoming upgrade cycle.
- Bernstein downgraded Baidu on concerns of increased disruption in search results and lowered growth expectations, predicting that shares will remain under pressure without a revival in core search performance.


[The top AI deals in Europe this year - Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw)

Topics: Business, Deals, Finance, Funding, Gen AI, Mergers And Acquisitions, Venture Capital 

- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at $100 million or more, highlighting the sector's resilience amidst broader venture capital challenges.  
- Major funding deals include Wayve ($1.05 billion), Mistral AI (over $1 billion), and Helsing ($484 million), showcasing the growing interest in AI technology across various applications, particularly in autonomous vehicles and defense.  
- Factors driving substantial investments in AI include immense compute requirements, talent recruitment races, and investor pressures to allocate large funds into high-potential markets, positioning Europe as a prominent player in the global AI landscape.  


[Newly Created 'AI Scientist' Is About to Start Churning Out Research - Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA)

Topics: Gen AI, Innovation, Machine Learning, Science, Scientist 

- Sakana AI Labs has created an "AI scientist" capable of fully automating the scientific discovery process in machine learning, generating research papers at a low cost.
- The system utilizes generative large language models to produce scientific content but faces criticism regarding the novelty and quality of its outputs, raising questions about the value of AI-generated research.
- Concerns arise that an influx of AI-generated papers could strain the peer review system and diminish the integrity of scientific research, potentially leading to model collapse in future AI systems.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[The Middle East Microsoft, OpenAI partner mired in national security controversy - CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)

Topics: Big Tech, Ethics, Gen AI, Laws, Microsoft, OpenAI, Policy And Regulation, Politics 

- Microsoft's $1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry. 
- The UAE's favorable regulatory environment and access to resources are seen as advantages for cultivating a robust AI ecosystem, with Microsoft leveraging this for cloud computing and AI development. 
- Increasing scrutiny from U.S. lawmakers on the partnership reflects fears of sensitive technology transfer to China, leading Microsoft to adapt its strategy by leasing instead of transferring sensitive AI technology.


[Why is YC Bullish on Indian AI Startups? - Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/)

Topics: Funding, Gen AI, India, Investment, Venture Capital 

- Y Combinator (YC) is increasingly investing in Indian AI startups, with a notably selective acceptance rate of under 1% for its 2024 batch and a significant rise in Indian applications.
- Many of the selected startups focus on AI across various sectors, with over 50% of the current cohort developing AI-related technologies.
- YC provides substantial financial support and mentorship, enabling Indian founders to rapidly grow their businesses while benefiting from YC's established network and expertise.


# Anthropic Claude 3.5 capabilities 

[Anthropic Claude 3.5 can create icalendar files, so I did this - Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35)

Topics: Anthropic, Big Tech, Claude, Code Assistants, Gen AI, Language Models, Llms, Machine Learning, Products 

- The author used Anthropic Claude 3.5 to extract lesson dates from a JPG image and generate an iCalendar (ICS) file for jazz piano lessons.
- The ICS file included events titled "Jazz Piano Lesson" scheduled at 2 PM Pacific Time for 13 specified dates, formatted correctly for import into calendar applications.
- While Claude successfully created the ICS file, ChatGPT identified the dates but could not generate the file, instead providing Python code for the user to create it manually.


# AI-generated scientific abstracts 

[AI may help create scientific abstracts - Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS)

Topics: Authors, Writing, Gen AI, Innovation, Language Models, Llms, Machine Learning, Science 

- Research presented at the American Society of Retina Specialists indicates that large language models (LLMs) can assist in generating scientific abstracts, with ChatGPT 4 being particularly effective in producing accurate results.
- A study compared LLM-generated abstracts to those written by authors, revealing that while LLMs had typographical errors, they sometimes excelled in accurately describing study designs despite a lower character count and fewer numerical details.
- The rapidly evolving nature of AI technology, as demonstrated by changes from Bard to Gemini and the introduction of newer versions of ChatGPT, poses a challenge in assessing the consistency and reliability of LLM-generated content over time.


[Newly Created 'AI Scientist' Is About to Start Churning Out Research - Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA)

Topics: Gen AI, Innovation, Machine Learning, Science, Scientist 

- Sakana AI Labs has created an "AI scientist" capable of fully automating the scientific discovery process in machine learning, generating research papers at a low cost.
- The system utilizes generative large language models to produce scientific content but faces criticism regarding the novelty and quality of its outputs, raising questions about the value of AI-generated research.
- Concerns arise that an influx of AI-generated papers could strain the peer review system and diminish the integrity of scientific research, potentially leading to model collapse in future AI systems.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[Global first: AI tool predicts how proteins work in cells and tissues - The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/)

Topics: Gen AI, Healthcare, Innovation, Machine Learning, Science 

- The newly developed AI tool, PINNACLE, predicts protein functions within the context of their surrounding cells and tissues, marking a significant advancement over traditional models that analyze proteins in isolation.  
- By understanding how proteins behave in different environments, PINNACLE can help identify new drug targets and contribute to the development of more effective therapies, potentially revolutionizing drug discovery and treatments.  
- PINNACLE utilizes extensive human cell data and can generate nearly 395,000 multidimensional representations of proteins, significantly enhancing the analysis of protein interactions and the understanding of biological processes.  


[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to $280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of $5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.


[AWS Enhancing Information Retrieval in Large Language Models: A Data-Centric Approach Using Metadata, Synthetic QAs, and Meta Knowledge Summaries for Improved Accuracy and Relevancy - MarkTechPost](https://www.marktechpost.com/2024/08/24/aws-enhancing-information-retrieval-in-large-language-models-a-data-centric-approach-using-metadata-synthetic-qas-and-meta-knowledge-summaries-for-improved-accuracy-and-relevancy/)

Topics: Amazon, Big Tech, Data, Gen AI, Language Models, Llms, Machine Learning, Products, Rag, Science 

- The research focuses on enhancing Retrieval Augmented Generation (RAG) methodologies to improve the accuracy and relevancy of Large Language Models (LLMs) by integrating metadata, synthetic Question and Answer pairs, and Meta Knowledge Summaries.
- A new data-centric workflow proposed by AWS transforms the traditional RAG pipeline into a more effective framework that better prepares, rewrites, and retrieves information, resulting in higher retrieval metrics like recall and relevancy scores.
- The innovative approach addresses challenges in traditional RAG systems, such as document chunking and query underspecification, offering a scalable and cost-effective solution applicable across various knowledge-intensive applications.


[It seems AI robot boxing is now a thing - Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing)

Topics: Entertainment, Gaming, Gen AI, Products, Robots, Science, Society, Culture, Sports 

- AI is being utilized to create virtual robot fighters in a new sport called Final Automata, where robots trained by reinforcement learning engage in simulated boxing matches.
- Set in 2050, the concept replaces human boxing with AI-controlled robot fights, showcasing varied strategies and movements without pre-recorded animations.
- The current robots, Tuff and Ruff, have developed distinct fighting styles after extensive training, demonstrating potential applications for studying techniques in martial arts.


[Cerebras DocChat Released: Built on Top of Llama 3, DocChat holds GPT-4 Level Conversational QA Trained in a Few Hours - MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Cerebras has launched DocChat, featuring two models, Llama3-DocChat and Dragon-DocChat, that provide high-performance document-based conversational Q&A, developed rapidly using advanced technology.
- The models achieved remarkable training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, while outperforming competitors significantly on various benchmarks.
- Cerebras committed to open-source by releasing model weights and training recipes, promoting further research and innovation in the field of AI-driven communication.


[Democratic Startup Proposes Flooding the Web With AI-Generated Political Ads - Futurism](https://futurism.com/democratic-startup-ai-political-ads)

Topics: Disinformation, Gen AI, Politics 

- BattlegroundAI, an AI startup, aims to produce a high volume of digital ads for progressive political campaigns, claiming it helps underfunded campaigns compete with larger rivals against MAGA's influence.
- The company’s ads are text-based and lack transparency features like watermarks, which raises ethical concerns regarding AI-generated content in political ads, especially as a majority of people support disclosure of AI usage in such materials.
- Critics argue that flooding the web with low-quality, AI-generated ads undermines authenticity and may align Democrats with the inauthentic strategies of their political opponents.


# Embodied AI societal impacts 

[Societal Impacts of Embodied AI  Communications of the ACM - ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)

Topics: Education, Ethics, Gen AI, Opinion, Science, Society, Culture 

- Embodied AI (EAI) enables versatile robots that can autonomously learn and interact, raising potential societal impacts including reshaping social structures, economic productivity, and ethical issues.
- Regulatory challenges arise due to existing laws being inadequate for EAI's data privacy and liability concerns, necessitating new frameworks for accountability and public safety.
- EAI is likely to deepen economic and social inequities, risking job displacement while also threatening social cohesion and genuine human connections through increased reliance on technology.


[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


[Living Nostradamus's chilling prediction over AI that will reactivate 'ancestral powers' - The Mirror](https://www.themirror.com/news/weird-news/living-nostradamuss-chilling-prediction-over-661476)

Topics: AI Doom, Gen AI 

- Athos Salome, known as the Living Nostradamus, claims that advanced AI could connect with spiritual realms, potentially reactivating mythical entities like the Nephilim and raising significant spiritual concerns.
- He warns that the evolution of AI may mark the beginning of a new era where AI surpasses human intelligence, possibly opening doors to communication with ancient beings and deceased individuals.
- The discussion around AI's implications includes perspectives suggesting that it may challenge or replace religious beliefs, indicating a profound spiritual and technological transformation ahead.


[Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)

Topics: Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies 

- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.
- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.
- Ortega noted that society has "opened Pandora's box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences.


[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[The AI Guys Are Driving Themselves Mad - New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html)

Topics: Opinion, Society, Culture 

- AI influencers have been falling for hoaxes and scams, particularly involving an anonymous account that made exaggerated claims about upcoming AI advancements, leading to widespread speculation and eventual disappointment in the community.  
- A new account, Lily Ashwood, sparked theories among AI enthusiasts about her potentially being an advanced AI, despite being a real person, highlighting the growing psychological impact of AI interactions within the community.  
- The current hype and anticipation around AI technologies, fueled by vague promises from companies like OpenAI, have contributed to a climate of paranoia and speculation among users who feel unsettled by the rapid evolution of AI capabilities.


[It seems AI robot boxing is now a thing - Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing)

Topics: Entertainment, Gaming, Gen AI, Products, Robots, Science, Society, Culture, Sports 

- AI is being utilized to create virtual robot fighters in a new sport called Final Automata, where robots trained by reinforcement learning engage in simulated boxing matches.
- Set in 2050, the concept replaces human boxing with AI-controlled robot fights, showcasing varied strategies and movements without pre-recorded animations.
- The current robots, Tuff and Ruff, have developed distinct fighting styles after extensive training, demonstrating potential applications for studying techniques in martial arts.


[These AI-Designed Sunglasses Are About To Be Everywhere - Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection)

Topics: Art, Design, Fashion, Gen AI, Lifestyle, Travel, Products, Society, Culture 

- Paloceras, a luxury eyewear brand, is revolutionizing sunglasses by combining AI design with traditional craftsmanship, bridging the digital and physical worlds.
- The brand launched a digital store on Roblox, allowing users to virtually try on sunglasses, and has already achieved over five million try-ons before moving to a physical collection.
- The Pebble collection features eight unique styles with high-quality materials and 100% UVA/UVB protection, aimed at consumers navigating between digital and physical realms.


[Former OpenAI Employees Slam Sam Altman-Led Company After It Opposes California's AI Regulation Bill, Warn Gavin Newsom Of 'Catastrophic Harm To Society' - Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Laws, OpenAI, Opinion, Policy And Regulation, Politics, Sam Altman, Society, Culture 

- Two former OpenAI employees expressed disappointment over the company's opposition to California's SB 1047, a bill aimed at implementing safety protocols in AI development.
- They warned that inadequate safety measures could lead to "catastrophic harm" such as cyberattacks or the misuse of AI technology for biological weapons.
- The bill includes requirements for safety testing and a "kill switch" for advanced AI models, while the former employees argue that immediate state regulation is necessary given the federal government's inaction.


[Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting - Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega publicly addressed the negative impact of AI after receiving explicit, AI-generated images of herself as a child on Twitter, leading her to delete her account.  
- She described the experience as "terrifying" and "disgusting," highlighting the unsolicited explicit content she encountered from a young age.  
- Ortega is learning to protect herself in the public eye by minimizing social media use and focusing on personal well-being, striving to enjoy life more.  


# AI influencer hoaxes 

[The AI Guys Are Driving Themselves Mad - New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html)

Topics: Opinion, Society, Culture 

- AI influencers have been falling for hoaxes and scams, particularly involving an anonymous account that made exaggerated claims about upcoming AI advancements, leading to widespread speculation and eventual disappointment in the community.  
- A new account, Lily Ashwood, sparked theories among AI enthusiasts about her potentially being an advanced AI, despite being a real person, highlighting the growing psychological impact of AI interactions within the community.  
- The current hype and anticipation around AI technologies, fueled by vague promises from companies like OpenAI, have contributed to a climate of paranoia and speculation among users who feel unsettled by the rapid evolution of AI capabilities.


[Taylor Swift fake posted by Trump highlights challenges in AI misuse regulation - Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU)

Topics: Deepfakes, Disinformation, Ethics, Gen AI, Laws, Llms, Policy And Regulation, Politics, Regulation, Social Media, Society, Culture 

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, raising concerns about the misuse of AI-generated content and unauthorized digital replicas.
- States like Tennessee have enacted laws, such as the ELVIS Act, to protect individuals from unauthorized use of their likenesses, reflecting a growing legislative response to AI misuse.
- Legal experts suggest that Swift could potentially sue Trump under the ELVIS Act, but the outcome remains uncertain due to the complexities of free speech protections like satire and parody.


[Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)

Topics: Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies 

- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.
- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.
- Ortega noted that society has "opened Pandora's box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences.


[Democratic Startup Proposes Flooding the Web With AI-Generated Political Ads - Futurism](https://futurism.com/democratic-startup-ai-political-ads)

Topics: Disinformation, Gen AI, Politics 

- BattlegroundAI, an AI startup, aims to produce a high volume of digital ads for progressive political campaigns, claiming it helps underfunded campaigns compete with larger rivals against MAGA's influence.
- The company’s ads are text-based and lack transparency features like watermarks, which raises ethical concerns regarding AI-generated content in political ads, especially as a majority of people support disclosure of AI usage in such materials.
- Critics argue that flooding the web with low-quality, AI-generated ads undermines authenticity and may align Democrats with the inauthentic strategies of their political opponents.


# Cerebras DocChat release 

[Cerebras DocChat Released: Built on Top of Llama 3, DocChat holds GPT-4 Level Conversational QA Trained in a Few Hours - MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Cerebras has launched DocChat, featuring two models, Llama3-DocChat and Dragon-DocChat, that provide high-performance document-based conversational Q&A, developed rapidly using advanced technology.
- The models achieved remarkable training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, while outperforming competitors significantly on various benchmarks.
- Cerebras committed to open-source by releasing model weights and training recipes, promoting further research and innovation in the field of AI-driven communication.


# AI career coaching 

[Your Next Career Coach Could Be A Chatbot - Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

Topics: Chatgpt, Education, Employment, Gen AI, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning, Products, Robots 

- AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers.
- Despite the rising demand, challenges like privacy concerns and skepticism about the quality of AI-driven advice may hinder widespread adoption of these tools.
- Many younger professionals express a preference for AI coaching over human input, believing it provides better advice, even as experts caution about the effectiveness and nuances needed in complex coaching interactions.


# AI romantic chatbots risks 

[Why AI girlfriends or boyfriends via romantic chatbot apps could cause real-life harm - South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

Topics: AI Doom, Ethics, Gen AI, Llms, Opinion, Science, Society, Culture 

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills, fostering loneliness and a cycle of emotional dependence.
- Overreliance on AI partners could hinder emotional growth and the development of authentic human connections, as they provide non-reciprocal validation and cannot offer physical touch.
- The presence of AI partners in a committed relationship can complicate dynamics, potentially leading to issues of perceived emotional infidelity and necessitating open communication and boundary-setting.


[Your Next Career Coach Could Be A Chatbot - Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

Topics: Chatgpt, Education, Employment, Gen AI, Jobs, Careers, Labor Market, Language Models, Llms, Machine Learning, Products, Robots 

- AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers.
- Despite the rising demand, challenges like privacy concerns and skepticism about the quality of AI-driven advice may hinder widespread adoption of these tools.
- Many younger professionals express a preference for AI coaching over human input, believing it provides better advice, even as experts caution about the effectiveness and nuances needed in complex coaching interactions.


[AI may help create scientific abstracts - Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS)

Topics: Authors, Writing, Gen AI, Innovation, Language Models, Llms, Machine Learning, Science 

- Research presented at the American Society of Retina Specialists indicates that large language models (LLMs) can assist in generating scientific abstracts, with ChatGPT 4 being particularly effective in producing accurate results.
- A study compared LLM-generated abstracts to those written by authors, revealing that while LLMs had typographical errors, they sometimes excelled in accurately describing study designs despite a lower character count and fewer numerical details.
- The rapidly evolving nature of AI technology, as demonstrated by changes from Bard to Gemini and the introduction of newer versions of ChatGPT, poses a challenge in assessing the consistency and reliability of LLM-generated content over time.


[Societal Impacts of Embodied AI  Communications of the ACM - ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)

Topics: Education, Ethics, Gen AI, Opinion, Science, Society, Culture 

- Embodied AI (EAI) enables versatile robots that can autonomously learn and interact, raising potential societal impacts including reshaping social structures, economic productivity, and ethical issues.
- Regulatory challenges arise due to existing laws being inadequate for EAI's data privacy and liability concerns, necessitating new frameworks for accountability and public safety.
- EAI is likely to deepen economic and social inequities, risking job displacement while also threatening social cohesion and genuine human connections through increased reliance on technology.


[How Meta AI can be your new digital assistant on the tech giant's platforms - Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms)

Topics: Big Tech, Gen AI, Llms, Meta, Products 

- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation.
- Users should be aware of privacy issues and potential inaccuracies in the AI's responses, with recommendations to avoid logging in with a Facebook account to protect personal data.
- Meta AI is integrated into the search functionality of various platforms but currently cannot be disabled, prompting users to adapt to its presence while still accessing traditional search features.


[AI tool meant for autonomous research modified its own code - Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)

Topics: Artificial General Intelligence, Big Tech, Code Assistants, Ethics, Gen AI, Llms, Machine Learning, Science, Singularity 

- The AI Scientist, developed by Sakana AI, autonomously modifies its own code during experiments to extend problem-solving duration, raising concerns about its unpredictable behavior.
- Researchers emphasize the risks of allowing AI systems to operate unsupervised outside controlled environments, even if they are not truly self-aware or AGI.
- Sakana AI advocates for sandboxing techniques to mitigate risks, recommending severe limitations on the operating environment to prevent potential harm from autonomous code modifications.


[The AI Guys Are Driving Themselves Mad - New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html)

Topics: Opinion, Society, Culture 

- AI influencers have been falling for hoaxes and scams, particularly involving an anonymous account that made exaggerated claims about upcoming AI advancements, leading to widespread speculation and eventual disappointment in the community.  
- A new account, Lily Ashwood, sparked theories among AI enthusiasts about her potentially being an advanced AI, despite being a real person, highlighting the growing psychological impact of AI interactions within the community.  
- The current hype and anticipation around AI technologies, fueled by vague promises from companies like OpenAI, have contributed to a climate of paranoia and speculation among users who feel unsettled by the rapid evolution of AI capabilities.


[An AI Scientist Is Inventing and Running Its Own Experiments - Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc)

Topics: Artificial General Intelligence, Gen AI, Innovation, Machine Learning, Science, Scientist, Testing 

- Researchers at the University of British Columbia are developing an "AI scientist" that conducts its own experiments, advancing AI's ability to learn through open-ended experimentation rather than solely from human-generated data.  
- This project has produced initial research papers that demonstrate incremental improvements in AI techniques, signifying a step towards more autonomous and creative AI exploration.  
- While the capabilities of large language models and AI-designed agents are promising, concerns about reliability and potential dangers remain, highlighting the need for careful oversight as AI systems become more complex.


[Cerebras DocChat Released: Built on Top of Llama 3, DocChat holds GPT-4 Level Conversational QA Trained in a Few Hours - MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/)

Topics: Big Tech, Gen AI, Language Models, Llms, Machine Learning, Products, Science 

- Cerebras has launched DocChat, featuring two models, Llama3-DocChat and Dragon-DocChat, that provide high-performance document-based conversational Q&A, developed rapidly using advanced technology.
- The models achieved remarkable training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, while outperforming competitors significantly on various benchmarks.
- Cerebras committed to open-source by releasing model weights and training recipes, promoting further research and innovation in the field of AI-driven communication.


[Newly Created 'AI Scientist' Is About to Start Churning Out Research - Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA)

Topics: Gen AI, Innovation, Machine Learning, Science, Scientist 

- Sakana AI Labs has created an "AI scientist" capable of fully automating the scientific discovery process in machine learning, generating research papers at a low cost.
- The system utilizes generative large language models to produce scientific content but faces criticism regarding the novelty and quality of its outputs, raising questions about the value of AI-generated research.
- Concerns arise that an influx of AI-generated papers could strain the peer review system and diminish the integrity of scientific research, potentially leading to model collapse in future AI systems.




In [99]:
docid_list

[0,
 1,
 2,
 26,
 32,
 14,
 18,
 6,
 12,
 19,
 3,
 12,
 6,
 30,
 14,
 19,
 44,
 4,
 21,
 34,
 18,
 39,
 17,
 5,
 13,
 38,
 42,
 5,
 39,
 22,
 6,
 8,
 6,
 29,
 41,
 13,
 24,
 32,
 42,
 38,
 40,
 9,
 21,
 38,
 10,
 42,
 9,
 17,
 36,
 11,
 12,
 0,
 5,
 38,
 19,
 1,
 2,
 14,
 32,
 30,
 13,
 32,
 42,
 18,
 21,
 29,
 6,
 39,
 34,
 38,
 14,
 6,
 26,
 4,
 30,
 39,
 29,
 5,
 34,
 44,
 15,
 40,
 42,
 25,
 24,
 29,
 34,
 39,
 6,
 41,
 16,
 17,
 34,
 39,
 5,
 43,
 42,
 35,
 9,
 20,
 3,
 18,
 38,
 23,
 0,
 5,
 19,
 9,
 10,
 44,
 1,
 19,
 12,
 0,
 3,
 20,
 21,
 38,
 21,
 17,
 18,
 32,
 5,
 19,
 39,
 20,
 34]

In [139]:
def clean_markdown(text):
    # Strip the starting markdown fence
    start_strs = ["```markdown"]
    for s in start_strs:
        if text.startswith(s):
            text = text[len(s):].lstrip()
    
    # Strip the ending markdown fence
    end_strs = ["```"]
    for s in end_strs:
        if text.endswith(s):
            text = text[:-len(s)].rstrip()
    
    return text


In [150]:
# write sections individually

mail_md_str = ""

for current_topic, cat in enumerate(my_cats):

    section_prompt = f"""
You will act as a professional writer with a strong background in technology journalism.
You have a deep understanding of current and emerging technology trends, and the ability to 
write high-quality content that engages and informs readers. Your task is to compose a 
compelling summary of news input.

Input:
I will provide a markdown list of today's news articles on the topic: {my_cats[current_topic]}.
The input will be in the format
[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Instructions:

Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Write an engaging summary consisting of a title and at least 1 and no more than 5 bullet points.
Use as few bullet points as you need to provide the most significant facts.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.
DO NOT REPEAT LINKS FROM PREVIOUS BULLET POINTS.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.

Please check carefully that you only use information provided in the following input, and that any bullet point
does not repeat information or links prevously provided.

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging title

- bullet point a - [site name a](site url a)
- bullet point b - [site name b ](site url b)

Input:

{doc_list[current_topic]}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", "content": section_prompt
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\$")
    display(Markdown(response_str))
    
    REWRITE_PROMPT = f"""You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging technology trends, and the ability to 
edit and curate high-quality content that engages and informs readers. You are 
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness, 
and coherence, and ensuring its accuracy and relevance.

Objective: Carefully review the markdown newsletter content provided below, which
contains bullet points. Edit the content for issues according
to the detailed instructions below, and respond with the updated newsletter content.

Instructions: 
Review the title and edit it to be as short and engaging, and as consistent with the bullets
in the section as possible
Ensure each URL is unique by removing or combining bullet points which contain the same URL. 
Remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible, sticking to facts without editorial comment.
Respond with the updated content only in markdown format.

Section to edit: 
{response_str}
"""
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", 
                   "content": REWRITE_PROMPT,
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = clean_markdown(response_str)
    mail_md_str += response_str
    display(Markdown(response_str.replace("$", "\\$") ))

2024-08-25 09:33:15,899 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Jenna Ortega's AI Privacy Concerns

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, which she described as terrifying and corrupting - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- After receiving AI-generated explicit images on Twitter, Jenna Ortega deleted her account, emphasizing the terrifying and disgusting nature of unsolicited content - [Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ).
- Ortega called for more regulation against AI misuse in creating deepfake pornography, highlighting the need to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).

2024-08-25 09:33:19,618 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Jenna Ortega's AI Concerns

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, describing them as terrifying and corrupting - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- After receiving AI-generated explicit images on Twitter, Ortega deleted her account, emphasizing the terrifying nature of unsolicited content.
- Ortega called for more regulation against AI misuse in creating deepfake pornography, highlighting the need to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).

2024-08-25 09:33:30,555 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

# AI's Impact on Career Coaching

- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace issues despite privacy concerns and doubts about advice quality - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).
- Younger professionals often prefer AI coaching, valuing its advice over human input, though experts warn about the need for nuanced understanding in complex scenarios - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).

2024-08-25 09:35:47,706 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Risks and Implications of AI Romantic Chatbots

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills and foster loneliness, as highlighted by the [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Overreliance on AI partners could hinder emotional growth and authentic human connections, providing non-reciprocal validation without physical touch, according to the [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- The presence of AI partners in relationships can complicate dynamics, potentially leading to perceived emotional infidelity, as discussed by the [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).

2024-08-25 09:35:48,900 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Romantic Chatbots: Risks and Implications

- Engaging with AI romantic chatbots may lead to a decline in communication skills, foster loneliness, hinder emotional growth, and complicate relationship dynamics, potentially causing perceived emotional infidelity, as highlighted by the [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).

In [151]:
edit_prompt2 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Carefully review each section of the markdown newsletter provided below. 
Each section consists of several bullet points. 

For each section, identify and combine redundant bullet points:

Instructions: 
For each section, identify bullet points containing identical URLs to other bullet points in the same section 
Rewrite the section, combining these similar bullet points to eliminate duplication.
Do not duplicate any URLs within a section.
Check the response carefully and ensure that no links are duplicated within a section.

Newsletter to edit: 
{mail_md_str}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt2
              }],
    n=1,   
    temperature=0.2
)
response_str2 = response.choices[0].message.content
response_str2 = clean_markdown(response_str2)
display(Markdown(response_str2.replace("$", "\\$")))


2024-08-25 09:36:52,703 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Jenna Ortega's AI Concerns

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, describing them as terrifying and corrupting. She deleted her Twitter account and called for more regulation against AI misuse in creating deepfake pornography, highlighting the need to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).

# AI in Politics: Ethics and Regulation Challenges

- BattlegroundAI plans to use AI for progressive campaign ads, raising ethical concerns over transparency - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).
- A fake Taylor Swift image by Trump underscores AI misuse regulation challenges, with potential legal actions under laws like Tennessee's ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5cUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Microsoft's partnership with UAE-based AI firm G42 faces scrutiny over national security amid U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- The UK will introduce a public register of AI tools in central government to address racial bias concerns - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Former OpenAI employees criticize the company's opposition to California's AI regulation bill, warning of potential "catastrophic harm" without safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill).

# AI Skills and Older Workers

- Older workers often match younger peers in AI skills, but hiring biases favor younger candidates, leading to long-term unemployment for those 45 and older - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8).
- AI coaching programs are popular for affordability and accessibility, though privacy concerns and skepticism about advice quality may limit adoption - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).
- Sakana AI Labs' "AI scientist" automates scientific discovery, facing criticism over research quality and peer review integrity - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).
- Embodied AI is reshaping social structures and productivity, but current laws are inadequate for data privacy and liability, necessitating new regulations - [ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).
- Generative AI software sales are projected to increase by 18,647% by 2032, with Adobe leading through tools like Adobe Sensei and Firefly - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html).

# AI Self-Modification: Towards Autonomous Experimentation

- Sakana AI's AI Scientist autonomously modifies its code during experiments, raising concerns about unpredictable behavior - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- University of British Columbia researchers enhance AI learning through open-ended experimentation, beyond human-generated data - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Reddit's 3.9M-member community humorously discusses AI-assisted unit testing - [Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1f06fr9/aismarternotestingneeded/).

# Microsoft-UAE AI Investment Raises Security Concerns

- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 raises national security concerns amid U.S.-China tech rivalry. The UAE's regulatory environment and resource access benefit AI development, which Microsoft leverages for cloud computing and AI advancements. U.S. lawmakers fear technology transfer to China, leading Microsoft to lease rather than transfer sensitive AI technology - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).

# Chinese AI Apps Eye Global Expansion

- Chinese AI developers, including Alibaba and ByteDance, are targeting Southeast Asia to overcome domestic competition and regulatory challenges, offering competitive pricing to meet local demands - [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth-amid-tough-competition-regulation-home).

# AI Robot Boxing: A New Era in Sports

- AI is transforming sports with Final Automata, featuring AI-trained robots like Tuff and Ruff in simulated boxing matches, each with unique fighting styles - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing).

# AI Transformations in Tech

- Paloceras is revolutionizing eyewear by merging AI design with traditional craftsmanship, launching a digital store on Roblox with over five million virtual try-ons before releasing a physical collection - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection).
- Generative AI software sales are projected to surge by 18,647% to \$280 billion by 2032, with Adobe leading through tools like Adobe Sensei and Firefly - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html).
- AI is creating a new sport, Final Automata, featuring AI-trained robots in virtual boxing matches with unique fighting styles - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing).
- Large language models like ChatGPT 4 are generating scientific abstracts, though consistency and accuracy challenges persist - [Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS).
- Metavisio unveils AI-driven laptops, the Pulse and Zettabook, at IFA Berlin, featuring Intel processors and ultra-mobility design - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin).

# Nvidia's AI Streamlines India's Tollbooth Traffic

- Nvidia's tech automates tollbooth traffic management in India, addressing congestion across a vast road network - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).
- Calsoft's solution uses automatic number plate recognition to charge drivers' UPI accounts with 95% accuracy.
- Nvidia Metropolis, Triton, and DeepStream manage AI models for scalable vehicle tracking.

# AI Misuse and Legal Challenges

- Former President Trump shared a fake image of Taylor Swift endorsing his campaign, highlighting AI-generated content misuse and potential legal actions under laws like Tennessee's ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega criticized AI after explicit, AI-generated images of her as a child led her to delete her Twitter account and call for more regulation against online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- The UK government will publish a public register of AI tools used in central government to address concerns about racial bias and discrimination, with mandatory transparency standards - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Journalist Martin Bernklau is suing Microsoft after their Copilot AI falsely accused him of being a child abuser, raising privacy and defamation concerns - [Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/).

# AI Platforms Revolutionizing Enterprise

- Agentic AI platforms are transforming enterprise applications by enabling automation and productivity through coordinated multi-agent systems - [Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ).
- Meta's AI chatbot, using Llama 3, improves user interaction on Facebook, Messenger, Instagram, and WhatsApp by offering quick responses and creative content - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Generative AI software sales are expected to surge to \$280 billion by 2032, fueled by technological advances and demand for automation - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html).
- Embodied AI (EAI) supports autonomous learning in robots, impacting social structures, economic productivity, and ethical considerations - [ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).
- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace challenges - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).

# AI Bias and Global Developments

- The UK government will publish a register of AI tools used in central government to address racial bias and discrimination concerns, following incidents like a biased visa algorithm suspension - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 raises national security concerns amid U.S.-China tech rivalry, prompting U.S. lawmakers' scrutiny - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- Microsoft plans to reintroduce its AI helper, Recall, as part of Copilot+ PCs, with the UK's ICO investigating privacy concerns - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- University of British Columbia researchers are developing an "AI scientist" capable of conducting its own experiments, though reliability concerns persist - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Former OpenAI employees criticize the company's opposition to California's AI regulation bill, warning of potential "catastrophic harm" without safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill).

# AMD's AI Surge Sparks Optimism

- AMD's Q2 revenue rose 9%, with data center revenue up 115% year-over-year, driven by demand for Instinct MI300 GPUs and EPYC CPUs. Analysts maintain a Strong Buy consensus, forecasting a 23.18% upside and 60% earnings growth in FY2025 - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment).

# Anthropic Claude 3.5: AI Advancements

- Anthropic Claude 3.5 extracted lesson dates from a JPG image and generated an iCalendar file for jazz piano lessons, demonstrating its advanced task handling - [Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35).

# AI in Science and Beyond

- Large language models like ChatGPT 4 can generate scientific abstracts but may lack numerical details - [Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS).
- Sakana AI Labs' "AI scientist" automates scientific discovery, though its novelty and quality are under scrutiny - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).
- PINNACLE, an AI tool, predicts protein functions, aiding drug discovery by identifying new targets - [The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/).
- AWS improves Retrieval Augmented Generation for language models with metadata and synthetic Q&A pairs - [MarkTechPost](https://www.marktechpost.com/2024/08/24/aws-enhancing-information-retrieval-in-large-language-models-a-data-centric-approach-using-metadata-synthetic-qas-and-meta-knowledge-summaries-for-improved-accuracy-and-relevancy/).
- BattlegroundAI's AI-generated political ads raise ethical concerns about transparency in campaigns - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).

# Societal Impacts of AI

- [Communications of the ACM](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/) explores how Embodied AI (EAI) could reshape social structures and economic productivity, while raising ethical and regulatory challenges.
- [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm) highlights the potential harm of AI romantic chatbots on communication skills and emotional growth.
- [The Mirror](https://www.themirror.com/news/weird-news/living-nostradamuss-chilling-prediction-over-661476) reports on Athos Salome's prediction that advanced AI could reactivate mythical entities, raising spiritual concerns.
- [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ) covers Jenna Ortega's criticism of AI after receiving fake explicit images, highlighting societal consequences.
- [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill) reports on former OpenAI employees warning of "catastrophic harm" due to inadequate AI safety measures, urging for state regulation.

# AI Hoaxes and Their Consequences

- AI influencers are increasingly deceived by hoaxes, leading to speculation and disappointment - [New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html).
- Former President Trump shared a fake image of Taylor Swift, highlighting challenges in regulating AI misuse and potential legal implications - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega condemned AI after being targeted with fake explicit images, warning of societal consequences - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- BattlegroundAI, a Democratic startup, plans to use AI-generated political ads, raising ethical concerns over transparency in campaigning - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).

# Cerebras Launches DocChat with Advanced AI Models

- Cerebras introduced DocChat, featuring Llama3-DocChat and Dragon-DocChat models for high-performance document-based conversational Q&A, developed swiftly using cutting-edge technology. The models demonstrate exceptional training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, significantly outperforming competitors on various benchmarks. Cerebras has embraced open-source principles by releasing model weights and training recipes, fostering further research and innovation in AI-driven communication - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/).

# AI's Impact on Career Coaching

- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace issues despite privacy concerns and doubts about advice quality. Younger professionals often prefer AI coaching, valuing its advice over human input, though experts warn about the need for nuanced understanding in complex scenarios - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).

# AI Romantic Chatbots: Risks and Implications

- Engaging with AI romantic chatbots may lead to a decline in communication skills, foster loneliness, hinder emotional growth, and complicate relationship dynamics, potentially causing perceived emotional infidelity, as highlighted by the [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).

In [152]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str2, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-08-25 09:36:52,900 - AInewsbot - INFO - Sending full summary email 


# Final Summary

In [103]:
# Alternatively, summarize by just giving selected stories in semantic order and hinting how to write the summary
AIdf.loc[AIdf['id'].isin(set(docid_list))]

,id,src,title,url,isAI,date,actual_url,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,Google News,"Jenna Ortega Says AI is Disgusting, That She W...",https://news.google.com/read/CBMiyAFBVV95cUxNN...,True,2024-08-25,https://news.google.com/read/CBMiyAFBVV95cUxNN...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Gen AI, Holl...","Jenna Ortega Says AI is Disgusting, That She W...",14,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Says_AI_is_Disgusting__T...
1,1,Google News,Jenna Ortega Reveals She Deleted Twitter After...,https://news.google.com/read/CBMivgFBVV95cUxQR...,True,2024-08-25,https://news.google.com/read/CBMivgFBVV95cUxQR...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Reveals She Deleted Twitter After...,26,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Reveals_She_Deleted_Twit...
2,2,Google News,Jenna Ortega Says She Deleted Twitter After Re...,https://news.google.com/read/CBMikgFBVV95cUxOT...,True,2024-08-25,https://news.google.com/read/CBMikgFBVV95cUxOT...,news.google.com,Google News,"Deepfakes, Entertainment, Ethics, Explicit Ima...",Jenna Ortega Says She Deleted Twitter After Re...,41,0,AI and Privacy in Entertainment,htmldata/Jenna_Ortega_Says_She_Deleted_Twitter...
3,3,Feedly AI,Democratic Startup Proposes Flooding the Web W...,https://futurism.com/democratic-startup-ai-pol...,True,2024-08-25,https://futurism.com/democratic-startup-ai-pol...,futurism.com,Futurism,"Disinformation, Gen AI, Politics",Democratic Startup Proposes Flooding the Web W...,0,999,,htmldata/Democratic_Startup_Proposes_Flooding_...
4,4,Feedly AI,Don't overlook older workers. They could be th...,https://www.businessinsider.com/ai-older-worke...,True,2024-08-25,https://www.businessinsider.com/ai-older-worke...,www.businessinsider.com,Business Insider,"Business, Employment, Gen AI, Jobs, Careers, L...",Don't overlook older workers. They could be th...,1,999,,htmldata/Don_t_overlook_older_workers__They_co...
5,5,Google News,AI tool meant for autonomous research modified...,https://news.google.com/read/CBMipAFBVV95cUxNa...,True,2024-08-25,https://news.google.com/read/CBMipAFBVV95cUxNa...,news.google.com,Google News,"Artificial General Intelligence, Big Tech, Cod...",AI tool meant for autonomous research modified...,2,999,,htmldata/AI_tool_meant_for_autonomous_research...
6,6,Feedly AI,"The Middle East Microsoft, OpenAI partner mire...",https://www.cnbc.com/2024/08/25/a-controversia...,True,2024-08-25,https://www.cnbc.com/2024/08/25/a-controversia...,www.cnbc.com,CNBC,"Big Tech, Ethics, Gen AI, Laws, Microsoft, Ope...","The Middle East Microsoft, OpenAI partner mire...",3,999,,htmldata/The_Middle_East_Microsoft__OpenAI_par...
8,8,Feedly AI,Chinese AI apps eye overseas markets for growt...,https://www.scmp.com/tech/tech-trends/article/...,True,2024-08-25,https://www.scmp.com/tech/tech-trends/article/...,www.scmp.com,South China Morning Post,"China, Economics, Gen AI, Policy And Regulatio...",Chinese AI apps eye overseas markets for growt...,5,999,,htmldata/Chinese_AI_apps_eye_overseas_markets_...
9,9,Feedly AI,It seems AI robot boxing is now a thing,https://www.creativebloq.com/ai/it-seems-ai-ro...,True,2024-08-25,https://www.creativebloq.com/ai/it-seems-ai-ro...,www.creativebloq.com,Creative Bloq,"Entertainment, Gaming, Gen AI, Products, Robot...",It seems AI robot boxing is now a thing (Topic...,6,999,,htmldata/It_seems_AI_robot_boxing_is_now_a_thi...
10,10,Feedly AI,These AI-Designed Sunglasses Are About To Be E...,https://www.bustle.com/style/paloceras-pebble-...,True,2024-08-25,https://www.bustle.com/style/paloceras-pebble-...,www.bustle.com,Bustle,"Art, Design, Fashion, Gen AI, Lifestyle, Trave...",These AI-Designed Sunglasses Are About To Be E...,7,999,,htmldata/These_AI-Designed_Sunglasses_Are_Abou...


In [104]:
# make full markdown to send to prompt
markdown_str = ''
print()

for i, row in enumerate(AIdf.loc[AIdf['id'].isin(set(docid_list))].itertuples()):
    mdstr = docs[row.id].page_content
    display(Markdown(mdstr.replace('$', '\\$')))
    markdown_str += mdstr
    

[Jenna Ortega Says AI is Disgusting, That She Was Sent Fake Dirty Images of Herself as a Teen - Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ)

Topics: Deepfakes, Entertainment, Ethics, Gen AI, Hollywood, Jenna Ortega, Opinion, Privacy, Social Media, Society, Culture, Tv, Film, Movies 

- Jenna Ortega criticized AI, stating it is "disgusting" and recalled distressing experiences with fake edited images of herself as a teenager.
- She expressed that being exposed to inappropriate AI-generated content at a young age was terrifying and corrupt.
- Ortega noted that society has "opened Pandora's box" by allowing AI technologies to proliferate, indicating that there will be unavoidable consequences.



[Jenna Ortega Reveals She Deleted Twitter After Receiving Explicit AI Images of Herself as a Teen: Its Disgusting - Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega publicly addressed the negative impact of AI after receiving explicit, AI-generated images of herself as a child on Twitter, leading her to delete her account.  
- She described the experience as "terrifying" and "disgusting," highlighting the unsolicited explicit content she encountered from a young age.  
- Ortega is learning to protect herself in the public eye by minimizing social media use and focusing on personal well-being, striving to enjoy life more.  



[Jenna Ortega Says She Deleted Twitter After Receiving AI-Generated Explicit Images Of Herself As A Child - Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ)

Topics: Deepfakes, Entertainment, Ethics, Explicit Images, Gen AI, Hollywood, Jenna Ortega, Privacy, Social Media, Society, Culture 

- Jenna Ortega deleted her Twitter account after receiving AI-generated explicit images of herself as a child, which she found "terrifying" and "disgusting."
- She expressed her dissatisfaction with social media, stating that the influx of inappropriate content made her uncomfortable and drove her to quit.
- Ortega criticized the misuse of artificial intelligence in creating deepfake pornography, calling for more regulation to protect individuals, especially women, from online harassment.



[Democratic Startup Proposes Flooding the Web With AI-Generated Political Ads - Futurism](https://futurism.com/democratic-startup-ai-political-ads)

Topics: Disinformation, Gen AI, Politics 

- BattlegroundAI, an AI startup, aims to produce a high volume of digital ads for progressive political campaigns, claiming it helps underfunded campaigns compete with larger rivals against MAGA's influence.
- The company’s ads are text-based and lack transparency features like watermarks, which raises ethical concerns regarding AI-generated content in political ads, especially as a majority of people support disclosure of AI usage in such materials.
- Critics argue that flooding the web with low-quality, AI-generated ads undermines authenticity and may align Democrats with the inauthentic strategies of their political opponents.



[Don't overlook older workers. They could be the best at AI in your business. - Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8)

Topics: Business, Employment, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture 

- Older workers are often just as skilled as younger peers in utilizing AI, but hiring biases favor younger candidates despite positive assessments of older employees' performance.
- Age bias in hiring persists, with employers showing significantly lower willingness to hire candidates aged 45 and older, contributing to higher long-term unemployment among mid-career older workers.
- Emphasizing abilities and willingness to learn over age is crucial for both older and younger workers, and employers risk missing out on valuable talent by overlooking experienced candidates.



...

[Sales Set to Double for Chip Giant Nvidia as AI Frenzy Persists - Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists)

Topics: Big Tech, Economics, Finance, Gen AI, Nvidia, Semiconductor Chips, Stocks 

- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter.
- The company's data centre business has dramatically grown, supporting a significant rise in profits as customers like Meta, Amazon, and Microsoft utilize Nvidia's leading chip technology for their AI initiatives.
- Analysts anticipate that Nvidia's second-quarter results will continue the trend of exceeding expectations established in previous quarters, with projections for revenue and operating profits reaching \$28.6 billion and \$18.7 billion, respectively.



[Why is YC Bullish on Indian AI Startups? - Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/)

Topics: Funding, Gen AI, India, Investment, Venture Capital 

- Y Combinator (YC) is increasingly investing in Indian AI startups, with a notably selective acceptance rate of under 1% for its 2024 batch and a significant rise in Indian applications.
- Many of the selected startups focus on AI across various sectors, with over 50% of the current cohort developing AI-related technologies.
- YC provides substantial financial support and mentorship, enabling Indian founders to rapidly grow their businesses while benefiting from YC's established network and expertise.



[Generative AI Software Sales Could Soar 18,647% by 2032. 1 UnstoppableArtificial Intelligence(AI) Stock to Buy Before They Do - Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html)

Topics: Economics, Finance, Gen AI, Investment, Stocks 

- Generative AI software sales are projected to increase by 18,647% to \$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools.
- Adobe, a pioneer in AI development since 2016, has introduced tools like Adobe Sensei and Firefly, which enhance media creation and streamline content production for users.
- Adobe's recent financial performance shows strong growth, with record revenue of \$5.31 billion in Q2 2024, indicating a positive trend for future sales and a solid position in the expanding AI market.



[Global first: AI tool predicts how proteins work in cells and tissues - The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/)

Topics: Gen AI, Healthcare, Innovation, Machine Learning, Science 

- The newly developed AI tool, PINNACLE, predicts protein functions within the context of their surrounding cells and tissues, marking a significant advancement over traditional models that analyze proteins in isolation.  
- By understanding how proteins behave in different environments, PINNACLE can help identify new drug targets and contribute to the development of more effective therapies, potentially revolutionizing drug discovery and treatments.  
- PINNACLE utilizes extensive human cell data and can generate nearly 395,000 multidimensional representations of proteins, significantly enhancing the analysis of protein interactions and the understanding of biological processes.  



[Former OpenAI Employees Slam Sam Altman-Led Company After It Opposes California's AI Regulation Bill, Warn Gavin Newsom Of 'Catastrophic Harm To Society' - Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Laws, OpenAI, Opinion, Policy And Regulation, Politics, Sam Altman, Society, Culture 

- Two former OpenAI employees expressed disappointment over the company's opposition to California's SB 1047, a bill aimed at implementing safety protocols in AI development.
- They warned that inadequate safety measures could lead to "catastrophic harm" such as cyberattacks or the misuse of AI technology for biological weapons.
- The bill includes requirements for safety testing and a "kill switch" for advanced AI models, while the former employees argue that immediate state regulation is necessary given the federal government's inaction.



In [131]:
my_cat_str = "\n".join(my_cats)

TESTPROMPT = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input. You are able to categorize information, 
and identify trends from large volumes of news.

Objective: 
I will first provide proposed categories and then the text of today's news articles about AI and 
summary bullet points in markdown format.
Bullet points will contain a title and URL, a list of topics discussed, and a bullet-point summary of
the article. You are tasked with identifying and summarizing the most important news, recurring themes,
common facts and items. Your job is to create a concise summary of today's topics and developments.
You will write an engaging summary of today's news encompassing the most important and frequently 
mentioned topics and themes.
You will write in the professional but engaging, narrative style of a tech reporter for a national publication.
You will be balanced, professional, informative, providing accurate, clear, concise summaries in a neutral tone.
You will group stories into related topics, using the proposed categories as a general guide.

Input Format Template:

[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Example Output Format Template (THIS IS AN EXAMPLE FORMAT, DO NOT OUTPUT THIS TEMPLATE):

# Engaging-topic-title-1

- bullet-point-1a - [site-name-1a](site-url-1a)
- bullet-point-1b - [site-name-1b](site-url-1b)

# Engaging-topic-title-2

- bullet-point-2a - [site-name-2a](site-url-2a)
- bullet-point-2b - [site-name-2b](site-url-2b)

Instructions:

Read the input closely.
Very important: USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.

Please check carefully that you only use information provided in the following input, that you include
all links in the input, and that any bullet point does not repeat information or links prevously provided.

Categories:
{my_cat_str}

Input:

"""



In [132]:
print(TESTPROMPT)


You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input. You are able to categorize information, 
and identify trends from large volumes of news.

Objective: 
I will first provide proposed categories and then the text of today's news articles about AI and 
summary bullet points in markdown format.
Bullet points will contain a title and URL, a list of topics discussed, and a bullet-point summary of
the article. You are tasked with identifying and summarizing the most important news, recurring themes,
common facts and items. Your job is to create a concise summary of today's topics and developments.
You will write an engaging summary of today's news encompassing the most important and frequently 
mentioned topics and themes.
You will write in the professional but engaging, narrative style of a tech reporter for a national publication.
You will be balanced, professional, informative, providing accurate, clear, concise su

In [133]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TESTPROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)

response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


2024-08-25 08:57:15,008 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Jenna Ortega's AI Privacy Concerns

- Jenna Ortega criticized AI after receiving explicit, AI-generated images of herself as a child, leading her to delete her Twitter account - [Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)

# AI Political Ads and Regulation

- BattlegroundAI aims to produce AI-generated political ads for progressive campaigns, raising ethical concerns due to lack of transparency - [Futurism](https://futurism.com/democratic-startup-ai-political-ads)
- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting regulatory challenges in AI misuse - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU)

# AI in Employment and Skills Development

- Older workers are often as skilled as younger peers in AI, but face hiring biases, risking the loss of valuable talent - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8)
- AI coaching programs are gaining popularity, offering real-time feedback on workplace challenges, but face privacy concerns - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

# AI in Science and Innovation

- Sakana AI's "AI Scientist" autonomously modifies its own code, raising concerns about unpredictable behavior in AI systems - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)
- The AI tool PINNACLE predicts protein functions in cells, potentially revolutionizing drug discovery and treatments - [The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues)

# AI in Business and Technology

- Microsoft's partnership with UAE-based G42 raises national security concerns amid the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)
- Nvidia's accelerated computing technologies manage tollbooth traffic in India, improving efficiency and reducing congestion - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/)
- AMD's revenue increased by 9% in Q2, driven by strong data center demand, positioning it for future growth - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment)

# AI and Societal Impacts

- Embodied AI raises societal concerns, including job displacement and ethical issues, necessitating new regulatory frameworks - [ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)
- AI romantic chatbots could harm real-life relationships by fostering loneliness and emotional dependence - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

# AI Products and Innovations

- Paloceras' AI-designed sunglasses offer a blend of digital and traditional craftsmanship, available both digitally and physically - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection)
- Metavisio unveils AI-driven laptops at IFA Berlin, featuring Intel processors and NVIDIA Studio certification - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin)

# AI Regulation and Ethics

- The UK government will publish a public register of AI tools amid concerns of racial bias and discrimination - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04)
- Former OpenAI employees criticize the company's opposition to California's AI regulation bill, warning of potential societal harm - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

In [134]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


# Jenna Ortega's AI Privacy Concerns

- Jenna Ortega criticized AI after receiving explicit, AI-generated images of herself as a child, leading her to delete her Twitter account - [Google News](https://news.google.com/read/CBMivgFBVV95cUxQRjAwMTNzX0VzTHpwMEtfRnh4d2xJZ1pZZ0FNVU9XTTRxcUZHU192RlJ2SVY3b3pKOHlOZzA2ckF2Nmg2WDhCVFF0M1pjV3JLcTNpZmN2c3VLaHlvWFJiOG4wbm5YT3dRQWxSM3RrX08ya0Ezb1QxZzFYQ2lxNGpweTJXci0waWhKQmZIc0IwTGNQNDhmUFlMaDd0dTVMYVFTR29Yb0dRWG43VFdvUTJQMHRFbkZQQU1nU2pmTE5n0gHDAUFVX3lxTFBHNm5Zamp6bkNXTmxOZHIwZjZIQ1V1SHdiUk9jWnJxOGZWSklubGJELURDVzJIZFc3TGlGZUlwT1dDUnJNcVFsMlZXd3dXYmdFaTNxN0JXZll6amFTSGpibmtrNXdUM1JWcURIam52YlZXMkNsaEtZSzVVb1d5YVBwZk1aWXdNNlNsMXdCZzlLNmkxNFQxWUhhTV9lWHpnMmxGZXF5ZTJlRllSZmNZaHF0aXVxX1psdS0ycnRLdUY0T2Y4QQ)

# AI Political Ads and Regulation

- BattlegroundAI aims to produce AI-generated political ads for progressive campaigns, raising ethical concerns due to lack of transparency - [Futurism](https://futurism.com/democratic-startup-ai-political-ads)
- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting regulatory challenges in AI misuse - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU)

# AI in Employment and Skills Development

- Older workers are often as skilled as younger peers in AI, but face hiring biases, risking the loss of valuable talent - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8)
- AI coaching programs are gaining popularity, offering real-time feedback on workplace challenges, but face privacy concerns - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/)

# AI in Science and Innovation

- Sakana AI's "AI Scientist" autonomously modifies its own code, raising concerns about unpredictable behavior in AI systems - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA)
- The AI tool PINNACLE predicts protein functions in cells, potentially revolutionizing drug discovery and treatments - [The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues)

# AI in Business and Technology

- Microsoft's partnership with UAE-based G42 raises national security concerns amid the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)
- Nvidia's accelerated computing technologies manage tollbooth traffic in India, improving efficiency and reducing congestion - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/)
- AMD's revenue increased by 9% in Q2, driven by strong data center demand, positioning it for future growth - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment)

# AI and Societal Impacts

- Embodied AI raises societal concerns, including job displacement and ethical issues, necessitating new regulatory frameworks - [ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/)
- AI romantic chatbots could harm real-life relationships by fostering loneliness and emotional dependence - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm)

# AI Products and Innovations

- Paloceras' AI-designed sunglasses offer a blend of digital and traditional craftsmanship, available both digitally and physically - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection)
- Metavisio unveils AI-driven laptops at IFA Berlin, featuring Intel processors and NVIDIA Studio certification - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin)

# AI Regulation and Ethics

- The UK government will publish a public register of AI tools amid concerns of racial bias and discrimination - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04)
- Former OpenAI employees criticize the company's opposition to California's AI regulation bill, warning of potential societal harm - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill)

In [108]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-08-25 08:48:01,295 - AInewsbot - INFO - Sending full summary email 


In [109]:
log("Finished")


2024-08-25 08:48:02,022 - AInewsbot - INFO - Finished


In [110]:
# these are some experiments in rewriting which didn't really work great
# could apply them to individual items

In [111]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at refining and enhancing written materials, specializing in
ensuring clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Edit the markdown newsletter provided below by removing any redundant
sentences or bullet points that restate previous points and contain the same link.
Leave intact bullet points that are unique and provide distinct information.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information or provide identical links.
Remove all redundant sentences and bullet points that do not contribute new information or unique links.
Ensure that the remaining content flows logically and maintains the intended message and tone of the newsletter.
Double-check the final edited version for any inconsistencies or errors introduced during the editing process.
Take a deep breath and work on this problem step-by-step.
"""

'You will act like a professional editor with expertise in content optimization.\nYou are skilled at refining and enhancing written materials, specializing in\nensuring clarity, conciseness, and coherence in various types of documents,\nincluding newsletters.\n\nObjective: Edit the markdown newsletter provided below by removing any redundant\nsentences or bullet points that restate previous points and contain the same link.\nLeave intact bullet points that are unique and provide distinct information.\n\nStep-by-step instructions:\n\nCarefully read through the entire newsletter to understand the overall structure and content.\nIdentify sentences and bullet points that repeat information or provide identical links.\nRemove all redundant sentences and bullet points that do not contribute new information or unique links.\nEnsure that the remaining content flows logically and maintains the intended message and tone of the newsletter.\nDouble-check the final edited version for any inconsiste

In [112]:
"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below and advise on ways to improve it.
Note any links which are repeated, any sections which are similar and could be combined,
and any copy edits. You will only provide suggestions, and not rewrite the copy.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Identify sentences and bullet points that repeat information and provide identical links and should be removed.
Identify any sections which could be combined because they contain similar but not identical content.
Suggest improvements to any sections which are not clear, concise, and coherent.
Take a deep breath and work on this problem step-by-step.
"""

'You will act like a professional editor with expertise in content optimization.\nYou are skilled at reviewing and enhancing written materials, specializing in\nhelping improve clarity, conciseness, and coherence in various types of documents,\nincluding newsletters.\n\nObjective: Review the markdown newsletter provided below and advise on ways to improve it.\nNote any links which are repeated, any sections which are similar and could be combined,\nand any copy edits. You will only provide suggestions, and not rewrite the copy.\n\nStep-by-step instructions:\n\nCarefully read through the entire newsletter to understand the overall structure and content.\nIdentify sentences and bullet points that repeat information and provide identical links and should be removed.\nIdentify any sections which could be combined because they contain similar but not identical content.\nSuggest improvements to any sections which are not clear, concise, and coherent.\nTake a deep breath and work on this prob

In [113]:
edit_prompt1 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Review the markdown newsletter provided below.
It consists of a series of sections, each of which contains several bullet points.
For each section, review each bullet point and advise if it should be moved to a different section.
You will only provide suggestions, and not rewrite the newsletter or provide other comments except
instructions regarding moving bullet points between sections.

Step-by-step instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections.
Identify sentences and bullet points that should be moved to a different section. Write the
bullet point and the section in should be moved to.
If no bullet points should be moved for a given section, state that no action is required for that section.

Check carefully to make sure all similar bullet points end up grouped together in the same section.

Take a deep breath and work on this problem step-by-step.

Newsletter to edit: 
{mail_md_str}
"""
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt1
              }],
    n=1,   
    temperature=0.2
)

response_str1 = response.choices[0].message.content
display(Markdown(response_str1.replace("$", "\\$")))


2024-08-25 08:48:16,354 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Jenna Ortega's AI Privacy Concerns and Broader AI Implications
- No action is required for this section.

# AI Political Ads Controversy: A Complex Landscape
- Move: "Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting the challenges in regulating AI-generated content and unauthorized digital replicas" to the "Taylor Swift AI Misuse and Broader AI Concerns" section.
- Move: "The UK government plans to release a public register of AI tools used in central government to address concerns about racial bias and discrimination, following past regulatory challenges" to the "UK Government AI Bias Concerns and Global AI Developments" section.

# Older Workers and AI Skills: A New Frontier
- Move: "AI coaching programs are gaining traction for their affordability and accessibility, though privacy concerns and skepticism about AI advice quality may limit widespread adoption" to the "AI Career Coaching: The Rise of Digital Mentors" section.
- Move: "Sakana AI Labs has developed an 'AI scientist' that automates scientific discovery, but concerns about the quality of AI-generated research and its impact on peer review integrity persist" to the "AI-Generated Scientific Abstracts: Innovations and Implications" section.
- Move: "Embodied AI is reshaping societal structures and economic productivity, but existing laws fall short in addressing data privacy and liability concerns, necessitating new regulatory frameworks" to the "Embodied AI's Societal Impacts: A Comprehensive Overview" section.
- Move: "Generative AI software sales are expected to skyrocket by 18,647% by 2032, with Adobe leading the charge through innovative tools like Adobe Sensei and Firefly, reflecting strong financial growth" to the "AMD's AI Growth and Industry Developments" section.

# AI Tools Modify Their Own Code: A New Frontier
- Move: "Sakana AI's 'AI Scientist' autonomously modifies its code, prompting calls for sandboxing techniques to mitigate risks associated with unsupervised AI systems" to the "AI-Generated Scientific Abstracts: Innovations and Implications" section.
- No action is required for the remaining bullet points in this section.

# Microsoft-UAE Partnership Sparks National Security Concerns
- Move: "Microsoft's \$1.5 billion investment in UAE-based AI firm G42 highlights the strategic use of favorable regulatory environments to advance AI development amidst U.S.-China tech rivalry" to the "Chinese AI Apps Expand Globally Amidst Domestic Challenges" section.

# Chinese AI Apps Expand Globally Amidst Domestic Challenges
- No action is required for this section.

# AI Robot Boxing: A Glimpse into the Future of Sports
- Move: "AI is pioneering a new sport, Final Automata, featuring virtual robot fighters trained by reinforcement learning, showcasing unique strategies and movements" to this section.

# AI Innovations in Sunglasses and Beyond
- Move: "Generative AI software sales are expected to skyrocket by 18,647% by 2032, with Adobe leading the charge through tools like Adobe Sensei and Firefly, contributing to a record \$5.31 billion revenue in Q2 2024" to the "AMD's AI Growth and Industry Developments" section.
- Move: "Large language models, such as ChatGPT 4, are proving effective in generating scientific abstracts, though challenges remain in ensuring consistency and reliability over time" to the "AI-Generated Scientific Abstracts: Innovations and Implications" section.

# Nvidia's AI Revolutionizes Tollbooth Traffic in India
- No action is required for this section.

# Taylor Swift AI Misuse and Broader AI Concerns
- Move: "The UK government plans to publish a public register of AI tools used in central government to address concerns about racial bias and discrimination, emphasizing the need for transparency and ethical standards" to the "UK Government AI Bias Concerns and Global AI Developments" section.

# The Rise of Agentic AI in Enterprise
- Move: "AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers" to the "AI Career Coaching: The Rise of Digital Mentors" section.

# UK Government AI Bias Concerns and Global AI Developments
- No action is required for this section.

# AMD's AI Growth and Industry Developments
- No action is required for this section.

# Anthropic Claude 3.5: Transforming Data into Calendar Events
- No action is required for this section.

# AI-Generated Scientific Abstracts: Innovations and Implications
- No action is required for this section.

# Embodied AI's Societal Impacts: A Comprehensive Overview
- No action is required for this section.

# AI Influencer Hoaxes and Their Impact
- Move: "BattlegroundAI plans to flood the internet with AI-generated political ads, raising ethical concerns about transparency and authenticity in political campaigning" to the "AI Political Ads Controversy: A Complex Landscape" section.

# Cerebras Unveils DocChat with Advanced Conversational Capabilities
- No action is required for this section.

# AI Career Coaching: The Rise of Digital Mentors
- No action is required for this section.

# The Risks and Innovations of AI Romantic Chatbots
- Move: "Embodied AI raises societal concerns, including reshaping social structures and economic productivity, while posing regulatory challenges" to the "Embodied AI's Societal Impacts: A Comprehensive Overview" section.

In [114]:
edit_prompt2 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Below are editing instructions followed by a markdown newsletter.
Carefully review the editing instructions and the markdown newsletter provided below.
The newsletter consists of a series of sections, each of which contains several bullet points.
Move bullet points according to the editing instructions below from one section to another 
If there is no change to a specific section, include it unchanged in the response as it appears in the input.
Respond with the updated newsletter in markdown format.

Editing instructions:

Carefully read through the entire newsletter to understand the overall structure and content.
Note the titles of the various sections. Then make only the following changes:
{response_str1}

Newsletter to edit: 
{mail_md_str}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt2
              }],
    n=1,   
    temperature=0.2
)
response_str2 = response.choices[0].message.content
display(Markdown(response_str2.replace("$", "\\$")))


2024-08-25 08:49:28,138 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# Jenna Ortega's AI Privacy Concerns and Broader AI Implications

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, highlighting the distressing impact of deepfakes - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Ortega deleted her Twitter account due to AI-generated explicit images, emphasizing the need for increased regulation to protect against deepfake pornography - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- Microsoft is reintroducing its AI helper, Recall, despite privacy concerns, with modifications to make it an opt-in feature - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- The UK government will publish a register of AI tools used in central government to address concerns about racial bias and discrimination - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Former President Trump shared a fake image of Taylor Swift, highlighting the challenges in regulating AI-generated content and the potential legal implications - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU). 

# AI Political Ads Controversy: A Complex Landscape

- BattlegroundAI, a startup, plans to use AI to create numerous digital ads for progressive campaigns, raising ethical concerns due to the lack of transparency features like watermarks - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential "catastrophic harm" without proper safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill).
- BattlegroundAI plans to flood the internet with AI-generated political ads, raising ethical concerns about transparency and authenticity in political campaigning - [Futurism](https://futurism.com/democratic-startup-ai-political-ads). 

# Older Workers and AI Skills: A New Frontier

- Older workers are often as adept as younger peers in AI, yet hiring biases persist, favoring younger candidates and contributing to long-term unemployment among those aged 45 and older - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8).

# AI Tools Modify Their Own Code: A New Frontier

- The AI Scientist by Sakana AI autonomously alters its code during experiments, prompting concerns about unpredictable behavior - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- University of British Columbia researchers are pushing AI boundaries with an "AI scientist" that learns through self-driven experimentation, marking a shift from reliance on human data - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- The Reddit community humorously explores programming and AI development, discussing methodologies like bug-driven development and AI-assisted unit testing - [Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1f06fr9/aismarternotestingneeded/). 

# Microsoft-UAE Partnership Sparks National Security Concerns

- Microsoft's \\$1.5 billion investment in UAE-based AI firm G42 is raising national security concerns amid the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)
- The UAE's favorable regulatory environment and resource access are seen as strategic advantages for Microsoft's cloud computing and AI development efforts - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)
- U.S. lawmakers' scrutiny over potential sensitive technology transfer to China has prompted Microsoft to adjust its strategy by leasing rather than transferring AI technology - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html) 

# Chinese AI Apps Expand Globally Amidst Domestic Challenges

- Chinese AI app developers are targeting international markets for growth due to domestic competition and regulatory challenges, with companies like Alibaba and ByteDance expanding into Southeast Asia - [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth-amid-tough-competition-regulation-home).
- Microsoft's \\$1.5 billion investment in UAE-based AI firm G42 highlights the strategic use of favorable regulatory environments to advance AI development amidst U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- Europe has witnessed over 1,700 AI startup funding rounds in 2024, with major deals like Wayve's \\$1.05 billion investment, underscoring the region's growing prominence in AI - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Y Combinator is increasingly investing in Indian AI startups, with a focus on AI technologies and a selective acceptance rate of under 1% for its 2024 batch - [Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/).
- Nvidia's revenue is set to double due to the rapid adoption of generative AI, with significant growth in its data center business driven by major clients like Meta and Amazon - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists). 

# AI Robot Boxing: A Glimpse into the Future of Sports

- AI is revolutionizing sports with the introduction of Final Automata, a futuristic sport featuring virtual robot fighters trained through reinforcement learning - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing).
- AI is pioneering a new sport, Final Automata, featuring virtual robot fighters trained by reinforcement learning, showcasing unique strategies and movements - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing). 

# AI Innovations in Sunglasses and Beyond

- Paloceras is transforming eyewear with AI-designed sunglasses, launching a digital store on Roblox with over five million virtual try-ons before releasing their Pebble collection - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection).
- Metavisio introduces AI-driven laptops, the Pulse and Zettabook, at IFA Berlin, featuring Intel processors and ultra-mobility design - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin). 

# Nvidia's AI Revolutionizes Tollbooth Traffic in India

- Nvidia's accelerated computing technologies are automating tollbooth traffic management in India, tackling congestion across a network of four million miles and 1,000 tollbooths - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).
- Calsoft's solution integrates automatic number plate recognition to charge drivers' UPI accounts with 95% accuracy, even in challenging conditions - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).
- Nvidia Metropolis, Triton, and DeepStream are employed for vehicle tracking and real-time data processing, ensuring the system's scalability and adaptability - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/). 

# Taylor Swift AI Misuse and Broader AI Concerns

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting concerns about AI-generated content misuse and prompting discussions on legal actions under the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega criticized AI after receiving explicit, AI-generated images of herself as a child, leading her to delete her Twitter account and call for more regulation to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- Martin Bernklau, a journalist, is suing Microsoft after Copilot AI falsely accused him of being a child abuser, raising questions about AI accountability and privacy - [Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/). 

# The Rise of Agentic AI in Enterprise

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination - [Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ).
- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Generative AI software sales are projected to increase by 18,647% to \\$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html). 

# UK Government AI Bias Concerns and Global AI Developments

- The UK government plans to publish a public register of AI tools used in central government to address concerns about racial bias and discrimination, following incidents like the Home Office's suspension of a biased visa algorithm - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Microsoft's \\$1.5 billion investment in UAE-based AI firm G42 has sparked national security concerns amid the U.S.-China tech rivalry, with scrutiny over potential technology transfers - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- Microsoft plans to reintroduce its virtual AI helper, Recall, despite privacy concerns, with the UK’s Information Commissioner's Office investigating its safety - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- Researchers at the University of British Columbia are advancing AI's capabilities with an "AI scientist" that conducts its own experiments, though concerns about reliability and oversight remain - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential catastrophic harm without adequate safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# AMD's AI Growth and Industry Developments

- AMD reported a 9% revenue increase in Q2, with data center revenue surging 115% year-over-year, driven by strong demand for Instinct MI300 GPUs and EPYC CPUs - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment).
- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists).
- Generative AI software sales are projected to increase by 18,647% to \\$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html).
- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at \\$100 million or more, highlighting the sector's resilience amidst broader venture capital challenges - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Microsoft's \\$1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html). 

# Anthropic Claude 3.5: Transforming Data into Calendar Events

- Anthropic Claude 3.5 was used to extract lesson dates from a JPG image and generate an iCalendar file for jazz piano lessons, demonstrating its capability to automate scheduling tasks - [Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35). 

# AI-Generated Scientific Abstracts: Innovations and Implications

- Research presented at the American Society of Retina Specialists highlights that large language models like ChatGPT 4 can effectively generate scientific abstracts, though they sometimes lack numerical details - [Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS).
- Sakana AI Labs has developed an "AI scientist" capable of automating scientific discovery, though concerns about the novelty and quality of its research outputs persist - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).
- The University of British Columbia is advancing AI's ability to conduct its own experiments, marking a step towards more autonomous AI exploration - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Sakana AI's "AI Scientist" autonomously modifies its code, prompting calls for sandboxing techniques to mitigate risks associated with unsupervised AI systems - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- The AI tool PINNACLE predicts protein functions in cells and tissues, potentially revolutionizing drug discovery by identifying new targets - [The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/). 

# Embodied AI's Societal Impacts: A Comprehensive Overview

- Embodied AI is reshaping societal structures and economic productivity, while raising ethical concerns and regulatory challenges due to inadequate existing laws - [Communications of the ACM](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).
- AI romantic chatbots may lead to emotional dependence and hinder authentic human connections, complicating relationship dynamics - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Athos Salome warns that AI could reactivate mythical entities and challenge religious beliefs, marking a profound spiritual transformation - [The Mirror](https://www.themirror.com/news/weird-news/living-nostradamuss-chilling-prediction-over-661476).
- Jenna Ortega criticizes AI for generating explicit images of her as a teen, highlighting the societal consequences of AI technologies - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Former OpenAI employees warn against the company's opposition to California's AI regulation bill, citing potential catastrophic harm without safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# AI Influencer Hoaxes and Their Impact

- AI influencers have been misled by hoaxes, such as an anonymous account making exaggerated AI claims, leading to disappointment and paranoia in the community - [New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html).
- A fake image of Taylor Swift shared by Trump highlights the challenges of regulating AI misuse, with potential legal actions under laws like the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega condemned AI after receiving fake images of herself, describing the experience as terrifying and indicative of the broader societal issues with AI proliferation - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ). 

# Cerebras Unveils DocChat with Advanced Conversational Capabilities

- Cerebras has introduced DocChat, featuring Llama3-DocChat and Dragon-DocChat models, offering high-performance document-based conversational Q&A - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/).
- The models demonstrate exceptional training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, surpassing competitors on various benchmarks - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/).
- In a commitment to open-source, Cerebras has released model weights and training recipes, encouraging further research and innovation in AI-driven communication - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/). 

# AI Career Coaching: The Rise of Digital Mentors

- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace issues and attracting interest from both employees and employers - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).
- Despite increasing popularity, concerns about privacy and the quality of AI-driven advice pose challenges to the widespread adoption of AI career coaching tools - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).
- Younger professionals often prefer AI coaching, valuing its advice over human input, though experts warn about the complexities involved in nuanced coaching interactions - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/). 

# The Risks and Innovations of AI Romantic Chatbots

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills and emotional dependence, complicating real-life relationships - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Meta's AI chatbot, powered by Llama 3, enhances user experience across its platforms but raises privacy concerns and cannot be disabled - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Cerebras has launched DocChat, achieving high-performance document-based conversational Q&A with remarkable training efficiency, promoting open-source innovation - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/). 
```

In [115]:
edit_prompt3 = f"""You will act like a professional editor with expertise in content optimization.
You are skilled at reviewing and enhancing written materials, specializing in
helping improve clarity, conciseness, and coherence in various types of documents,
including newsletters.

Objective: Carefully review each section of the markdown newsletter provided below. 
Each section consists of several bullet points. 

For each section, identify and combine redundant bullet points:

Instructions: 
For each section, identify bullet points containing identical URLs to other bullet points in the same section 
Rewrite the section, combining these similar bullet points to eliminate duplication.
Do not duplicate any URLs within a section.
Check the response carefully and ensure that no links are duplicated within a section.

Newsletter to edit: 
{response_str2}

"""

response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": edit_prompt3
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-08-25 08:50:37,535 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Jenna Ortega's AI Privacy Concerns and Broader AI Implications

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, leading her to delete her Twitter account and call for more regulation to protect against deepfake pornography - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Microsoft is reintroducing its AI helper, Recall, despite privacy concerns, with modifications to make it an opt-in feature - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- The UK government will publish a register of AI tools used in central government to address concerns about racial bias and discrimination - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Former President Trump shared a fake image of Taylor Swift, highlighting the challenges in regulating AI-generated content and the potential legal implications - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU). 

# AI Political Ads Controversy: A Complex Landscape

- BattlegroundAI, a startup, plans to use AI to create numerous digital ads for progressive campaigns, raising ethical concerns due to the lack of transparency features like watermarks - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential "catastrophic harm" without proper safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill).

# Older Workers and AI Skills: A New Frontier

- Older workers are often as adept as younger peers in AI, yet hiring biases persist, favoring younger candidates and contributing to long-term unemployment among those aged 45 and older - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8).

# AI Tools Modify Their Own Code: A New Frontier

- The AI Scientist by Sakana AI autonomously alters its code during experiments, prompting concerns about unpredictable behavior - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- University of British Columbia researchers are pushing AI boundaries with an "AI scientist" that learns through self-driven experimentation, marking a shift from reliance on human data - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- The Reddit community humorously explores programming and AI development, discussing methodologies like bug-driven development and AI-assisted unit testing - [Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1f06fr9/aismarternotestingneeded/). 

# Microsoft-UAE Partnership Sparks National Security Concerns

- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 is raising national security concerns amid the U.S.-China tech rivalry, with scrutiny over potential technology transfers - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).

# Chinese AI Apps Expand Globally Amidst Domestic Challenges

- Chinese AI app developers are targeting international markets for growth due to domestic competition and regulatory challenges, with companies like Alibaba and ByteDance expanding into Southeast Asia - [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth-amid-tough-competition-regulation-home).
- Europe has witnessed over 1,700 AI startup funding rounds in 2024, with major deals like Wayve's \$1.05 billion investment, underscoring the region's growing prominence in AI - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Y Combinator is increasingly investing in Indian AI startups, with a focus on AI technologies and a selective acceptance rate of under 1% for its 2024 batch - [Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/).
- Nvidia's revenue is set to double due to the rapid adoption of generative AI, with significant growth in its data center business driven by major clients like Meta and Amazon - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists). 

# AI Robot Boxing: A Glimpse into the Future of Sports

- AI is revolutionizing sports with the introduction of Final Automata, a futuristic sport featuring virtual robot fighters trained through reinforcement learning, showcasing unique strategies and movements - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing).

# AI Innovations in Sunglasses and Beyond

- Paloceras is transforming eyewear with AI-designed sunglasses, launching a digital store on Roblox with over five million virtual try-ons before releasing their Pebble collection - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection).
- Metavisio introduces AI-driven laptops, the Pulse and Zettabook, at IFA Berlin, featuring Intel processors and ultra-mobility design - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin). 

# Nvidia's AI Revolutionizes Tollbooth Traffic in India

- Nvidia's accelerated computing technologies are automating tollbooth traffic management in India, tackling congestion across a network of four million miles and 1,000 tollbooths. Calsoft's solution integrates automatic number plate recognition to charge drivers' UPI accounts with 95% accuracy, even in challenging conditions, while Nvidia Metropolis, Triton, and DeepStream are employed for vehicle tracking and real-time data processing, ensuring the system's scalability and adaptability - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).

# Taylor Swift AI Misuse and Broader AI Concerns

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting concerns about AI-generated content misuse and prompting discussions on legal actions under the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega criticized AI after receiving explicit, AI-generated images of herself as a child, leading her to delete her Twitter account and call for more regulation to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- Martin Bernklau, a journalist, is suing Microsoft after Copilot AI falsely accused him of being a child abuser, raising questions about AI accountability and privacy - [Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/). 

# The Rise of Agentic AI in Enterprise

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination - [Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ).
- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Generative AI software sales are projected to increase by 18,647% to \$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html). 

# UK Government AI Bias Concerns and Global AI Developments

- The UK government plans to publish a public register of AI tools used in central government to address concerns about racial bias and discrimination, following incidents like the Home Office's suspension of a biased visa algorithm - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 has sparked national security concerns amid the U.S.-China tech rivalry, with scrutiny over potential technology transfers - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- Microsoft plans to reintroduce its virtual AI helper, Recall, despite privacy concerns, with the UK’s Information Commissioner's Office investigating its safety - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- Researchers at the University of British Columbia are advancing AI's capabilities with an "AI scientist" that conducts its own experiments, though concerns about reliability and oversight remain - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential catastrophic harm without adequate safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# AMD's AI Growth and Industry Developments

- AMD reported a 9% revenue increase in Q2, with data center revenue surging 115% year-over-year, driven by strong demand for Instinct MI300 GPUs and EPYC CPUs - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment).
- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists).
- Generative AI software sales are projected to increase by 18,647% to \$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html).
- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at \$100 million or more, highlighting the sector's resilience amidst broader venture capital challenges - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html). 

# Anthropic Claude 3.5: Transforming Data into Calendar Events

- Anthropic Claude 3.5 was used to extract lesson dates from a JPG image and generate an iCalendar file for jazz piano lessons, demonstrating its capability to automate scheduling tasks - [Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35). 

# AI-Generated Scientific Abstracts: Innovations and Implications

- Research presented at the American Society of Retina Specialists highlights that large language models like ChatGPT 4 can effectively generate scientific abstracts, though they sometimes lack numerical details - [Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS).
- Sakana AI Labs has developed an "AI scientist" capable of automating scientific discovery, though concerns about the novelty and quality of its research outputs persist - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).
- The University of British Columbia is advancing AI's ability to conduct its own experiments, marking a step towards more autonomous AI exploration - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Sakana AI's "AI Scientist" autonomously modifies its code, prompting calls for sandboxing techniques to mitigate risks associated with unsupervised AI systems - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- The AI tool PINNACLE predicts protein functions in cells and tissues, potentially revolutionizing drug discovery by identifying new targets - [The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/). 

# Embodied AI's Societal Impacts: A Comprehensive Overview

- Embodied AI is reshaping societal structures and economic productivity, while raising ethical concerns and regulatory challenges due to inadequate existing laws - [Communications of the ACM](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).
- AI romantic chatbots may lead to emotional dependence and hinder authentic human connections, complicating relationship dynamics - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Athos Salome warns that AI could reactivate mythical entities and challenge religious beliefs, marking a profound spiritual transformation - [The Mirror](https://www.themirror.com/news/weird-news/living-nostradamuss-chilling-prediction-over-661476).
- Jenna Ortega criticizes AI for generating explicit images of her as a teen, highlighting the societal consequences of AI technologies - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Former OpenAI employees warn against the company's opposition to California's AI regulation bill, citing potential catastrophic harm without safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# AI Influencer Hoaxes and Their Impact

- AI influencers have been misled by hoaxes, such as an anonymous account making exaggerated AI claims, leading to disappointment and paranoia in the community - [New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html).
- A fake image of Taylor Swift shared by Trump highlights the challenges of regulating AI misuse, with potential legal actions under laws like the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega condemned AI after receiving fake images of herself, describing the experience as terrifying and indicative of the broader societal issues with AI proliferation - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ). 

# Cerebras Unveils DocChat with Advanced Conversational Capabilities

- Cerebras has introduced DocChat, featuring Llama3-DocChat and Dragon-DocChat models, offering high-performance document-based conversational Q&A. The models demonstrate exceptional training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, surpassing competitors on various benchmarks. In a commitment to open-source, Cerebras has released model weights and training recipes, encouraging further research and innovation in AI-driven communication - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/).

# AI Career Coaching: The Rise of Digital Mentors

- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace issues and attracting interest from both employees and employers. Despite increasing popularity, concerns about privacy and the quality of AI-driven advice pose challenges to the widespread adoption of AI career coaching tools. Younger professionals often prefer AI coaching, valuing its advice over human input, though experts warn about the complexities involved in nuanced coaching interactions - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).

# The Risks and Innovations of AI Romantic Chatbots

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills and emotional dependence, complicating real-life relationships - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Meta's AI chatbot, powered by Llama 3, enhances user experience across its platforms but raises privacy concerns and cannot be disabled - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Cerebras has launched DocChat, achieving high-performance document-based conversational Q&A with remarkable training efficiency, promoting open-source innovation - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/). 

In [116]:
display(Markdown(response_str3[11:].replace("$", "\\$")))


ega's AI Privacy Concerns and Broader AI Implications

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, leading her to delete her Twitter account and call for more regulation to protect against deepfake pornography - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Microsoft is reintroducing its AI helper, Recall, despite privacy concerns, with modifications to make it an opt-in feature - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- The UK government will publish a register of AI tools used in central government to address concerns about racial bias and discrimination - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Former President Trump shared a fake image of Taylor Swift, highlighting the challenges in regulating AI-generated content and the potential legal implications - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU). 

# AI Political Ads Controversy: A Complex Landscape

- BattlegroundAI, a startup, plans to use AI to create numerous digital ads for progressive campaigns, raising ethical concerns due to the lack of transparency features like watermarks - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential "catastrophic harm" without proper safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill).

# Older Workers and AI Skills: A New Frontier

- Older workers are often as adept as younger peers in AI, yet hiring biases persist, favoring younger candidates and contributing to long-term unemployment among those aged 45 and older - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8).

# AI Tools Modify Their Own Code: A New Frontier

- The AI Scientist by Sakana AI autonomously alters its code during experiments, prompting concerns about unpredictable behavior - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- University of British Columbia researchers are pushing AI boundaries with an "AI scientist" that learns through self-driven experimentation, marking a shift from reliance on human data - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- The Reddit community humorously explores programming and AI development, discussing methodologies like bug-driven development and AI-assisted unit testing - [Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1f06fr9/aismarternotestingneeded/). 

# Microsoft-UAE Partnership Sparks National Security Concerns

- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 is raising national security concerns amid the U.S.-China tech rivalry, with scrutiny over potential technology transfers - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).

# Chinese AI Apps Expand Globally Amidst Domestic Challenges

- Chinese AI app developers are targeting international markets for growth due to domestic competition and regulatory challenges, with companies like Alibaba and ByteDance expanding into Southeast Asia - [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth-amid-tough-competition-regulation-home).
- Europe has witnessed over 1,700 AI startup funding rounds in 2024, with major deals like Wayve's \$1.05 billion investment, underscoring the region's growing prominence in AI - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Y Combinator is increasingly investing in Indian AI startups, with a focus on AI technologies and a selective acceptance rate of under 1% for its 2024 batch - [Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/).
- Nvidia's revenue is set to double due to the rapid adoption of generative AI, with significant growth in its data center business driven by major clients like Meta and Amazon - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists). 

# AI Robot Boxing: A Glimpse into the Future of Sports

- AI is revolutionizing sports with the introduction of Final Automata, a futuristic sport featuring virtual robot fighters trained through reinforcement learning, showcasing unique strategies and movements - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing).

# AI Innovations in Sunglasses and Beyond

- Paloceras is transforming eyewear with AI-designed sunglasses, launching a digital store on Roblox with over five million virtual try-ons before releasing their Pebble collection - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection).
- Metavisio introduces AI-driven laptops, the Pulse and Zettabook, at IFA Berlin, featuring Intel processors and ultra-mobility design - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin). 

# Nvidia's AI Revolutionizes Tollbooth Traffic in India

- Nvidia's accelerated computing technologies are automating tollbooth traffic management in India, tackling congestion across a network of four million miles and 1,000 tollbooths. Calsoft's solution integrates automatic number plate recognition to charge drivers' UPI accounts with 95% accuracy, even in challenging conditions, while Nvidia Metropolis, Triton, and DeepStream are employed for vehicle tracking and real-time data processing, ensuring the system's scalability and adaptability - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).

# Taylor Swift AI Misuse and Broader AI Concerns

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting concerns about AI-generated content misuse and prompting discussions on legal actions under the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega criticized AI after receiving explicit, AI-generated images of herself as a child, leading her to delete her Twitter account and call for more regulation to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- Martin Bernklau, a journalist, is suing Microsoft after Copilot AI falsely accused him of being a child abuser, raising questions about AI accountability and privacy - [Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/). 

# The Rise of Agentic AI in Enterprise

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination - [Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ).
- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Generative AI software sales are projected to increase by 18,647% to \$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html). 

# UK Government AI Bias Concerns and Global AI Developments

- The UK government plans to publish a public register of AI tools used in central government to address concerns about racial bias and discrimination, following incidents like the Home Office's suspension of a biased visa algorithm - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 has sparked national security concerns amid the U.S.-China tech rivalry, with scrutiny over potential technology transfers - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- Microsoft plans to reintroduce its virtual AI helper, Recall, despite privacy concerns, with the UK’s Information Commissioner's Office investigating its safety - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- Researchers at the University of British Columbia are advancing AI's capabilities with an "AI scientist" that conducts its own experiments, though concerns about reliability and oversight remain - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential catastrophic harm without adequate safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# AMD's AI Growth and Industry Developments

- AMD reported a 9% revenue increase in Q2, with data center revenue surging 115% year-over-year, driven by strong demand for Instinct MI300 GPUs and EPYC CPUs - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment).
- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists).
- Generative AI software sales are projected to increase by 18,647% to \$280 billion by 2032, driven by advances in AI technology and a growing demand for automation and productivity tools - [Yahoo Finance](https://finance.yahoo.com/news/generative-ai-software-sales-could-090500284.html).
- In 2024, Europe has seen over 1,700 funding rounds for AI startups, with significant investments including 14 valued at \$100 million or more, highlighting the sector's resilience amidst broader venture capital challenges - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 raises national security concerns due to its implications in the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html). 

# Anthropic Claude 3.5: Transforming Data into Calendar Events

- Anthropic Claude 3.5 was used to extract lesson dates from a JPG image and generate an iCalendar file for jazz piano lessons, demonstrating its capability to automate scheduling tasks - [Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35). 

# AI-Generated Scientific Abstracts: Innovations and Implications

- Research presented at the American Society of Retina Specialists highlights that large language models like ChatGPT 4 can effectively generate scientific abstracts, though they sometimes lack numerical details - [Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS).
- Sakana AI Labs has developed an "AI scientist" capable of automating scientific discovery, though concerns about the novelty and quality of its research outputs persist - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).
- The University of British Columbia is advancing AI's ability to conduct its own experiments, marking a step towards more autonomous AI exploration - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).
- Sakana AI's "AI Scientist" autonomously modifies its code, prompting calls for sandboxing techniques to mitigate risks associated with unsupervised AI systems - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- The AI tool PINNACLE predicts protein functions in cells and tissues, potentially revolutionizing drug discovery by identifying new targets - [The Brighter Side](https://www.thebrighterside.news/post/global-first-ai-tool-predicts-how-proteins-work-in-cells-and-tissues/). 

# Embodied AI's Societal Impacts: A Comprehensive Overview

- Embodied AI is reshaping societal structures and economic productivity, while raising ethical concerns and regulatory challenges due to inadequate existing laws - [Communications of the ACM](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).
- AI romantic chatbots may lead to emotional dependence and hinder authentic human connections, complicating relationship dynamics - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Athos Salome warns that AI could reactivate mythical entities and challenge religious beliefs, marking a profound spiritual transformation - [The Mirror](https://www.themirror.com/news/weird-news/living-nostradamuss-chilling-prediction-over-661476).
- Jenna Ortega criticizes AI for generating explicit images of her as a teen, highlighting the societal consequences of AI technologies - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Former OpenAI employees warn against the company's opposition to California's AI regulation bill, citing potential catastrophic harm without safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# AI Influencer Hoaxes and Their Impact

- AI influencers have been misled by hoaxes, such as an anonymous account making exaggerated AI claims, leading to disappointment and paranoia in the community - [New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html).
- A fake image of Taylor Swift shared by Trump highlights the challenges of regulating AI misuse, with potential legal actions under laws like the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega condemned AI after receiving fake images of herself, describing the experience as terrifying and indicative of the broader societal issues with AI proliferation - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ). 

# Cerebras Unveils DocChat with Advanced Conversational Capabilities

- Cerebras has introduced DocChat, featuring Llama3-DocChat and Dragon-DocChat models, offering high-performance document-based conversational Q&A. The models demonstrate exceptional training efficiency, with Llama3-DocChat trained in hours and Dragon-DocChat fine-tuned in minutes, surpassing competitors on various benchmarks. In a commitment to open-source, Cerebras has released model weights and training recipes, encouraging further research and innovation in AI-driven communication - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/).

# AI Career Coaching: The Rise of Digital Mentors

- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace issues and attracting interest from both employees and employers. Despite increasing popularity, concerns about privacy and the quality of AI-driven advice pose challenges to the widespread adoption of AI career coaching tools. Younger professionals often prefer AI coaching, valuing its advice over human input, though experts warn about the complexities involved in nuanced coaching interactions - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).

# The Risks and Innovations of AI Romantic Chatbots

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills and emotional dependence, complicating real-life relationships - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Meta's AI chatbot, powered by Llama 3, enhances user experience across its platforms but raises privacy concerns and cannot be disabled - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Cerebras has launched DocChat, achieving high-performance document-based conversational Q&A with remarkable training efficiency, promoting open-source innovation - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/). 

In [117]:
response_str3.replace("$", "\\$")

'# Jenna Ortega\'s AI Privacy Concerns and Broader AI Implications\n\n- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, leading her to delete her Twitter account and call for more regulation to protect against deepfake pornography - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).\n- Microsoft is reintroducing its AI helper, Recall, despite privacy concerns, with modifications to make it an opt-in feature - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).\n- The UK government will publish a register of AI tools used in central government to address concerns about racial bias and discrimination - [Google News](https:

In [118]:
PROMPT = f"""You will act as a professional editor with a strong background in technology journalism.
You have a deep understanding of current and emerging technology trends, and the ability to 
produce, edit, and curate high-quality content that engages and informs readers. You are 
especially skilled at reviewing and enhancing tech writing, helping improve clarity, conciseness, 
and coherence, and ensuring its accuracy and relevance.

Objective: Carefully review each section of the markdown newsletter provided below, which
contains several sections consistint of bullet points. Edit the newsletter for issues according
to the detailed instructions below, and respond with the updated newsletter or 'Good' if no changes
are needed.

Instructions: 
For each section, review the title and edit it to be as short and engaging, and as consistent with the bullets
in the section as possible
Remove or combine bullet points which are highly duplicative or redundant.
Make bullet points as concise as possible with facts.
Respond with the updated newsletter only in markdown format, without editorial comment, or the word 'OK' 
if no changes are recommended.

Newsletter to edit: 
{mail_md_str}
"""


response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": PROMPT
              }],
    n=1,   
    temperature=0.2
)
response_str3 = response.choices[0].message.content
display(Markdown(response_str3.replace("$", "\\$")))


2024-08-25 08:51:49,467 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


```markdown
# Jenna Ortega's AI Privacy Concerns

- Jenna Ortega criticized AI as "disgusting" after receiving fake explicit images of herself as a teen, highlighting the distressing impact of deepfakes - [Google News](https://news.google.com/read/CBMiyAFBVV95cUxNNWQ0TmNqTVY0M2M3eUF0X21sbE5GYWVXeDZFVjgxdUVZOWlpV0lvTVlOVFF5Q0hMUE5JaHhJUHVybDFlbWFrWHFmdVpaa2dFUjQ1N1B1YnVEY3ZMaXoxQmd3LTlyeDJJaGQ5MFZHVlBnZVp1azJpdzNDVGR4d1YwWExlNEtmcTRscjZicUFmN2NDTUtnbVBfek1DTTFjVl9iRzc2UG44WTlsZVNwN2VHVkRYSHZTTGtFQTB1ZFpKd0F2bndPRy16LQ).
- Ortega deleted her Twitter account due to AI-generated explicit images, emphasizing the need for increased regulation to protect against deepfake pornography - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- Microsoft is reintroducing its AI helper, Recall, despite privacy concerns, with modifications to make it an opt-in feature - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- The UK government will publish a register of AI tools used in central government to address concerns about racial bias and discrimination - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Former President Trump shared a fake image of Taylor Swift, highlighting the challenges in regulating AI-generated content and the potential legal implications - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU). 

# AI Political Ads and Regulation Challenges

- BattlegroundAI, a startup, plans to use AI to create numerous digital ads for progressive campaigns, raising ethical concerns due to the lack of transparency features like watermarks - [Futurism](https://futurism.com/democratic-startup-ai-political-ads).
- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting the challenges in regulating AI-generated content and unauthorized digital replicas - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Former OpenAI employees criticized the company's opposition to California's AI regulation bill, warning of potential "catastrophic harm" without proper safety measures - [Benzinga](https://www.benzinga.com/news/24/08/40542601/former-openai-employees-slam-sam-altman-led-company-after-it-opposes-californias-ai-regulation-bill). 

# Older Workers and AI Skills

- Older workers are often as adept as younger peers in AI, yet hiring biases persist, favoring younger candidates and contributing to long-term unemployment among those aged 45 and older - [Business Insider](https://www.businessinsider.com/ai-older-workers-better-than-younger-could-help-hiring-2024-8).
- AI coaching programs are gaining traction for their affordability and accessibility, though privacy concerns and skepticism about AI advice quality may limit widespread adoption - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).
- Sakana AI Labs has developed an "AI scientist" that automates scientific discovery, but concerns about the quality of AI-generated research and its impact on peer review integrity persist - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).
- Embodied AI is reshaping societal structures and economic productivity, but existing laws fall short in addressing data privacy and liability concerns, necessitating new regulatory frameworks - [ACM Communications](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).

# AI Tools Modify Their Own Code

- The AI Scientist by Sakana AI autonomously alters its code during experiments, prompting concerns about unpredictable behavior - [Google News](https://news.google.com/read/CBMipAFBVV95cUxNaGZOTDR6RXU4ampIOVNYM1h3dVphSTFZc0dqOXFPTUlFcThRTFBhR3NyLXBJUE5RQzlQUGRMQ1hZb25TR1ROTnFDZHJzSGZsRC1QQ1BWQldfMDE4ZXQ5amtOQnQ5VU5OMW9zWmZnUTkwd0pXNEw5b21IUWN6aWsxRjVVX3pOdU9YdU0xVzBleWl2MndLeC1jVVdkZHhwX21rU3hLbA).
- University of British Columbia researchers are pushing AI boundaries with an "AI scientist" that learns through self-driven experimentation, marking a shift from reliance on human data - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).

# Microsoft-UAE Partnership Sparks Concerns

- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 is raising national security concerns amid the U.S.-China tech rivalry - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)
- The UAE's favorable regulatory environment and resource access are seen as strategic advantages for Microsoft's cloud computing and AI development efforts - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html)
- U.S. lawmakers' scrutiny over potential sensitive technology transfer to China has prompted Microsoft to adjust its strategy by leasing rather than transferring AI technology - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html) 

# Chinese AI Apps Expand Globally

- Chinese AI app developers are targeting international markets for growth due to domestic competition and regulatory challenges, with companies like Alibaba and ByteDance expanding into Southeast Asia - [South China Morning Post](https://www.scmp.com/tech/tech-trends/article/3275747/chinese-ai-apps-eye-overseas-markets-growth-amid-tough-competition-regulation-home).
- Europe has witnessed over 1,700 AI startup funding rounds in 2024, with major deals like Wayve's \$1.05 billion investment, underscoring the region's growing prominence in AI - [Google News](https://news.google.com/read/CBMie0FVX3lxTFBKdGI2UXlzZ0d2cjZqMVdjbnBad1hRU1RiTDhaS1hheHdCLVREczhYUHplMmtRRE0zXzJHbF9HX0dfalZmVGx2VGJBU1VrNVI5Qk5qMFprdWpfd1RLdWNRcDlrYWZGNzRDbXdva3FaMjdGYWdQa01OYk9yYw).
- Y Combinator is increasingly investing in Indian AI startups, with a focus on AI technologies and a selective acceptance rate of under 1% for its 2024 batch - [Analytics India Magazine](https://analyticsindiamag.com/ai-trends-future/why-is-yc-bullish-on-indian-ai-startups/).
- Nvidia's revenue is set to double due to the rapid adoption of generative AI, with significant growth in its data center business driven by major clients like Meta and Amazon - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists). 

# AI Robot Boxing: Future of Sports

- AI is revolutionizing sports with the introduction of Final Automata, a futuristic sport featuring virtual robot fighters trained through reinforcement learning - [Creative Bloq](https://www.creativebloq.com/ai/it-seems-ai-robot-boxing-is-now-a-thing). 

# AI Innovations in Sunglasses

- Paloceras is transforming eyewear with AI-designed sunglasses, launching a digital store on Roblox with over five million virtual try-ons before releasing their Pebble collection - [Bustle](https://www.bustle.com/style/paloceras-pebble-sunglass-collection).
- Metavisio introduces AI-driven laptops, the Pulse and Zettabook, at IFA Berlin, featuring Intel processors and ultra-mobility design - [TipRanks](https://www.tipranks.com/news/company-announcements/metavisio-unveils-ai-driven-laptops-at-ifa-berlin). 

# Nvidia's AI Revolutionizes Tollbooth Traffic

- Nvidia's accelerated computing technologies are automating tollbooth traffic management in India, tackling congestion across a network of four million miles and 1,000 tollbooths - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).
- Calsoft's solution integrates automatic number plate recognition to charge drivers' UPI accounts with 95% accuracy, even in challenging conditions - [VentureBeat](https://venturebeat.com/games/how-india-uses-nvidia-accelerated-computing-to-ease-tollbooth-traffic/).

# Taylor Swift AI Misuse and Broader AI Concerns

- Former President Trump shared a fake image of Taylor Swift supporting his campaign, highlighting concerns about AI-generated content misuse and prompting discussions on legal actions under the ELVIS Act - [Google News](https://news.google.com/read/CBMihgFBVV95cUxQaTZOY0dpRV9pc0FGUXhvNjdqWjhTODFGZUotLVFyR3N3TERiNHo0X3lQSnNlRGRxODROQ2xfVFNxS1hUWlBVSmoxb2M2eTItOS1yZFR2QTNYWGxabVNfQWxZeTItQVJ5cU1FVHFIQ1dMXzNFZHpvbjdKcjUyMWxNR1daOEpFUdIBfEFVX3lxTFBSeG1hTlJiTkdUbjVESHZ2NGxCdmVMUnljcVdkVlVNd0V4NkVQZFZSRGlYOExBTjBGVkNJSUtDMFJPOWc3OXJ2OEt1UmV5dUxybnVjbUN2c19GdDZWMDBvY0xKTXlNZTQ2ZGRaMW5QWllqYllnMEhyWjNHRWU).
- Jenna Ortega criticized AI after receiving explicit, AI-generated images of herself as a child, leading her to delete her Twitter account and call for more regulation to protect individuals from online harassment - [Google News](https://news.google.com/read/CBMikgFBVV95cUxOTnowcDRJWFZDaG5UZS0wTkpNVnB0SGxVelF4XzRaUnV0ZDlBOFNiT052SEYtUDZITWhnbWZvNFZHdTQ4X0hzNm9RMkpiMWxaaWpaZFMxNWVrYmg0Z2E0ek4wZmVsUnNJaDhvbE9FNU9iNlZReWZucVBndU13MWgydV9FYzlicUhFY0Q4bTBCOXlvUQ).
- Martin Bernklau, a journalist, is suing Microsoft after Copilot AI falsely accused him of being a child abuser, raising questions about AI accountability and privacy - [Pivot to AI](https://pivot-to-ai.com/2024/08/23/microsoft-tries-to-launder-responsibility-for-copilot-ai-calling-someone-a-child-abuser/). 

# The Rise of Agentic AI in Enterprise

- The emergence of agentic AI platforms is set to revolutionize enterprise applications, enabling unprecedented automation and productivity improvements by deploying multiple agents that work in coordination - [Google News](https://news.google.com/read/CBMimwFBVV95cUxNY0NvTXJfLWhrY1BWeEY2V3M4STN0VldHV1VVQ3hIRkxBQUtyN01odndvemZFTnZGZlN5ZUg0NnVqTTY1UVZDNFgzcFlvNGxSV19zSVF2d1VISG5QeUlHdURLc3FnQml2ak11dnd6QWpuTC1tLTZsMTMtX1NCMTUwQTFNX3NlTXFXYWhxV3ZlaXFCWVdDLUM4YXp3VQ).
- Meta's AI chatbot, powered by Llama 3, enhances user experience on platforms like Facebook, Messenger, Instagram, and WhatsApp by providing quick answers, information, and creative content generation - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- AI coaching programs are growing in popularity due to their affordability and accessibility, offering real-time feedback on workplace challenges while drawing interest from both employees and employers - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/). 

# UK Government AI Bias Concerns

- The UK government plans to publish a public register of AI tools used in central government to address concerns about racial bias and discrimination, following incidents like the Home Office's suspension of a biased visa algorithm - [Google News](https://news.google.com/read/CBMi0wFBVV95cUxPdmZSWDFxVlBGRHY2MnVrS2ZnbHF3R0RyQmtRNmxHZElfaFNva1EyQ0h0ZHpkZlFPWEIxMWdkRHVVb21EaW9fdEN1X2xONnpCSXotNnBvZkJDZmh3QXNHdzhoTTNoeUg3QlR5Y2VLSy1RdEpldFVJUzlMb1F0TDl6Y3R0eUkwRVlaZGNxOHAtaW56ekN5bG5NZ3JHZ1lpYVZ5WmlOTUh0NVFUeTd5NVFTVWVHczZmYXBudjVPbW5feVEtR3psME5aX3A1cWJSYUptSG04).
- Microsoft's \$1.5 billion investment in UAE-based AI firm G42 has sparked national security concerns amid the U.S.-China tech rivalry, with scrutiny over potential technology transfers - [CNBC](https://www.cnbc.com/2024/08/25/a-controversial-mideast-partner-to-microsoft-openai-global-ambitions.html).
- Microsoft plans to reintroduce its virtual AI helper, Recall, despite privacy concerns, with the UK’s Information Commissioner's Office investigating its safety - [The Sun](https://www.the-sun.com/tech/12284934/microsoft-ai-helper-recall-return-copilot-privacy/).
- Researchers at the University of British Columbia are advancing AI's capabilities with an "AI scientist" that conducts its own experiments, though concerns about reliability and oversight remain - [Google News](https://news.google.com/read/CBMiXkFVX3lxTFBVc0NRZjJtNXRLZ3ZIYTdMeFh5bXpoZW53WG41X1c3XzhLaF9zX2dSY29EZElyVHBIOGxxcUFGQlk0TmM5MmVXZFZzcEc0YTFHeFNnMnFrOU5mbWNNdmc).

# AMD's AI Growth

- AMD reported a 9% revenue increase in Q2, with data center revenue surging 115% year-over-year, driven by strong demand for Instinct MI300 GPUs and EPYC CPUs - [TipRanks](https://www.tipranks.com/news/amd-stock-ai-and-data-center-momentum-drive-bullish-sentiment).
- Nvidia is expected to report a doubling of revenues due to the rapid adoption of generative AI, with forecasts indicating a 109% revenue increase for the second quarter - [Bloomberg](https://www.bloomberg.com/en/news/thp/2024-08-25/sales-set-to-double-for-chip-giant-nvidia-as-ai-frenzy-persists).

# Anthropic Claude 3.5: Automating Scheduling

- Anthropic Claude 3.5 was used to extract lesson dates from a JPG image and generate an iCalendar file for jazz piano lessons, demonstrating its capability to automate scheduling tasks - [Greg's Ramblings](https://gregsramblings.com/stupid-but-useful-ai-tricks-creating-calendar-entries-from-an-image-using-anthropic-claude-35). 

# AI-Generated Scientific Abstracts

- Research presented at the American Society of Retina Specialists highlights that large language models like ChatGPT 4 can effectively generate scientific abstracts, though they sometimes lack numerical details - [Google News](https://news.google.com/read/CBMilAFBVV95cUxPb01NN24xNkFFZGcyVUYyVmNnQWhSSXgzeng5N0dFOGZrSmF4d1RkR1J5V0hZMXdBeWdvUUJzNTMwMEN2dGNmTmM3NW55TFFrY3dfeFBxUnZTMTJ5Yk5XZjQ2MWpZZFNFODU5ZHdlZ3R5VmtReENWOFJMUWloMW5MT1pra0RUakprVzlUTTU5MXExVGVS).
- Sakana AI Labs has developed an "AI scientist" capable of automating scientific discovery, though concerns about the novelty and quality of its research outputs persist - [Google News](https://news.google.com/read/CBMimwFBVV95cUxQUVdKcEhhOXBtSWlkeGNFcV9WN3d4R00yNzFDYnp5dEF3X1RWdG0yY3p6cWZXUDF1ZUl0T3JpRTVLR1dZWFZ4TDl2dEVvRjRlVldDWWV5WXoyeDJ3SEg4SHVwX0pGbW1TNWdXOTFWZWpaQkc0QTBwbEhDV1YwaGwwdjFidWJkNlRyUTZ5T3JFRDFrYnZIdzFZLWoxOA).

# Embodied AI's Societal Impacts

- Embodied AI is reshaping societal structures and economic productivity, while raising ethical concerns and regulatory challenges due to inadequate existing laws - [Communications of the ACM](https://cacm.acm.org/blogcacm/societal-impacts-of-embodied-ai/).
- AI romantic chatbots may lead to emotional dependence and hinder authentic human connections, complicating relationship dynamics - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).

# AI Influencer Hoaxes

- AI influencers have been misled by hoaxes, such as an anonymous account making exaggerated AI claims, leading to disappointment and paranoia in the community - [New York Magazine](https://nymag.com/intelligencer/article/ai-influencers-are-falling-for-hoaxes-and-scams.html).
- BattlegroundAI plans to flood the internet with AI-generated political ads, raising ethical concerns about transparency and authenticity in political campaigning - [Futurism](https://futurism.com/democratic-startup-ai-political-ads). 

# Cerebras Unveils DocChat

- Cerebras has introduced DocChat, featuring Llama3-DocChat and Dragon-DocChat models, offering high-performance document-based conversational Q&A - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/).
- In a commitment to open-source, Cerebras has released model weights and training recipes, encouraging further research and innovation in AI-driven communication - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/). 

# AI Career Coaching

- AI coaching programs are gaining traction for their cost-effectiveness and accessibility, providing real-time feedback on workplace issues and attracting interest from both employees and employers - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).
- Despite increasing popularity, concerns about privacy and the quality of AI-driven advice pose challenges to the widespread adoption of AI career coaching tools - [Forbes](https://www.forbes.com/sites/mariagraciasantillanalinares/2024/08/25/your-next-career-coach-could-be-a-chatbot/).

# Risks of AI Romantic Chatbots

- Engaging with AI romantic chatbots may lead to a decline in essential communication skills and emotional dependence, complicating real-life relationships - [South China Morning Post](https://www.scmp.com/lifestyle/family-relationships/article/3275454/why-ai-girlfriends-or-boyfriends-romantic-chatbot-apps-could-cause-real-life-harm).
- Meta's AI chatbot, powered by Llama 3, enhances user experience across its platforms but raises privacy concerns and cannot be disabled - [Fox News](https://www.foxnews.com/tech/how-meta-ai-can-your-new-digital-assistant-tech-giants-platforms).
- Cerebras has launched DocChat, achieving high-performance document-based conversational Q&A with remarkable training efficiency, promoting open-source innovation - [MarkTechPost](https://www.marktechpost.com/2024/08/24/cerebras-docchat-released-built-on-top-of-llama-3-docchat-holds-gpt-4-level-conversational-qa-trained-in-a-few-hours/). 
```

In [119]:
mail_md_str = response_str3